Now, based on the training and development set, think of a better design for developing an improved sentiment analysis system for tweets using any model you like. Please explain clearly the
model/method that you used for designing the new system. We will check your code and may call you
for an interview if we have questions about your code. Please run your system on the development set
RU/dev.in and ES/dev.in. Write your outputs to RU/dev.p4.out and ES/dev.p4.out.
Report the precision, recall and F scores of your new systems for these two languages

Stopwords for ES taken from: https://github.com/stopwords-iso/stopwords-es/blob/master/stopwords-es.txt
Stopwords for RU taken from: https://github.com/stopwords-iso/stopwords-ru/blob/master/stopwords-ru.txt 


In [3]:
import os
import math
import sys

In [61]:
def read_data(path):
    dataset = []
    f=open(path, "r", encoding="utf-8")
    training_set = f.readlines()
    for line in training_set:
        line = line.rstrip('\n')
        line = line.rpartition('')
        line = list(line)
        del line[1]
        del line[1]
        if line !=['','']:
            dataset.append(line)
    return dataset

In [5]:
#Open file path for ES and RU Stop words
with open(r'C:\SUTD FILES\Term 5\ML PROJECT\MachineLearningT5\Data\ES\stopwords-es.txt', "r", encoding = "utf-8") as f:
    STOP_words_ES = f.read().splitlines()

with open(r'C:\SUTD FILES\Term 5\ML PROJECT\MachineLearningT5\Data\RU\stopwords-ru.txt', "r", encoding = "utf-8") as f:
    STOP_words_RU = f.read().splitlines()
#STOP_words_ES
#STOP_words_RU

#Open file path for ES and RU Train files
with open(r'C:\SUTD FILES\Term 5\ML PROJECT\MachineLearningT5\Data\ES\train', "r", encoding = "utf-8") as f:
    ES_Train = f.read().splitlines()

with open(r'C:\SUTD FILES\Term 5\ML PROJECT\MachineLearningT5\Data\RU\train', "r", encoding = "utf-8") as f:
    RU_Train = f.read().splitlines()
#ES_Train
#RU_Train

#open file path for ES and RU Test files
with open(r'C:\SUTD FILES\Term 5\ML PROJECT\MachineLearningT5\Data\ES\dev.in', "r", encoding = "utf-8") as f:
    ES_Test = f.read().splitlines()

with open(r'C:\SUTD FILES\Term 5\ML PROJECT\MachineLearningT5\Data\RU\dev.in', "r", encoding = "utf-8") as f:
    RU_Test = f.read().splitlines()
#ES_Test
#RU_Test

#Open file path for ES and RU P4 out
with open(r'C:\SUTD FILES\Term 5\ML PROJECT\MachineLearningT5\Data\ES\dev.p4.out', "r", encoding = "utf-8") as f:
    ES_dev_write = f.read().splitlines()

with open(r'C:\SUTD FILES\Term 5\ML PROJECT\MachineLearningT5\Data\RU\dev.p4.out', "r", encoding = "utf-8") as f:
    RU_dev_write = f.read().splitlines()
#ES_dev_write
#RU_dev_write

In [6]:
#convert stop words into list 
def read_stopwords(stopwords_path):
    stopwords_list = []
    for line in stopwords_path:
        line2 = line.strip("\n")
        stopwords_list.append(line2)
    #Remove empty lists within list
    stop_words = [ele for ele in stopwords_list if ele != []]
    return stop_words

print(read_stopwords(STOP_words_RU))
print(read_stopwords(STOP_words_ES))

['c', 'а', 'алло', 'без', 'белый', 'близко', 'более', 'больше', 'большой', 'будем', 'будет', 'будете', 'будешь', 'будто', 'буду', 'будут', 'будь', 'бы', 'бывает', 'бывь', 'был', 'была', 'были', 'было', 'быть', 'в', 'важная', 'важное', 'важные', 'важный', 'вам', 'вами', 'вас', 'ваш', 'ваша', 'ваше', 'ваши', 'вверх', 'вдали', 'вдруг', 'ведь', 'везде', 'вернуться', 'весь', 'вечер', 'взгляд', 'взять', 'вид', 'видел', 'видеть', 'вместе', 'вне', 'вниз', 'внизу', 'во', 'вода', 'война', 'вокруг', 'вон', 'вообще', 'вопрос', 'восемнадцатый', 'восемнадцать', 'восемь', 'восьмой', 'вот', 'впрочем', 'времени', 'время', 'все', 'все еще', 'всегда', 'всего', 'всем', 'всеми', 'всему', 'всех', 'всею', 'всю', 'всюду', 'вся', 'всё', 'второй', 'вы', 'выйти', 'г', 'где', 'главный', 'глаз', 'говорил', 'говорит', 'говорить', 'год', 'года', 'году', 'голова', 'голос', 'город', 'да', 'давать', 'давно', 'даже', 'далекий', 'далеко', 'дальше', 'даром', 'дать', 'два', 'двадцатый', 'двадцать', 'две', 'двенадцатый', 'д

In [7]:
#removing stop words in dataset and labelling the stop words as 0
def remove_stopwords(dataset,stop_words,symbols_list=[]):
    original_dataset = []
    for line in dataset:
        # if to include \n
        if len(line) == 1:
            original_dataset.append("\n")
        else:
            line = line.rstrip('\n') #split into the "text" and the "tag/label" using line.split()
            line = line.rpartition(' ')
            line = list(line)
            del line[1]
            if line != ['', '']:
                if line[0] in stop_words: #if the word belongs to the list of stopwords or list of symbols, assign the label O
                    line[1] = "O" #assign label O
                elif line[0] in symbols_list:
                    continue
                original_dataset.append(line)
    Edataset = [ele for ele in original_dataset]
    return Edataset
print(ES_Test)
print(remove_stopwords(ES_Test,STOP_words_ES))

['Plato', 'degustación', ':', 'un', 'poco', 'abundante', 'de', 'más', ',', 'pero', 'bien', 'cocinado', '.', '', 'restaurante', 'excelente', 'con', 'carne', 'de', 'alta', 'calidad', '.', '', 'Las', 'posibilidades', 'en', 'el', 'restaurante', 'son', 'fundamentalmente', 'tres', ';', 'carta', 'normal', ',', 'menú', 'degustacion', 'y', 'una', 'opción', 'intermedia', 'que', 'es', 'una', 'selección', 'de', 'primeros', 'y', 'postres', 'y', 'carta', 'para', 'el', 'segundo', '.', '', 'No', 'perderse', 'el', 'sorbete', 'de', 'mojito', '.', '', 'para', 'mi', 'perfecto', '!', '', 'Devolucion', 'a', 'cocina', ',', 'amabilidad', 'de', 'camarera', ',', 'requerimiento', 'de', 'cuenta', 'y', 'adios', '.', '', 'Así', 'como', 'el', 'romesco', ',', 'que', 'era', 'un', 'poco', '"', 'de', 'bote', '"', '.', '', 'Destacar', 'los', 'arroces', ',', 'la', 'caldereta', 'de', 'bogavante', ',', 'las', 'zamburiñas', 'al', 'horno', 'y', 'los', 'platos', 'de', '"', 'picoteo', '"', 'y', 'los', 'pescados', 'en', 'general

In [10]:
def get_symbols(edataset): #to get all of the symbols
    symbol_set=[]
    for element in edataset:
        #print(element)
        if any(not char.isalnum() for char in element):
            #print(symbol_set)
            symbol_set.append(element)
    return symbol_set

#print(ES_Train[0][0])
print(get_symbols(ES_Train))

Estuvimos O
hace O
poco O
mi O
pareja O
y O
yo O
comiendo O
y O
resultó O
todo O
muy O
bien O
, O
tanto O
la O
comida B-positive
, O
el O
vino B-positive
, O
el O
trato B-positive
, O
la O
decoración B-positive
… O
nos O
gustó O
todo O
mucho O
. O

Por O
poner O
algún O
pero O
, O
quizá O
el O
jamón B-negative
no O
era O
todo O
lo O
" O
ibérico O
" O
que O
cabía O
esperar O
. O

Bien O
lo O
sabe O
el O
autor O
del O
blog O
. O
) O

Comida B-positive
exquisita O
. O

Restaurante B-positive
diferente O
, O
creativo O
y O
agradable O
. O

Si O
no O
has O
probado O
sus O
carnes B-positive
te O
estas O
perdiendo O
algo O
muy O
grande O
! O

En O
resumen O
, O
comida B-positive
bien-muy O
bien O
, O
servicio B-positive
correcto O
y O
profesional O

02-12- O
2012 O
elegimos O
este O
restaurante B-negative
por O
los O
comentarios O
, O
pero O
ha O
sido O
una O
Grandisima O
Decepción O
. O

Salimos O
encantadas O
del O
restaurante B-positive
. O

Ubicación O

Comimos O
muy O
bien O
y O
bien O
a

In [ ]:
#remove stopwords
def smooth_labels_transmission(transmission_parameters,alpha_sm=0.01):
    smoothed_dict = transmission_parameters
    for entry in smoothed_dict: #for each dictionary
        for label in smoothed_dict[entry]: #for each label in each dictionary
            if label != 'START':
                smoothed_dict[entry][label] *= (1 - alpha_sm)
                smoothed_dict[entry][label] += (alpha_sm/(len(smoothed_dict[entry])-1)) #add by the smoothing factor/number of labels without START
            # there is no transition to START at all
    return smoothed_dict #return smoothed dictionary

In [ ]:
def smooth_labels(labels, factor=0.1):
	# smooth the labels
	labels *= (1 - factor)
	labels += (factor / labels.shape[1])
	# returned the smoothed labels
	return labels

In [ ]:
def smooth_labels_emission(emission_dict,alpha_sm=0.01):
    smoothed_dict = emission_dict
    for label, word_prob_dict in emission_dict.items():
        for word in word_prob_dict.keys():
            smoothed_dict[label][word] *= (1 - alpha_sm)
            smoothed_dict[label][word] += (alpha_sm/(len(word_prob_dict)))
    return smoothed_dict

In [ ]:
emission_para = {'Estuvimos_O': 0.00011625589583471734, 'hace_O': 0.000448415598219624, 'poco_O': 0.0009300471666777387, 'mi_O': 0.0012123829137049094, 'pareja_O': 0.00023251179166943468, 'y_O': 0.017023184747226465, 'yo_O': 0.0006144954494120774, 'comiendo_O': 0.00018268783631169867, 'resultó_O': 8.303992559622666e-05, 'todo_O': 0.0019265262738324587, 'muy_O': 0.006593370092340397, 'bien_O': 0.0027569255297947254, ',_O': 0.02765229522354348, 'tanto_O': 0.0006643194047698133, 'la_O': 0.012555636750149472, 'comida_B-positive': 0.00525713578872499, 'el_O': 0.010678934431674749, 'vino_B-positive': 0.0002164702971827937, 'trato_B-positive': 0.0013915947676036739, 'decoración_B-positive': 0.00024739462535176425, '…_O': 0.0007639673154852853, 'nos_O': 0.002441373812529064, 'gustó_O': 0.00019929582143094398, 'mucho_O': 0.000896831196439248, '._O': 0.02697136783365442, 'Por_O': 0.0005148475386966054, 'poner_O': 9.964791071547199e-05, 'algún_O': 0.00011625589583471734, 'pero_O': 0.0031887331428951038, 'quizá_O': 8.303992559622666e-05, 'jamón_B-negative': 6.337537233031244e-05, 'no_O': 0.006144954494120773, 'era_O': 0.0008802232113200027, 'lo_O': 0.004500763967315485, '"_O': 0.0006643194047698133, 'ibérico_O': 4.9823955357735996e-05, 'que_O': 0.014050355410881551, 'cabía_O': 3.3215970238490664e-05, 'esperar_O': 9.964791071547199e-05, 'Bien_O': 0.00011625589583471734, 'sabe_O': 9.964791071547199e-05, 'autor_O': 4.9823955357735996e-05, 'del_O': 0.002042782169667176, 'blog_O': 4.9823955357735996e-05, ')_O': 0.002192254035740384, 'Comida_B-positive': 0.0004329405943655874, 'exquisita_O': 0.00016607985119245332, 'Restaurante_B-positive': 0.00015462164084485266, 'diferente_O': 6.643194047698133e-05, 'creativo_O': 3.3215970238490664e-05, 'agradable_O': 0.000697535375008304, 'Si_O': 0.00041519962798113334, 'has_O': 0.00018268783631169867, 'probado_O': 0.00024911977678868, 'sus_O': 0.0002823357470271707, 'carnes_B-positive': 0.0003092432816897053, 'te_O': 0.0008802232113200027, 'estas_O': 4.9823955357735996e-05, 'perdiendo_O': 3.3215970238490664e-05, 'algo_O': 0.0005812794791735866, 'grande_O': 8.303992559622666e-05, '!_O': 0.0026406696339600078, 'En_O': 0.0006809273898890587, 'resumen_O': 0.00011625589583471734, 'bien-muy_O': 3.3215970238490664e-05, 'servicio_B-positive': 0.003803692364783375, 'correcto_O': 0.000697535375008304, 'profesional_O': 0.00018268783631169867, '02-12-_O': 3.3215970238490664e-05, '2012_O': 4.9823955357735996e-05, 'elegimos_O': 6.643194047698133e-05, 'este_O': 0.001295422839301136, 'restaurante_B-negative': 0.0006020660371379682, 'por_O': 0.004185212250049824, 'los_O': 0.004118780309572843, 'comentarios_O': 0.00016607985119245332, 'ha_O': 0.0010130870922739654, 'sido_O': 0.000448415598219624, 'una_O': 0.0038198365774264266, 'Grandisima_O': 3.3215970238490664e-05, 'Decepción_O': 3.3215970238490664e-05, 'Salimos_O': 3.3215970238490664e-05, 'encantadas_O': 3.3215970238490664e-05, 'restaurante_B-positive': 0.001453443423941615, 'Ubicación_O': 3.3215970238490664e-05, 'Comimos_O': 0.00018268783631169867, 'atendidos_O': 4.9823955357735996e-05, 'Los_O': 0.00049823955357736, 'servicios_B-negative': 6.337537233031244e-05, 'en_O': 0.008403640470338139, 'estado_O': 0.00033215970238490664, 'lamentable_O': 9.964791071547199e-05, 'manteles_B-negative': 6.337537233031244e-05, 'copas_B-negative': 6.337537233031244e-05, 'etc_O': 0.00013286388095396266, '<->_O': 0.00021590380655018933, 'Gambas_B-negative': 6.337537233031244e-05, 'resecas_O': 3.3215970238490664e-05, 'sabemos_O': 4.9823955357735996e-05, 'llenaron_O': 3.3215970238490664e-05, 'con_O': 0.005347771208396997, 'trozos_O': 6.643194047698133e-05, 'de_O': 0.016807280940676277, 'esa_O': 0.00018268783631169867, 'sucedanio_B-neutral': 6.40020480655381e-05, 'en_I-neutral': 6.406149903907751e-05, 'forma_I-neutral': 6.406149903907751e-05, 'de_I-neutral': 0.00019218449711723255, 'palo_I-neutral': 6.406149903907751e-05, '-_I-neutral': 6.406149903907751e-05, 'pero_I-neutral': 6.406149903907751e-05, 'con_I-neutral': 0.00016015374759769378, 'sabor_I-neutral': 6.406149903907751e-05, 'a_I-neutral': 9.609224855861628e-05, 'gamba_I-neutral': 6.406149903907751e-05, 'casi_O': 0.000348767687504152, 'caucho-imposible_O': 3.3215970238490664e-05, 'comer_O': 0.00074735933036604, 'supuesto_O': 8.303992559622666e-05, 'ya_O': 0.00099647910715472, 'estoy_O': 8.303992559622666e-05, 'dando_O': 9.964791071547199e-05, 'voces_O': 3.3215970238490664e-05, 'para_O': 0.0031887331428951038, 'avisar_O': 3.3215970238490664e-05, 'a_O': 0.007141433601275493, 'todas_O': 0.00021590380655018933, 'mis_O': 0.00023251179166943468, 'relaciones_O': 3.3215970238490664e-05, 'Borja_O': 3.3215970238490664e-05, 'Alagarmos_O': 3.3215970238490664e-05, 'cena_O': 0.000448415598219624, 'hasta_O': 0.0002657277619079253, 'las_O': 0.002142430080382648, 'tres_O': 0.00018268783631169867, 'tomando_O': 4.9823955357735996e-05, 'copas_O': 0.000149471866073208, 'Ofrecen_O': 3.3215970238490664e-05, 'merluza_O': 6.643194047698133e-05, 'salsa_O': 0.00011625589583471734, 'El_O': 0.0026074536637215174, 'Minicuit_B-positive': 6.184865633794106e-05, 'igual_O': 0.0002823357470271707, 'Me_O': 0.000348767687504152, 'invitaron_O': 6.643194047698133e-05, 'restaurante_O': 0.0014781106756128347, 'cual_O': 0.00011625589583471734, 'desconocia_O': 3.3215970238490664e-05, 'su_O': 0.0014282867202550986, 'existencia_O': 4.9823955357735996e-05, 'De_O': 0.00041519962798113334, 'formas_O': 8.303992559622666e-05, 'decidimos_O': 8.303992559622666e-05, 'quedarnos_O': 4.9823955357735996e-05, 'domingo_O': 0.00013286388095396266, '15_O': 9.964791071547199e-05, 'marzo_O': 6.643194047698133e-05, 'me_O': 0.0024247658274098188, 'desplace_O': 3.3215970238490664e-05, 'desde_O': 0.0003155517172656613, 'galicia_O': 3.3215970238490664e-05, 'unos_O': 0.00049823955357736, 'amigos_O': 0.00038198365774264265, 'competir_O': 3.3215970238490664e-05, 'maraton_O': 3.3215970238490664e-05, 'bcn_O': 6.643194047698133e-05, 'Para_O': 0.00033215970238490664, 'guiris_O': 4.9823955357735996e-05, 'cominda_B-positive': 6.184865633794106e-05, 'impresionante_O': 0.00016607985119245332, 'atencion_O': 0.00016607985119245332, 'aperitivo_O': 0.00011625589583471734, 'es_O': 0.005065435461369827, 'ni_O': 0.000697535375008304, 'normal_O': 0.00021590380655018933, 'bueno_O': 0.0009300471666777387, 'un_O': 0.006078522553643792, 'recomendar_O': 9.964791071547199e-05, 'todos_O': 0.0005148475386966054, 'Calidad_O': 0.00016607985119245332, 'alimentos_B-positive': 9.277298450691158e-05, 'excelente_O': 0.0012622068690626453, 'mesas_O': 0.00021590380655018933, 'se_O': 0.002657277619079253, 'mostraron_O': 3.3215970238490664e-05, 'atentos_O': 0.00016607985119245332, 'momento_O': 0.00033215970238490664, '-_O': 0.00048163156845811467, '10_O': 0.00021590380655018933, 'Excelentes_O': 4.9823955357735996e-05, 'istalaciones_B-positive': 6.184865633794106e-05, 'buena_O': 0.002275293961336611, 'atención_B-positive': 0.0009277298450691159, 'e_O': 0.00016607985119245332, 'inmejorable_O': 0.00023251179166943468, 'cocina_B-positive': 0.00058756223521044, 'digno_O': 4.9823955357735996e-05, 'probar_O': 0.00011625589583471734, 'Las_O': 0.0003155517172656613, 'a_I-positive': 0.000412816360229907, 'la_I-positive': 0.000412816360229907, 'brasa_I-positive': 0.0002857959416976279, 'son_O': 0.0005148475386966054, 'superiores_O': 3.3215970238490664e-05, 'Se_O': 0.00016607985119245332, 'podria_O': 4.9823955357735996e-05, 'mejorar_O': 0.00019929582143094398, 'iluminación_B-negative': 6.337537233031244e-05, 'planta_O': 3.3215970238490664e-05, 'superior_O': 6.643194047698133e-05, 'demasiada_O': 4.9823955357735996e-05, 'luz_O': 3.3215970238490664e-05, 'plana_O': 3.3215970238490664e-05, 'Tienen_O': 4.9823955357735996e-05, 'carta_B-positive': 0.0005566379070414695, 'bastante_O': 0.0005646714940543414, 'extensa_O': 9.964791071547199e-05, 'donde_O': 0.00036537567262339733, 'cada_O': 0.0002657277619079253, 'plato_O': 0.0005314555238158506, 'parece_O': 0.00039859164286188797, 'mejor_O': 0.000847007241081512, 'anterior_O': 0.00011625589583471734, 'Yo_O': 0.0002823357470271707, 'recomiendo_O': 0.0006311034345313226, 'Aunque_O': 9.964791071547199e-05, 'había_O': 0.00039859164286188797, 'fumadores_O': 8.303992559622666e-05, 'mesa_O': 0.0006144954494120774, 'al_O': 0.0018268783631169867, 'lado_O': 0.00019929582143094398, 'sentimos_O': 4.9823955357735996e-05, 'ninguna_O': 0.00013286388095396266, 'molestia_O': 3.3215970238490664e-05, 'durante_O': 9.964791071547199e-05, 'comida_O': 0.000548063508935096, 'Hace_O': 4.9823955357735996e-05, 'años_O': 0.00021590380655018933, 'conozco_O': 6.643194047698133e-05, 'ponerle_O': 4.9823955357735996e-05, 'pega_O': 0.00013286388095396266, 'excesivo_O': 8.303992559622666e-05, 'ruido_B-negative': 0.00015843843082578111, 'mucha_O': 0.0002657277619079253, 'gente_O': 0.0003155517172656613, 'sala_O': 0.00011625589583471734, 'resuena_O': 3.3215970238490664e-05, 'podíamos_O': 4.9823955357735996e-05, 'escucharnos_O': 3.3215970238490664e-05, 'Rapidez_B-positive': 9.277298450691158e-05, 'peticion_O': 3.3215970238490664e-05, 'platos_O': 0.0007307513452467947, 'solucitud_O': 3.3215970238490664e-05, 'cuenta_O': 0.0003155517172656613, 'Lo_O': 0.0006144954494120774, 'unico_O': 9.964791071547199e-05, 'puedo_O': 0.0002823357470271707, 'decir_O': 0.00033215970238490664, 'calidad_O': 0.002391549857171328, 'comimos_O': 0.000149471866073208, 'fue_O': 0.0014282867202550986, 'estupenda_O': 0.00024911977678868, 'sorprendio_O': 9.964791071547199e-05, 'cantidad_O': 0.0006311034345313226, 'mas_O': 0.0008636152262007573, 'escasa_O': 0.00019929582143094398, '(_O': 0.002192254035740384, 'peor_O': 0.00016607985119245332, 'puede_O': 0.00033215970238490664, 'pasar_O': 0.00016607985119245332, 'gusta_O': 0.00023251179166943468, 'opinar_O': 6.643194047698133e-05, 'pasado_O': 0.000298943732146416, 'tiempo_O': 0.00041519962798113334, 'cuando_O': 0.00048163156845811467, 'asimilado_O': 3.3215970238490664e-05, 'experiencia_O': 0.00016607985119245332, 'A_O': 0.00041519962798113334, 'veces_O': 0.00048163156845811467, 'cruda_O': 3.3215970238490664e-05, 'algunas_O': 0.00011625589583471734, 'zonas_O': 3.3215970238490664e-05, 'Servicio_B-positive': 0.0008349568605622043, 'La_O': 0.0024745897827675545, 'bebida_B-negative': 9.506305849546866e-05, 'dejado_O': 8.303992559622666e-05, 'desear_O': 0.00013286388095396266, 'sentidos_O': 4.9823955357735996e-05, 'poca_O': 0.00016607985119245332, 'incluido_O': 8.303992559622666e-05, 'cervezas_O': 3.3215970238490664e-05, 'hemos_O': 0.000448415598219624, 'tenido_O': 0.000149471866073208, 'pagar_O': 8.303992559622666e-05, 'extra_O': 6.643194047698133e-05, 'pequeño_O': 0.00013286388095396266, 'porque_O': 0.0005646714940543414, 'barato_O': 0.00011625589583471734, 'poder_O': 0.00011625589583471734, 'acabar_O': 6.643194047698133e-05, 'menú_O': 0.000597887464292832, 'líquido_O': 4.9823955357735996e-05, 'hacer_O': 0.000348767687504152, 'bajar_O': 6.643194047698133e-05, '“_O': 0.00049823955357736, 'rissoto_O': 3.3215970238490664e-05, '”_O': 0.0005812794791735866, 'hubiera_O': 0.000149471866073208, 'hecho_O': 9.964791071547199e-05, 'delicias_O': 4.9823955357735996e-05, 'presentador_O': 3.3215970238490664e-05, 'Bricomanía_O': 3.3215970238490664e-05, 'capítulo_O': 4.9823955357735996e-05, 'explica_O': 3.3215970238490664e-05, 'como_O': 0.0020593901547864214, 'rebozar_O': 3.3215970238490664e-05, 'paredes_O': 6.643194047698133e-05, 'viejas_O': 3.3215970238490664e-05, 'precioso_O': 6.643194047698133e-05, 'moderno_O': 9.964791071547199e-05, 'acogedor_O': 0.00041519962798113334, 'Como_O': 0.00023251179166943468, 'soy_O': 0.00013286388095396266, 'tierra_O': 8.303992559622666e-05, 'les_O': 0.000348767687504152, 'dije_O': 6.643194047698133e-05, 'ir_O': 0.00043180761310037866, 'calçotada_O': 8.303992559622666e-05, 'probaran_O': 4.9823955357735996e-05, 'Galicia_O': 3.3215970238490664e-05, 'suele_O': 4.9823955357735996e-05, 'JR_O': 3.3215970238490664e-05, 'Buena_O': 0.000298943732146416, 'precio_O': 0.0018767023184747226, 'razonable_O': 6.643194047698133e-05, 'comida_B-neutral': 0.0003200102403276905, 'sin_O': 0.0013784627648973627, 'echar_O': 6.643194047698133e-05, 'cohetes_O': 3.3215970238490664e-05, 'Enhorabuena_O': 8.303992559622666e-05, 'Bal_O': 0.00011625589583471734, 'D_O': 0.000149471866073208, '’_O': 0.000149471866073208, 'Onsera_O': 9.964791071547199e-05, 'conservar_O': 3.3215970238490664e-05, 'estrella_O': 0.00021590380655018933, 'Michelin_O': 9.964791071547199e-05, '2010_O': 6.643194047698133e-05, 'lamento_O': 3.3215970238490664e-05, 'hayan_O': 6.643194047698133e-05, 'quitado_O': 4.9823955357735996e-05, 'Lillas_O': 4.9823955357735996e-05, 'Pastia_O': 4.9823955357735996e-05, 'Hacia_O': 6.643194047698133e-05, 'frio_B-negative': 6.337537233031244e-05, 'Copio_O': 3.3215970238490664e-05, 'escribí_O': 3.3215970238490664e-05, 'unas_O': 0.0003155517172656613, 'semanas_O': 8.303992559622666e-05, 'otra_O': 0.000298943732146416, 'página_O': 0.00011625589583471734, 'web_O': 0.00011625589583471734, 'tras_O': 4.9823955357735996e-05, 'también_O': 0.00041519962798113334, 'dolorasa_O': 3.3215970238490664e-05, 'visita_O': 0.00011625589583471734, 'sótano_B-negative': 6.337537233031244e-05, 'Fui_O': 6.643194047698133e-05, 'dado_O': 0.00013286388095396266, 'impresión_O': 8.303992559622666e-05, 'tapas_O': 6.643194047698133e-05, 'excepcional_O': 0.00013286388095396266, 'gusto_O': 0.00039859164286188797, 'Rapido_O': 3.3215970238490664e-05, 'cómodo_O': 8.303992559622666e-05, 'eficaz_O': 3.3215970238490664e-05, 'sitio_B-neutral': 9.600307209830715e-05, 'está_O': 0.00049823955357736, 'mal_O': 0.0005314555238158506, 'clasifican_O': 3.3215970238490664e-05, 'típica_O': 4.9823955357735996e-05, 'catalana_O': 8.303992559622666e-05, 'dieta_O': 3.3215970238490664e-05, 'mediterránea_O': 3.3215970238490664e-05, 'realidad_O': 6.643194047698133e-05, 'pizzería-grill_O': 3.3215970238490664e-05, 'además_O': 0.00016607985119245332, 'sirven_O': 0.00011625589583471734, 'calçots_O': 6.643194047698133e-05, 'escalibada_O': 4.9823955357735996e-05, 'embutidos_O': 3.3215970238490664e-05, 'No_O': 0.0009300471666777387, 'entiendo_O': 0.00011625589583471734, 'reservaron_O': 3.3215970238490664e-05, 'espacio_O': 6.643194047698133e-05, 'cerrado_O': 0.00011625589583471734, 'salones_O': 6.643194047698133e-05, 'tienen_O': 0.0002823357470271707, 'DecepcionanteCon_O': 3.3215970238490664e-05, 'motivo_O': 6.643194047698133e-05, 'viaje_O': 3.3215970238490664e-05, 'Zaragoza_O': 0.00039859164286188797, 'cenamos_O': 0.00016607985119245332, 'aconsejados_O': 3.3215970238490664e-05, 'hotel_O': 6.643194047698133e-05, 'Colette_O': 9.964791071547199e-05, 'Muy_O': 0.0010629110476317013, 'cocinados_O': 4.9823955357735996e-05, '18/03_O': 3.3215970238490664e-05, '/_O': 0.0005148475386966054, '2008_O': 3.3215970238490664e-05, 'Buen_O': 0.000348767687504152, 'ambiente_B-positive': 0.001051427157744998, 'buen_O': 0.0010296950773932107, 'céntrico_O': 8.303992559622666e-05, 'buenos_O': 0.00041519962798113334, 'precios_O': 0.00043180761310037866, 'bacalao_B-negative': 0.00012675074466062487, 'dorado_I-negative': 6.379992344009187e-05, 'revuelto_O': 3.3215970238490664e-05, 'típico_O': 3.3215970238490664e-05, 'bacalao_O': 8.303992559622666e-05, 'deliciosa_O': 0.00011625589583471734, 'atencion_B-positive': 0.00018554596901382316, 'demás_O': 0.00016607985119245332, 'indico_O': 4.9823955357735996e-05, 'inicio_O': 4.9823955357735996e-05, 'trato_O': 0.00023251179166943468, 'Muchas_O': 4.9823955357735996e-05, 'gracias_O': 0.00033215970238490664, 'Mayúsculas_O': 4.9823955357735996e-05, 'Gracias_O': 0.000149471866073208, 'tu_O': 0.00024911977678868, 'crónica_O': 6.643194047698133e-05, 'gran_O': 0.00023251179166943468, 'ayuda_O': 4.9823955357735996e-05, 'decidir_O': 4.9823955357735996e-05, 'NO_O': 0.00013286388095396266, 'celebrar_O': 9.964791071547199e-05, 'especial_O': 0.0002657277619079253, 'dejaré_O': 3.3215970238490664e-05, 'sitio_O': 0.00018268783631169867, 'críticos_O': 3.3215970238490664e-05, 'gastronómicos_O': 6.643194047698133e-05, 'experimentados_O': 3.3215970238490664e-05, 'Menú_O': 0.00011625589583471734, 'calçotada_B-positive': 0.00027831895352073476, 'recomendable_O': 0.000697535375008304, 'relación_O': 0.0006643194047698133, 'calidad-precio_O': 0.00036537567262339733, 'Caótico_B-negative': 6.337537233031244e-05, 'Excelente_O': 0.00039859164286188797, 'escaso_O': 0.000149471866073208, 'personal_B-negative': 9.506305849546866e-05, 'esta_O': 0.000697535375008304, 'llenisimo_O': 3.3215970238490664e-05, 'Vino_O': 6.643194047698133e-05, 'tinto_O': 9.964791071547199e-05, 'O_O': 6.643194047698133e-05, 'Calatayud_O': 3.3215970238490664e-05, 'Cinco_O': 3.3215970238490664e-05, 'Platos_B-positive': 6.184865633794106e-05, 'cantidades_O': 9.964791071547199e-05, 'suficientes_O': 4.9823955357735996e-05, 'llenarte.No_O': 3.3215970238490664e-05, 'tardaban_O': 3.3215970238490664e-05, 'nada_O': 0.0008137912708430213, 'servirnos_O': 4.9823955357735996e-05, 'siempre_O': 0.0006643194047698133, 'tuviésemos_O': 3.3215970238490664e-05, 'estaba_O': 0.0006643194047698133, 'especialmente_O': 9.964791071547199e-05, 'huevos_B-positive': 0.00012369731267588213, 'rotos_I-positive': 6.351020926613953e-05, 'cierto_O': 0.000298943732146416, 'explicaban_O': 3.3215970238490664e-05, 'granja_O': 3.3215970238490664e-05, 'masia_O': 6.643194047698133e-05, 'Estuve_O': 0.00024911977678868, 'viernes_O': 9.964791071547199e-05, 'noche_O': 0.000298943732146416, 'increible_O': 4.9823955357735996e-05, 'camareros_O': 4.9823955357735996e-05, 'impecble_O': 3.3215970238490664e-05, 'simpáticos_O': 3.3215970238490664e-05, 'lograda_O': 4.9823955357735996e-05, 'recomedable_O': 3.3215970238490664e-05, 'vez_O': 0.0005646714940543414, 'reserva_O': 0.000448415598219624, 'ningún_O': 0.00013286388095396266, 'problema_O': 0.000149471866073208, 'preguntándonos_O': 3.3215970238490664e-05, 'si_O': 0.00149471866073208, 'orden_O': 3.3215970238490664e-05, 'punto_O': 0.00038198365774264265, 'segundo_O': 0.0003155517172656613, 'menos_O': 0.000298943732146416, 'cantiadad_O': 3.3215970238490664e-05, 'Cenamos_O': 0.00011625589583471734, 'señora_O': 4.9823955357735996e-05, 'correctamente_O': 6.643194047698133e-05, 'ser_O': 0.0006643194047698133, 'fuera_O': 0.00011625589583471734, 'común_O': 4.9823955357735996e-05, 'sorbetes_B-negative': 6.337537233031244e-05, 'fueron_O': 0.00011625589583471734, 'mediocres_O': 4.9823955357735996e-05, 'estaban_O': 0.00018268783631169867, 'altura_O': 0.00019929582143094398, 'pagado_O': 0.00016607985119245332, 'aquí_O': 9.964791071547199e-05, 'sobre_O': 0.00036537567262339733, 'doce_O': 3.3215970238490664e-05, 'cuarto_O': 3.3215970238490664e-05, 'aparecen_O': 4.9823955357735996e-05, 'Posesas_O': 4.9823955357735996e-05, 'empiezan_O': 4.9823955357735996e-05, 'fotos_O': 6.643194047698133e-05, 'cliente_O': 0.00021590380655018933, 'dar_O': 0.00016607985119245332, 'vueltas_O': 3.3215970238490664e-05, 'local_O': 0.00023251179166943468, 'Después_O': 8.303992559622666e-05, 'varios_O': 0.000149471866073208, 'restaurantes_O': 0.0003155517172656613, 'quedé_O': 6.643194047698133e-05, 'pelín_O': 3.3215970238490664e-05, 'decepcionado_O': 3.3215970238490664e-05, 'Comer_O': 3.3215970238490664e-05, 'o_O': 0.000896831196439248, 'cenar_O': 0.00048163156845811467, 'Barceloneta_O': 4.9823955357735996e-05, 'garantía_O': 3.3215970238490664e-05, 'ubicación_B-positive': 0.00015462164084485266, 'EVITADLO_O': 3.3215970238490664e-05, 'SI_O': 3.3215970238490664e-05, 'PODÉIS_O': 3.3215970238490664e-05, 'Un_O': 0.000946655151796984, 'pienso_O': 9.964791071547199e-05, 'volver_O': 0.000348767687504152, 'Venga_O': 3.3215970238490664e-05, 'tan_O': 0.00021590380655018933, 'dificil_O': 0.00011625589583471734, 'p_O': 3.3215970238490664e-05, 'besugo_B-negative': 6.337537233031244e-05, 'salsa_I-negative': 6.379992344009187e-05, 'solo_O': 0.0003155517172656613, 'contenia_O': 3.3215970238490664e-05, 'cabeza_O': 6.643194047698133e-05, 'pedí_O': 6.643194047698133e-05, 'foie_B-neutral': 6.40020480655381e-05, 'tostadas_I-neutral': 6.406149903907751e-05, 'más_O': 0.0012123829137049094, 'correcta_O': 0.00016607985119245332, 'pocas_O': 0.00011625589583471734, 'tostadas_O': 3.3215970238490664e-05, 'foie_O': 0.00011625589583471734, 'Todo_O': 0.00049823955357736, 'perfecto_O': 0.00041519962798113334, 'Recepcionistas_B-positive': 6.184865633794106e-05, 'Sabado_O': 3.3215970238490664e-05, '13h_O': 3.3215970238490664e-05, 'sea_O': 0.00016607985119245332, 'cocina_O': 0.00043180761310037866, 'agobios_O': 3.3215970238490664e-05, 'paella_B-negative': 0.00019012611699093733, 'simplemente_O': 8.303992559622666e-05, 'Sin_O': 8.303992559622666e-05, 'dudarlo_O': 6.643194047698133e-05, 'repetiremos_O': 0.00013286388095396266, 'primeros_O': 8.303992559622666e-05, 'vayase_O': 3.3215970238490664e-05, 'Palafox_O': 3.3215970238490664e-05, 'tiene_O': 0.00041519962798113334, 'Jésús_O': 3.3215970238490664e-05, 'bigotes_O': 3.3215970238490664e-05, 'perfección_O': 3.3215970238490664e-05, 'Sitio_B-positive': 0.00015462164084485266, 'recomendado_O': 0.00011625589583471734, 'partir_O': 4.9823955357735996e-05, 'ahi_O': 4.9823955357735996e-05, 'caras_O': 6.643194047698133e-05, 'serias_O': 3.3215970238490664e-05, 'humor_O': 3.3215970238490664e-05, 'distraidas_O': 3.3215970238490664e-05, 'camareros_B-negative': 0.00022181380315609354, '..._O': 0.0006809273898890587, 'hacían_O': 3.3215970238490664e-05, 'otras_O': 0.00018268783631169867, 'abalanzaban_O': 3.3215970238490664e-05, 'liquidar_O': 3.3215970238490664e-05, 'apetecibles_O': 4.9823955357735996e-05, 'difícil_O': 8.303992559622666e-05, 'decidirse_O': 3.3215970238490664e-05, 'local_B-positive': 0.00058756223521044, 'aco_O': 3.3215970238490664e-05, '*_O': 3.3215970238490664e-05, 'edor_O': 3.3215970238490664e-05, 'instalado_O': 3.3215970238490664e-05, 'Altamente_O': 3.3215970238490664e-05, 'tener_O': 0.0002657277619079253, 'solicité_O': 3.3215970238490664e-05, 'tenía_O': 0.00019929582143094398, 'incremento_O': 3.3215970238490664e-05, 'siendo_O': 9.964791071547199e-05, 'exactamente_O': 4.9823955357735996e-05, 'marcaba_O': 3.3215970238490664e-05, 'menú_B-positive': 0.0008968055169001454, 'elegido_O': 4.9823955357735996e-05, 'reservado_O': 4.9823955357735996e-05, 'internet_O': 0.00011625589583471734, 'bonito_O': 0.00021590380655018933, 'excesible_O': 3.3215970238490664e-05, 'Pedimos_O': 0.00023251179166943468, 'menu_B-positive': 0.0003092432816897053, 'de_I-positive': 0.0018417960687180464, 'calçots_I-positive': 6.351020926613953e-05, 'valia_O': 3.3215970238490664e-05, 'pena_O': 0.000348767687504152, 'Te_O': 0.00011625589583471734, 'gustará_O': 3.3215970238490664e-05, 'quieres_O': 0.00011625589583471734, 'quedar_O': 9.964791071547199e-05, 'conseguiras_O': 3.3215970238490664e-05, 'Fabuloso_O': 4.9823955357735996e-05, 'comienzo_O': 4.9823955357735996e-05, 'año_O': 0.00013286388095396266, 'estancia_O': 4.9823955357735996e-05, 'genial_O': 0.0003155517172656613, 'recibido_O': 9.964791071547199e-05, 'anteriores_O': 6.643194047698133e-05, 'vinos_B-negative': 0.00022181380315609354, 'carísimos_O': 3.3215970238490664e-05, 'Ha_O': 4.9823955357735996e-05, 'tiempos_B-positive': 6.184865633794106e-05, 'ambiente_B-neutral': 0.00016000512016384525, 'quizas_O': 4.9823955357735996e-05, 'demasiado_O': 0.0003155517172656613, 'iluminado_O': 3.3215970238490664e-05, 'sabor_O': 0.00021590380655018933, 'cualquier_O': 0.00021590380655018933, 'caso_O': 0.00021590380655018933, 'Ahora_O': 4.9823955357735996e-05, 'merece_O': 0.000149471866073208, 'vistazo_O': 3.3215970238490664e-05, 'críticas_O': 6.643194047698133e-05, 'expertos_O': 3.3215970238490664e-05, 'culinarios_O': 3.3215970238490664e-05, 'Gustos_O': 3.3215970238490664e-05, 'sera_O': 3.3215970238490664e-05, 'vean_O': 3.3215970238490664e-05, 'sitios_O': 9.964791071547199e-05, 'deseo_O': 3.3215970238490664e-05, 'postres_B-negative': 0.00022181380315609354, 'ausencia_O': 3.3215970238490664e-05, 'chocolate_O': 8.303992559622666e-05, 'personalmente_O': 9.964791071547199e-05, 'imperdonable_O': 4.9823955357735996e-05, 'eso_O': 0.0005148475386966054, 'cuestión_O': 4.9823955357735996e-05, 'personal_O': 0.00013286388095396266, 'magnífico_O': 6.643194047698133e-05, 'caracoles_O': 3.3215970238490664e-05, 'carta_O': 0.00033215970238490664, 'paletilla_O': 3.3215970238490664e-05, 'cordero_O': 3.3215970238490664e-05, 'sugerencia_O': 3.3215970238490664e-05, 'tampoco_O': 0.00011625589583471734, 'pan_O': 8.303992559622666e-05, 'pedido_O': 0.00011625589583471734, '.._O': 0.00033215970238490664, 'fin_O': 0.00013286388095396266, 'os_O': 0.00024911977678868, 'piseis_O': 3.3215970238490664e-05, 'sitio_B-negative': 0.00015843843082578111, 'cobraran_O': 3.3215970238490664e-05, 'entrar_O': 0.00011625589583471734, 'Hoy_O': 8.303992559622666e-05, 'he_O': 0.000647711419650568, 'ido_O': 0.00036537567262339733, 'mia_O': 3.3215970238490664e-05, 'padres_O': 3.3215970238490664e-05, 'salido_O': 3.3215970238490664e-05, 'defraudado_O': 8.303992559622666e-05, 'pusieron_O': 6.643194047698133e-05, 'cobrarlo_O': 3.3215970238490664e-05, 'Carpaccio_B-negative': 6.337537233031244e-05, 'de_I-negative': 0.0009250988898813321, 'bonito_I-negative': 6.379992344009187e-05, ':_O': 0.001145950973227928, 'Seria_O': 3.3215970238490664e-05, 'discuto_O': 3.3215970238490664e-05, 'supo_O': 3.3215970238490664e-05, 'totalmente_O': 0.000149471866073208, 'insipido_O': 3.3215970238490664e-05, 'tambien_O': 0.00013286388095396266, 'tenia_O': 0.00011625589583471734, 'remolacha_B-negative': 6.337537233031244e-05, 'insipida_O': 3.3215970238490664e-05, 'helado_O': 0.00019929582143094398, 'creo_O': 0.00038198365774264265, 'recordar_O': 6.643194047698133e-05, 'mango_O': 3.3215970238490664e-05, 'algun_O': 4.9823955357735996e-05, 'Nos_O': 0.00041519962798113334, 'dejamos_O': 3.3215970238490664e-05, 'guiar_O': 3.3215970238490664e-05, 'ofrecieron_O': 0.00011625589583471734, 'entrantes_O': 0.00023251179166943468, 'compartir_O': 0.000149471866073208, 'buñuelos_O': 3.3215970238490664e-05, 'alioli_O': 3.3215970238490664e-05, 'membrillo_O': 4.9823955357735996e-05, 'calamares_O': 8.303992559622666e-05, 'plancha_O': 6.643194047698133e-05, 'He_O': 0.00013286388095396266, 'varias_O': 0.00016607985119245332, 'amable_O': 0.000448415598219624, 'cuanto_O': 0.00023251179166943468, 'producto_B-positive': 0.0003401676098586758, 'sorpresas_O': 6.643194047698133e-05, 'plato_B-negative': 0.000285189175486406, 'jamon_I-negative': 6.379992344009187e-05, 'entrante_I-negative': 6.379992344009187e-05, '25_O': 9.964791071547199e-05, 'euros_O': 0.00048163156845811467, '?_O': 0.000448415598219624, 'Mal_O': 3.3215970238490664e-05, 'Tampoco_O': 4.9823955357735996e-05, 'avisaron_O': 3.3215970238490664e-05, 'menu_O': 0.0003155517172656613, 'ese_O': 0.0002823357470271707, 'dia_O': 0.0002823357470271707, 'través_O': 6.643194047698133e-05, 'nunca_O': 0.0002657277619079253, 'han_O': 0.00038198365774264265, 'valor_O': 0.000149471866073208, 'seguro_O': 0.00048163156845811467, 'Tardaron_O': 3.3215970238490664e-05, 'media_O': 0.00018268783631169867, 'hora_O': 0.0002657277619079253, 'traer_O': 6.643194047698133e-05, 'pedidas_O': 3.3215970238490664e-05, 'beber_O': 9.964791071547199e-05, 'vino_O': 0.00041519962798113334, 'recomienda_O': 3.3215970238490664e-05, 'casa_O': 0.0003155517172656613, 'Viña_O': 3.3215970238490664e-05, 'Salceda_O': 3.3215970238490664e-05, 'Reserva_O': 6.643194047698133e-05, 'C_O': 4.9823955357735996e-05, 'sitio_B-positive': 0.0008658811887311748, 'repetir_O': 0.0002823357470271707, 'pizzas_B-positive': 9.277298450691158e-05, 'horno_O': 9.964791071547199e-05, 'leña_O': 4.9823955357735996e-05, 'buenas_O': 0.00018268783631169867, 'Compruebo_O': 3.3215970238490664e-05, 'acertado_O': 6.643194047698133e-05, 'eres_O': 9.964791071547199e-05, 'Aragonés_O': 4.9823955357735996e-05, 'entendido_O': 6.643194047698133e-05, 'fórmula_O': 8.303992559622666e-05, 'estuvo_O': 0.000348767687504152, 'cenando_O': 0.00018268783631169867, 'riquísima_O': 3.3215970238490664e-05, 'camareras_O': 4.9823955357735996e-05, 'carne_O': 9.964791071547199e-05, 'fallo_O': 4.9823955357735996e-05, 'lleva_O': 4.9823955357735996e-05, 'va_O': 0.00011625589583471734, 'atraída_O': 3.3215970238490664e-05, 'diseño_O': 3.3215970238490664e-05, 'fama_O': 4.9823955357735996e-05, 'cogiendo_O': 3.3215970238490664e-05, 'Fuimos_O': 0.00023251179166943468, 'semana_O': 0.00019929582143094398, 'pasada_O': 9.964791071547199e-05, 'descubrimiento_O': 4.9823955357735996e-05, 'aunque_O': 0.0005646714940543414, 'nadie_O': 0.00018268783631169867, 'pidió_O': 6.643194047698133e-05, 'pizza_B-positive': 6.184865633794106e-05, 'pinta_O': 3.3215970238490664e-05, 'volveremos_O': 0.0002657277619079253, 'probarlas_O': 3.3215970238490664e-05, 'comida_B-negative': 0.000570378350972812, 'elevado_O': 0.00011625589583471734, 'desgustación_I-positive': 6.351020926613953e-05, 'alrededor_O': 6.643194047698133e-05, '50_O': 9.964791071547199e-05, '€_O': 0.000448415598219624, 'persona_O': 0.0005148475386966054, 'perfecta_O': 0.00011625589583471734, 'considero_O': 3.3215970238490664e-05, 'caro_O': 0.0006144954494120774, 'Supero_O': 3.3215970238490664e-05, 'nuestras_O': 8.303992559622666e-05, 'expectativas_O': 6.643194047698133e-05, 'iluminación_B-neutral': 9.600307209830715e-05, 'acogedora_O': 6.643194047698133e-05, 'Sinceramente_O': 9.964791071547199e-05, 'asegurar_O': 3.3215970238490664e-05, 'volveré_O': 0.00013286388095396266, 'Restaurante_B-neutral': 0.0001280040961310762, 'pasta_O': 4.9823955357735996e-05, 'americana_O': 3.3215970238490664e-05, 'regular_O': 4.9823955357735996e-05, 'carne_B-negative': 0.00025350148932124975, 'mejorable_O': 8.303992559622666e-05, 'alterar_O': 3.3215970238490664e-05, 'numero_O': 4.9823955357735996e-05, 'piezas_O': 4.9823955357735996e-05, 'Ayer_O': 0.00013286388095396266, 'estuvimos_O': 0.00016607985119245332, 'Aragonia_O': 6.643194047698133e-05, 'MUY_O': 4.9823955357735996e-05, 'RECOMENDABLE_O': 3.3215970238490664e-05, 'UNA_O': 4.9823955357735996e-05, 'MARAVILLA_O': 3.3215970238490664e-05, 'OS_O': 4.9823955357735996e-05, 'LO_O': 0.00016607985119245332, 'RECOMIENDO_O': 8.303992559622666e-05, 'Fantastico_O': 3.3215970238490664e-05, 'luz_B-positive': 6.184865633794106e-05, 'natural_I-positive': 6.351020926613953e-05, 'entra_O': 3.3215970238490664e-05, 'vidriera_O': 3.3215970238490664e-05, 'terraza_O': 9.964791071547199e-05, 'añadida_O': 3.3215970238490664e-05, 'da_O': 0.00016607985119245332, 'sensación_O': 0.00016607985119245332, 'bienestar_O': 3.3215970238490664e-05, 'conseguir_O': 4.9823955357735996e-05, 'otros_O': 0.00033215970238490664, 'lugares_O': 3.3215970238490664e-05, 'Pero_O': 0.00021590380655018933, 'nuevo_O': 9.964791071547199e-05, 'escasos_O': 3.3215970238490664e-05, 'especiales_O': 6.643194047698133e-05, '<+>_O': 0.0003155517172656613, 'extraordinaria_O': 0.00011625589583471734, 'pescados_B-positive': 6.184865633794106e-05, 'mariscos_B-positive': 9.277298450691158e-05, 'Repetiría_O': 3.3215970238490664e-05, 'exquisito_O': 0.00018268783631169867, 'Hola_O': 0.00019929582143094398, 'tomates_O': 3.3215970238490664e-05, 'Almería_O': 3.3215970238490664e-05, 'Y_O': 0.0006144954494120774, 'imaginais_O': 3.3215970238490664e-05, 'esfuerzo_O': 6.643194047698133e-05, 'hacemos_O': 3.3215970238490664e-05, 'ajustarlos_O': 3.3215970238490664e-05, 'allí_O': 0.00019929582143094398, 'cuesta_O': 6.643194047698133e-05, 'tela_O': 3.3215970238490664e-05, 'pna_O': 3.3215970238490664e-05, 'ver_O': 0.000149471866073208, 'nuestros_O': 0.00011625589583471734, 'clientes_O': 0.00024911977678868, 'saliendo_O': 3.3215970238490664e-05, 'felices_O': 3.3215970238490664e-05, 'contentos_O': 0.00013286388095396266, 'productos_O': 4.9823955357735996e-05, 'decorado_O': 8.303992559622666e-05, 'postre_B-positive': 0.0003092432816897053, 'probamos_O': 8.303992559622666e-05, 'setas_O': 6.643194047698133e-05, 'pedir_O': 0.00024911977678868, 'Repetiremos_O': 0.00016607985119245332, 'turistico_O': 3.3215970238490664e-05, 'Hay_O': 9.964791071547199e-05, 'vinos_B-positive': 0.0002164702971827937, 'jóvenes_O': 3.3215970238490664e-05, 'Cariñena_O': 3.3215970238490664e-05, '2_O': 0.0002823357470271707, 'botella_O': 0.00013286388095396266, 'día_O': 0.000298943732146416, 'siguiente_O': 4.9823955357735996e-05, 'fuimos_O': 0.0002823357470271707, 'aparcan_O': 3.3215970238490664e-05, 'coche_O': 4.9823955357735996e-05, 'solomillo_B-positive': 0.0002164702971827937, 'ternera_I-positive': 6.351020926613953e-05, 'carne_B-positive': 0.00024739462535176425, 'justo_O': 8.303992559622666e-05, 'pedimos_O': 0.00021590380655018933, 'Celebramos_O': 6.643194047698133e-05, 'cumpleaños_O': 9.964791071547199e-05, 'Trato_B-negative': 6.337537233031244e-05, 'pobre_O': 4.9823955357735996e-05, 'platos_B-negative': 0.0003485645478167184, 'sentido_O': 4.9823955357735996e-05, 'local_B-negative': 0.00022181380315609354, 'ruido_O': 9.964791071547199e-05, 'Cocina_B-positive': 9.277298450691158e-05, 'platos_B-positive': 0.00058756223521044, 'estan_O': 0.000149471866073208, 'exquisutos_O': 3.3215970238490664e-05, '12_O': 9.964791071547199e-05, 'compañeros_O': 4.9823955357735996e-05, 'trabajo_O': 0.00011625589583471734, 'cogimos_O': 3.3215970238490664e-05, '20_O': 0.000149471866073208, 'Absolutamente_O': 6.643194047698133e-05, 'dieron_O': 4.9823955357735996e-05, 'opción_O': 9.964791071547199e-05, 'calsots_B-positive': 6.184865633794106e-05, 'detalle_O': 0.00011625589583471734, 'agradece_O': 6.643194047698133e-05, 'suficiente_O': 9.964791071547199e-05, 'aprobar_O': 3.3215970238490664e-05, 'servicio_O': 0.00024911977678868, 'Magnífica_O': 4.9823955357735996e-05, 'atención_O': 0.000149471866073208, 'Ana_B-positive': 0.00012369731267588213, 'hoy_O': 0.00011625589583471734, 'guinda_O': 3.3215970238490664e-05, 'carajillo_O': 3.3215970238490664e-05, 'ron_O': 3.3215970238490664e-05, 'debe_O': 8.303992559622666e-05, 'obra_O': 4.9823955357735996e-05, 'camarera_B-positive': 9.277298450691158e-05, 'Extremadamente_O': 4.9823955357735996e-05, 'juntas_O': 0.000149471866073208, 'mesas_B-negative': 0.000285189175486406, 'alguno_O': 0.00013286388095396266, 'Seguro_O': 4.9823955357735996e-05, 'quedas_O': 4.9823955357735996e-05, 'hambre_O': 9.964791071547199e-05, 'claidad_O': 3.3215970238490664e-05, 'Calidad-precio_B-positive': 6.184865633794106e-05, 'hay_O': 0.0005646714940543414, 'negativo_O': 8.303992559622666e-05, 'exterior_O': 4.9823955357735996e-05, 'luminoso_O': 4.9823955357735996e-05, 'accesos_O': 3.3215970238490664e-05, 'faciles_O': 3.3215970238490664e-05, 'extraña_O': 4.9823955357735996e-05, 'comentario_O': 0.00018268783631169867, 'Marta_O': 3.3215970238490664e-05, 'propietario_B-negative': 6.337537233031244e-05, 'debería_O': 0.00011625589583471734, 'dedicarse_O': 3.3215970238490664e-05, 'menesteres_O': 3.3215970238490664e-05, 'interesante_O': 6.643194047698133e-05, 'genero_O': 3.3215970238490664e-05, 'empezar_O': 6.643194047698133e-05, 'puerta_B-negative': 9.506305849546866e-05, 'almacen_O': 3.3215970238490664e-05, 'llamas_O': 3.3215970238490664e-05, 'portero_O': 3.3215970238490664e-05, 'automatico_O': 3.3215970238490664e-05, 'abren_O': 3.3215970238490664e-05, 'esto_O': 0.00021590380655018933, 'direis_O': 3.3215970238490664e-05, 'secundario_O': 3.3215970238490664e-05, 'Tanto_O': 0.00013286388095396266, 'historia_O': 3.3215970238490664e-05, 'bodega_B-positive': 0.00018554596901382316, 'Unos_O': 4.9823955357735996e-05, 'aperitivos_O': 3.3215970238490664e-05, 'ofrecidos_O': 3.3215970238490664e-05, 'cóctel_O': 3.3215970238490664e-05, 'martini_O': 3.3215970238490664e-05, 'blanco_O': 9.964791071547199e-05, 'enfriado_O': 3.3215970238490664e-05, 'nitrógeno_O': 3.3215970238490664e-05, 'pastilla_O': 3.3215970238490664e-05, 'mi-cuit_O': 3.3215970238490664e-05, 'envuelto_O': 3.3215970238490664e-05, 'frambuesa_O': 3.3215970238490664e-05, 'sidral_O': 3.3215970238490664e-05, 'algodón_O': 3.3215970238490664e-05, 'coco_O': 3.3215970238490664e-05, 'entndemos_O': 3.3215970238490664e-05, 'qué_O': 9.964791071547199e-05, 'hicieron_O': 9.964791071547199e-05, 'escoger_O': 3.3215970238490664e-05, 'entre_O': 0.00041519962798113334, 'primer_O': 0.00016607985119245332, 'turno_O': 4.9823955357735996e-05, 'opinión_O': 0.00023251179166943468, 'maitre_B-negative': 9.506305849546866e-05, 'tomó_O': 3.3215970238490664e-05, 'nota_O': 0.000149471866073208, 'profesionalidad_O': 0.00013286388095396266, 'rozando_O': 3.3215970238490664e-05, 'mala_O': 0.00013286388095396266, 'educación_O': 9.964791071547199e-05, 'Increible_O': 4.9823955357735996e-05, 'increíble_O': 4.9823955357735996e-05, 'personal_B-positive': 0.00058756223521044, 'magnifico_O': 4.9823955357735996e-05, 'Creo_O': 0.00013286388095396266, 'espaciaran_O': 3.3215970238490664e-05, 'cambiaran_O': 3.3215970238490664e-05, 'iluminación_O': 3.3215970238490664e-05, 'calido_O': 3.3215970238490664e-05, 'ganaría_O': 3.3215970238490664e-05, 'tal_O': 8.303992559622666e-05, 'dirigido_O': 3.3215970238490664e-05, 'publico_O': 3.3215970238490664e-05, 'fije_O': 3.3215970238490664e-05, 'detalles_O': 0.00013286388095396266, 'quiera_O': 6.643194047698133e-05, 'estar_O': 0.000298943732146416, 'come_O': 0.00018268783631169867, 'presentamos_O': 3.3215970238490664e-05, 'intención_O': 8.303992559622666e-05, 'elegir_O': 0.00021590380655018933, 'presentaron_O': 4.9823955357735996e-05, 'ajustado_O': 9.964791071547199e-05, 'ésa_O': 3.3215970238490664e-05, 'nuestra_O': 0.00024911977678868, 'selección_O': 6.643194047698133e-05, 'resto_O': 0.000149471866073208, 'Que_O': 0.00024911977678868, 'molestaba_O': 3.3215970238490664e-05, 'cambiaban_O': 3.3215970238490664e-05, 'despues_O': 9.964791071547199e-05, 'aplicacion_O': 3.3215970238490664e-05, 'ley_O': 3.3215970238490664e-05, 'cambiar_O': 3.3215970238490664e-05, 'VUELVO_O': 3.3215970238490664e-05, 'MAS_O': 3.3215970238490664e-05, 'Hemos_O': 0.0002823357470271707, 'comido_O': 0.00024911977678868, 'fenomenal_O': 0.00011625589583471734, '¿_O': 0.00024911977678868, 'Alguien_O': 3.3215970238490664e-05, 'cuyo_O': 4.9823955357735996e-05, '[_O': 6.643194047698133e-05, ']_O': 6.643194047698133e-05, 'bien.Tanto_O': 3.3215970238490664e-05, 'consejos_O': 3.3215970238490664e-05, 'apropiados_O': 3.3215970238490664e-05, 'tiempo_B-positive': 6.184865633794106e-05, 'espera_I-positive': 6.351020926613953e-05, 'dio_O': 6.643194047698133e-05, 'comportamiento_O': 3.3215970238490664e-05, 'negligente_O': 3.3215970238490664e-05, 'ensayo_O': 3.3215970238490664e-05, 'disculpa_O': 3.3215970238490664e-05, 'falta_O': 0.00024911977678868, 'total_O': 0.00011625589583471734, 'saludo_O': 0.00024911977678868, 'tus_O': 9.964791071547199e-05, 'Personalmente_O': 3.3215970238490664e-05, 'actitud_O': 3.3215970238490664e-05, 'camareros_B-positive': 0.0002164702971827937, 'maitrê_B-positive': 6.184865633794106e-05, 'michelin_O': 6.643194047698133e-05, 'grandes_O': 6.643194047698133e-05, 'personas_B-positive': 6.184865633794106e-05, 'independiente_O': 3.3215970238490664e-05, 'sincero_O': 3.3215970238490664e-05, 'crónicas_O': 3.3215970238490664e-05, 'sigue_O': 0.00011625589583471734, 'así_O': 0.0006643194047698133, 'Segunda_O': 3.3215970238490664e-05, 'Fiesta_O': 3.3215970238490664e-05, 'Comunión_O': 3.3215970238490664e-05, 'toca_O': 4.9823955357735996e-05, 'Junquera_O': 3.3215970238490664e-05, 'carísimo_O': 4.9823955357735996e-05, 'ofrece_O': 4.9823955357735996e-05, 'rica_O': 6.643194047698133e-05, 'excelemte_O': 3.3215970238490664e-05, 'Caro_O': 3.3215970238490664e-05, 'Reservé_O': 3.3215970238490664e-05, '1.01_O': 3.3215970238490664e-05, '07_O': 3.3215970238490664e-05, 'llegué_O': 6.643194047698133e-05, 'tenían_O': 0.00019929582143094398, 'apuntada_O': 3.3215970238490664e-05, 'temas_O': 4.9823955357735996e-05, 'Ideal_O': 6.643194047698133e-05, 'cenas_O': 8.303992559622666e-05, '4_O': 0.0002823357470271707, '8_O': 6.643194047698133e-05, 'personas_O': 0.0005646714940543414, 'flojita_O': 3.3215970238490664e-05, 'Sorprende_O': 3.3215970238490664e-05, 'cobren_O': 3.3215970238490664e-05, 'dulces_B-negative': 6.337537233031244e-05, 'acompañando_O': 3.3215970238490664e-05, 'café_O': 4.9823955357735996e-05, 'habíamos_O': 9.964791071547199e-05, 'solicitadoRealmente_O': 3.3215970238490664e-05, 'aprender_O': 3.3215970238490664e-05, 'atento_O': 0.00038198365774264265, 'cordial_O': 9.964791071547199e-05, 'Volveré_O': 6.643194047698133e-05, 'justa_O': 6.643194047698133e-05, 'proporcion_O': 3.3215970238490664e-05, 'ay_O': 4.9823955357735996e-05, 'vinos_I-positive': 0.00022228573243148836, 'completa_O': 4.9823955357735996e-05, 'eficiente_O': 8.303992559622666e-05, 'arroces_B-positive': 0.0002164702971827937, 'suquets_B-positive': 6.184865633794106e-05, 'inmejorables_O': 4.9823955357735996e-05, 'decorado_B-positive': 6.184865633794106e-05, 'atractivo_O': 4.9823955357735996e-05, 'tarde_O': 0.000149471866073208, 'realizamos_O': 3.3215970238490664e-05, 'circuito_O': 9.964791071547199e-05, 'termal_O': 9.964791071547199e-05, 'toda_O': 0.0002657277619079253, 'bañandonos_O': 3.3215970238490664e-05, 'Una_O': 0.000348767687504152, 'excepción_O': 4.9823955357735996e-05, 'notoria_O': 3.3215970238490664e-05, 'bodegas_O': 4.9823955357735996e-05, 'Lalanne_O': 4.9823955357735996e-05, 'Lucrecia_O': 3.3215970238490664e-05, 'brut_O': 3.3215970238490664e-05, 'nature_O': 3.3215970238490664e-05, 'Reinante_O': 3.3215970238490664e-05, 'Real_O': 4.9823955357735996e-05, 'Aragón_O': 9.964791071547199e-05, 'acierto_O': 0.00013286388095396266, 'elección_O': 6.643194047698133e-05, 'Tuve_O': 3.3215970238490664e-05, 'aún_O': 0.00016607985119245332, 'negaban_O': 3.3215970238490664e-05, 'Pensamos_O': 4.9823955357735996e-05, 'tendremos_O': 3.3215970238490664e-05, 'variedad_O': 0.00019929582143094398, 'CalÇots_B-positive': 6.184865633794106e-05, 'Alcachofas_B-positive': 6.184865633794106e-05, 'sublimes_O': 3.3215970238490664e-05, 'mejores_O': 0.00023251179166943468, 'Barcelona_O': 0.000348767687504152, 'ciudad_O': 9.964791071547199e-05, 'EL_O': 4.9823955357735996e-05, 'SERVICIO_O': 3.3215970238490664e-05, 'dice_O': 0.00016607985119245332, 'usted_O': 0.000149471866073208, 'empatizó_O': 3.3215970238490664e-05, 'le_O': 0.000647711419650568, 'len_O': 3.3215970238490664e-05, 'pregunten_O': 3.3215970238490664e-05, 'Pues_O': 0.00011625589583471734, 'fíjese_O': 3.3215970238490664e-05, 'mí_O': 0.00018268783631169867, 'molesten_O': 4.9823955357735996e-05, 'pretensiones_O': 3.3215970238490664e-05, 'abundante_O': 0.00023251179166943468, 'rogout_B-positive': 6.184865633794106e-05, 'manitas_I-positive': 6.351020926613953e-05, 'prueba_O': 6.643194047698133e-05, 'dios_O': 3.3215970238490664e-05, 'padre_O': 6.643194047698133e-05, 'Rte_O': 3.3215970238490664e-05, 'Balmes_O': 4.9823955357735996e-05, 'Rosello_O': 3.3215970238490664e-05, 'encanto_O': 0.00018268783631169867, 'localizacion_B-positive': 6.184865633794106e-05, 'aseo_O': 3.3215970238490664e-05, 'confort_O': 3.3215970238490664e-05, 'Quiero_O': 4.9823955357735996e-05, 'pensar_O': 4.9823955357735996e-05, 'compara_O': 3.3215970238490664e-05, 'cava_O': 0.00013286388095396266, 'tintorro_O': 3.3215970238490664e-05, 'acepción_O': 3.3215970238490664e-05, 'despreciativa_O': 3.3215970238490664e-05, 'meri_O': 3.3215970238490664e-05, 'pongas_O': 3.3215970238490664e-05, 'asi_O': 0.00024911977678868, 'mujer_O': 0.00018268783631169867, 'hagan_O': 4.9823955357735996e-05, 'pelota_O': 3.3215970238490664e-05, 'inviten_O': 3.3215970238490664e-05, 'chupitos_O': 4.9823955357735996e-05, 'i_O': 6.643194047698133e-05, 'encima_O': 0.000149471866073208, 'regalen_O': 4.9823955357735996e-05, 'segundos_B-positive': 9.277298450691158e-05, 'Podría_O': 3.3215970238490664e-05, 'organizada_O': 3.3215970238490664e-05, 'cambio_O': 9.964791071547199e-05, 'cuchillería_B-negative': 6.337537233031244e-05, 'mismo_O': 0.000298943732146416, 'utilizado_O': 4.9823955357735996e-05, 'dos_O': 0.0008303992559622667, 'entrante_O': 9.964791071547199e-05, 'principal_O': 9.964791071547199e-05, 'vi_O': 8.303992559622666e-05, 'flojo_O': 6.643194047698133e-05, 'pedi_O': 3.3215970238490664e-05, 'salsa_B-negative': 9.506305849546866e-05, 'provenzal_I-negative': 6.379992344009187e-05, 'cobraron_O': 4.9823955357735996e-05, 'demas_O': 0.00013286388095396266, 'Recomendable_O': 0.000149471866073208, 'visitado_O': 4.9823955357735996e-05, 'ocasiones_O': 9.964791071547199e-05, 'destacar_O': 4.9823955357735996e-05, 'materia_B-positive': 0.00012369731267588213, 'prima_I-positive': 0.00012702041853227905, 'saber_O': 0.00013286388095396266, 'Buenas_O': 6.643194047698133e-05, 'raciones_B-positive': 0.0002164702971827937, 'primero_O': 0.000149471866073208, 'soberbio_O': 3.3215970238490664e-05, 'centro_O': 0.00016607985119245332, 'merluza_B-positive': 9.277298450691158e-05, 'plancha_I-positive': 6.351020926613953e-05, 'Acogedor_O': 3.3215970238490664e-05, 'tranquilo_O': 0.00016607985119245332, 'Risotto_B-negative': 6.337537233031244e-05, 'hongos_I-negative': 6.379992344009187e-05, 'trufa_I-negative': 6.379992344009187e-05, 'y_I-negative': 9.56998851601378e-05, 'foie_I-negative': 6.379992344009187e-05, 'misterio_O': 3.3215970238490664e-05, 'sarten_O': 3.3215970238490664e-05, 'estropearlo_O': 3.3215970238490664e-05, 'ahora_O': 0.000149471866073208, 'risotto_O': 4.9823955357735996e-05, 'corrio_O': 3.3215970238490664e-05, 'suerte_O': 6.643194047698133e-05, 'granos_O': 3.3215970238490664e-05, 'arroz_O': 0.000149471866073208, 'intenso_O': 4.9823955357735996e-05, 'nata_O': 6.643194047698133e-05, 'ocasión_O': 6.643194047698133e-05, 'merezca_O': 3.3215970238490664e-05, 'echen_O': 3.3215970238490664e-05, 'víspera_O': 3.3215970238490664e-05, 'Reyes_O': 3.3215970238490664e-05, 'gracia_O': 4.9823955357735996e-05, 'postre_O': 0.0002657277619079253, 'trufas_O': 4.9823955357735996e-05, 'Además_O': 0.00011625589583471734, 'avisado_O': 3.3215970238490664e-05, 'ellas_O': 6.643194047698133e-05, 'iba_O': 9.964791071547199e-05, 'silla_O': 4.9823955357735996e-05, 'ruedas_O': 3.3215970238490664e-05, '1/02_O': 3.3215970238490664e-05, 'quejarse_O': 3.3215970238490664e-05, 'balneario_O': 0.00011625589583471734, 'nosotros_O': 0.00018268783631169867, '9_O': 0.00011625589583471734, 'dias_O': 9.964791071547199e-05, 'variada_O': 3.3215970238490664e-05, 'entonces_O': 9.964791071547199e-05, 'decepción_O': 9.964791071547199e-05, 'Esta_O': 6.643194047698133e-05, 'Vaya_O': 3.3215970238490664e-05, 'fiasco_O': 3.3215970238490664e-05, 'verdad_O': 0.0006144954494120774, 'encantado_O': 0.000149471866073208, 'Agradable_O': 6.643194047698133e-05, 'tomar_O': 0.00018268783631169867, 'copas_B-positive': 6.184865633794106e-05, 'cockteles_B-positive': 6.184865633794106e-05, 'vinito_B-positive': 6.184865633794106e-05, 'tendre_O': 3.3215970238490664e-05, 'proximas_O': 3.3215970238490664e-05, 'Repetiré_O': 6.643194047698133e-05, 'regrese_O': 3.3215970238490664e-05, 'conocíamos_O': 3.3215970238490664e-05, 'esperábamos_O': 6.643194047698133e-05, 'Comunicaros_O': 3.3215970238490664e-05, 'podeis_O': 3.3215970238490664e-05, 'quitar_O': 3.3215970238490664e-05, 'lista_O': 6.643194047698133e-05, 'mes_O': 4.9823955357735996e-05, 'Afirmar_O': 3.3215970238490664e-05, 'pueden_O': 6.643194047698133e-05, 'degustar_O': 6.643194047698133e-05, 'verdaderas_O': 3.3215970238490664e-05, 'delicias_B-negative': 6.337537233031244e-05, 'auténtica_O': 4.9823955357735996e-05, 'temeridad_O': 3.3215970238490664e-05, 'explico_O': 4.9823955357735996e-05, 'tengas_O': 3.3215970238490664e-05, 'leer_O': 6.643194047698133e-05, 'precio_B-positive': 9.277298450691158e-05, 'comparando_O': 3.3215970238490664e-05, 'similares_O': 3.3215970238490664e-05, 'cocina_B-neutral': 9.600307209830715e-05, 'innovadora_O': 3.3215970238490664e-05, 'aspire_O': 3.3215970238490664e-05, 'máxima_O': 3.3215970238490664e-05, 'camarera_O': 4.9823955357735996e-05, 'atendio_O': 3.3215970238490664e-05, 'oferta_B-negative': 6.337537233031244e-05, 'q_O': 0.000149471866073208, 'sale_O': 9.964791071547199e-05, 'aqui_O': 6.643194047698133e-05, 'luego_O': 0.000348767687504152, 'alli_O': 8.303992559622666e-05, 'salio_O': 9.964791071547199e-05, '5_O': 0.00018268783631169867, 'pers_O': 3.3215970238490664e-05, 'menu.por_O': 3.3215970238490664e-05, 'servicio_B-negative': 0.0008555675264592179, 'Llegamos_O': 4.9823955357735996e-05, '22.30_O': 4.9823955357735996e-05, 'teniamos_O': 3.3215970238490664e-05, 'única_O': 0.00013286388095396266, 'habia_O': 0.00011625589583471734, 'pakistaní_O': 3.3215970238490664e-05, 'apenas_O': 3.3215970238490664e-05, 'hablaba_O': 3.3215970238490664e-05, 'castellano_O': 3.3215970238490664e-05, 'dijo_O': 6.643194047698133e-05, '21.30_O': 3.3215970238490664e-05, 'podrían_O': 0.00011625589583471734, 'haber_O': 0.00018268783631169867, 'peladas_O': 3.3215970238490664e-05, 'fritas_O': 6.643194047698133e-05, 'enhorabuena_O': 4.9823955357735996e-05, 'magnifica_O': 3.3215970238490664e-05, 'amigo_O': 6.643194047698133e-05, 'Comedor_B-positive': 9.277298450691158e-05, 'consejo_O': 4.9823955357735996e-05, 'Nacho_O': 3.3215970238490664e-05, 'felicidades_O': 3.3215970238490664e-05, '18_O': 6.643194047698133e-05, 'sé_O': 6.643194047698133e-05, 'feliz_O': 3.3215970238490664e-05, 'Perfecto_O': 8.303992559622666e-05, 'vuestro_O': 6.643194047698133e-05, 'Locos_O': 3.3215970238490664e-05, 'resentidos_O': 3.3215970238490664e-05, 'permitida_O': 3.3215970238490664e-05, 'entrada_O': 9.964791071547199e-05, 'Bacalao_O': 3.3215970238490664e-05, 'asado_O': 4.9823955357735996e-05, 'vinagreta_O': 3.3215970238490664e-05, 'tomate_O': 0.00013286388095396266, 'borrajas_O': 3.3215970238490664e-05, 'uno_O': 0.00039859164286188797, 'menus_O': 6.643194047698133e-05, 'preparados_O': 3.3215970238490664e-05, 'equivocamos_O': 3.3215970238490664e-05, 'cambiasen_O': 3.3215970238490664e-05, 'ahí_O': 6.643194047698133e-05, 'esos_O': 8.303992559622666e-05, 'exigente_O': 3.3215970238490664e-05, 'Está_O': 3.3215970238490664e-05, 'lugar_B-positive': 0.0005566379070414695, 'estupendamente_O': 4.9823955357735996e-05, 'presentado_O': 8.303992559622666e-05, 'productos_B-positive': 0.0002164702971827937, 'primera_O': 0.0002823357470271707, 'cantidades_B-negative': 9.506305849546866e-05, 'bajo_O': 6.643194047698133e-05, 'vista_O': 9.964791071547199e-05, 'escasas_O': 8.303992559622666e-05, 'Local_B-positive': 0.00015462164084485266, 'completamente_O': 4.9823955357735996e-05, 'blancas_O': 4.9823955357735996e-05, 'limpio_O': 3.3215970238490664e-05, 'San_O': 6.643194047698133e-05, 'Jordi_O': 4.9823955357735996e-05, 'familia_O': 9.964791071547199e-05, 'redondo_O': 3.3215970238490664e-05, 'destaca_O': 3.3215970238490664e-05, 'atentísimo_O': 3.3215970238490664e-05, 'ves_O': 3.3215970238490664e-05, 'ganas_O': 0.00011625589583471734, 'sí_O': 0.00018268783631169867, 'mantienen_O': 6.643194047698133e-05, 'nivel_O': 0.00018268783631169867, 'innovación_O': 3.3215970238490664e-05, 'soplo_O': 3.3215970238490664e-05, 'aire_O': 3.3215970238490664e-05, 'fresco_O': 9.964791071547199e-05, 'calle_O': 8.303992559622666e-05, 'zona_B-negative': 6.337537233031244e-05, 'dicho_O': 0.00018268783631169867, 'valoro_O': 6.643194047698133e-05, 'amplitud_O': 3.3215970238490664e-05, 'mesas_B-positive': 0.00015462164084485266, 'redondas_O': 3.3215970238490664e-05, 'explicar_O': 0.00011625589583471734, 'económico_O': 4.9823955357735996e-05, 'ocupan_O': 3.3215970238490664e-05, 'sólo_O': 0.0002657277619079253, 'comensales_O': 0.00019929582143094398, 'significa_O': 3.3215970238490664e-05, 'menor_O': 4.9823955357735996e-05, 'facturación_O': 3.3215970238490664e-05, 'camareras_B-positive': 6.184865633794106e-05, 'recibida_O': 4.9823955357735996e-05, 'vuestra_O': 4.9823955357735996e-05, 'Con_O': 0.00016607985119245332, 'tantas_O': 3.3215970238490664e-05, 'virtudes_O': 3.3215970238490664e-05, 'esperaba_O': 6.643194047698133e-05, 'cañazo_O': 3.3215970238490664e-05, 'final_O': 0.00016607985119245332, 'familiar_O': 0.00011625589583471734, 'celebramos_O': 3.3215970238490664e-05, 'todavía_O': 4.9823955357735996e-05, 'bailar.La_O': 3.3215970238490664e-05, 'cena_B-neutral': 6.40020480655381e-05, 'Parecía_O': 4.9823955357735996e-05, 'quiseran_O': 3.3215970238490664e-05, 'fueramos_O': 3.3215970238490664e-05, 'enseguida_O': 3.3215970238490664e-05, 'llenar_O': 4.9823955357735996e-05, 'vez.Para_O': 3.3215970238490664e-05, 'terraza_B-negative': 9.506305849546866e-05, 'arriba_O': 8.303992559622666e-05, 'evento_O': 4.9823955357735996e-05, 'politico_O': 3.3215970238490664e-05, 'haciendo_O': 8.303992559622666e-05, 'campaña_O': 3.3215970238490664e-05, 'altavoces_O': 3.3215970238490664e-05, 'cosa_O': 0.000149471866073208, 'interesaba_O': 3.3215970238490664e-05, 'minimo_O': 3.3215970238490664e-05, 'Era_O': 8.303992559622666e-05, 'imprescindible_O': 3.3215970238490664e-05, 'cita_O': 3.3215970238490664e-05, 'previa_O': 3.3215970238490664e-05, 'avisé_O': 3.3215970238490664e-05, 'eligiendo_O': 3.3215970238490664e-05, 'martes_O': 8.303992559622666e-05, 'Joaquina_O': 4.9823955357735996e-05, 'aclaraciones_O': 3.3215970238490664e-05, 'respecto_O': 6.643194047698133e-05, 'nuevos_O': 4.9823955357735996e-05, 'artículos_O': 3.3215970238490664e-05, 'momento.Solamente_O': 3.3215970238490664e-05, 'aclarar_O': 3.3215970238490664e-05, 'crítica_O': 6.643194047698133e-05, 'hablo_O': 4.9823955357735996e-05, 'contando_O': 4.9823955357735996e-05, 'utilizar_O': 4.9823955357735996e-05, 'servicios_O': 3.3215970238490664e-05, 'empresa_O': 8.303992559622666e-05, 'cuales_O': 3.3215970238490664e-05, 'pagué_O': 3.3215970238490664e-05, 'rigurosamente_O': 3.3215970238490664e-05, 'hablar_O': 9.964791071547199e-05, 'ternasco_B-positive': 9.277298450691158e-05, 'uff_O': 3.3215970238490664e-05, 'todavíalo_O': 3.3215970238490664e-05, 'saboreo_O': 3.3215970238490664e-05, 'comelón_O': 3.3215970238490664e-05, 'apartado_O': 9.964791071547199e-05, 'sabroso_O': 9.964791071547199e-05, 'olvidar_O': 8.303992559622666e-05, 'aquel_O': 4.9823955357735996e-05, 'paladar_O': 3.3215970238490664e-05, 'Óptimo_O': 4.9823955357735996e-05, 'confundido_O': 4.9823955357735996e-05, 'Visité_O': 3.3215970238490664e-05, 'Scala_O': 3.3215970238490664e-05, 'aproximadamente_O': 4.9823955357735996e-05, 'será_O': 9.964791071547199e-05, 'salgo_O': 3.3215970238490664e-05, 'gratamente_O': 3.3215970238490664e-05, 'sorprendido_O': 6.643194047698133e-05, 'no-recomendación_O': 3.3215970238490664e-05, 'eran_O': 0.00019929582143094398, 'confianza_O': 3.3215970238490664e-05, 'Ciertamente_O': 3.3215970238490664e-05, 'desilusionante_O': 3.3215970238490664e-05, 'debido_O': 6.643194047698133e-05, 'escasez_O': 3.3215970238490664e-05, 's_O': 3.3215970238490664e-05, 'egundos_B-negative': 6.337537233031244e-05, 'platos_I-negative': 6.379992344009187e-05, 'describir_O': 3.3215970238490664e-05, 'clásico_O': 3.3215970238490664e-05, 'ubicacion_B-positive': 6.184865633794106e-05, 'verano_O': 8.303992559622666e-05, 'invierno_O': 3.3215970238490664e-05, 'Ambiente_B-positive': 0.00027831895352073476, 'Ambiente_B-negative': 6.337537233031244e-05, 'cargado_O': 3.3215970238490664e-05, 'divina_O': 3.3215970238490664e-05, 'pongo_O': 4.9823955357735996e-05, 'poquito_O': 4.9823955357735996e-05, 'animado_O': 3.3215970238490664e-05, 'dormíamos_O': 3.3215970238490664e-05, 'esperando_O': 3.3215970238490664e-05, 'espectáculo_O': 0.00013286388095396266, 'prometido_O': 3.3215970238490664e-05, 'antes_O': 0.00024911977678868, 'irme_O': 3.3215970238490664e-05, 'comente_O': 3.3215970238490664e-05, 'responsable_B-negative': 6.337537233031244e-05, 'barra_I-negative': 6.379992344009187e-05, 'contestaba_O': 3.3215970238490664e-05, 'principio_O': 8.303992559622666e-05, 'palabras_O': 3.3215970238490664e-05, 'culpa_O': 4.9823955357735996e-05, 'indicado_O': 3.3215970238490664e-05, '23:30_O': 6.643194047698133e-05, 'fuese_O': 3.3215970238490664e-05, 'hubiese_O': 9.964791071547199e-05, 'empezado_O': 3.3215970238490664e-05, 'piedras_O': 3.3215970238490664e-05, 'esas_O': 3.3215970238490664e-05, 'venden_O': 3.3215970238490664e-05, 'tiendas_O': 3.3215970238490664e-05, 'turísticas_O': 3.3215970238490664e-05, 'acompañamiento_O': 4.9823955357735996e-05, '…_B-negative': 6.337537233031244e-05, '¡_O': 0.00023251179166943468, 'nombre_O': 0.00011625589583471734, 'próximo_O': 4.9823955357735996e-05, 'vosotros_O': 3.3215970238490664e-05, 'dejaies_O': 3.3215970238490664e-05, 'preparar_O': 3.3215970238490664e-05, 'cosas_O': 0.00013286388095396266, 'nuestro_O': 0.00021590380655018933, 'cariño_O': 8.303992559622666e-05, 'amor_O': 3.3215970238490664e-05, 'acabe_O': 3.3215970238490664e-05, 'diga_O': 9.964791071547199e-05, 'deja_O': 0.00013286388095396266, 'querer_O': 4.9823955357735996e-05, 'seguir_O': 9.964791071547199e-05, 'nuevas_O': 3.3215970238490664e-05, 'ideasgracias_O': 3.3215970238490664e-05, 'saludos_O': 4.9823955357735996e-05, 'Reserve_O': 3.3215970238490664e-05, 'estuve_O': 0.00016607985119245332, 'estando_O': 4.9823955357735996e-05, 'born_O': 4.9823955357735996e-05, 'JUAS_O': 3.3215970238490664e-05, 'repito_O': 3.3215970238490664e-05, 'das_O': 6.643194047698133e-05, 'respectivos_O': 3.3215970238490664e-05, 'parámetros_O': 3.3215970238490664e-05, 'calificar_O': 6.643194047698133e-05, 'Px_O': 3.3215970238490664e-05, 'dices_O': 6.643194047698133e-05, 'tú_O': 9.964791071547199e-05, 'peso_O': 4.9823955357735996e-05, 'Cx_O': 3.3215970238490664e-05, 'explicada_O': 3.3215970238490664e-05, 'sirve_O': 8.303992559622666e-05, 'Mi_O': 0.00023251179166943468, 'quiere_O': 6.643194047698133e-05, 'pretencioso_O': 3.3215970238490664e-05, 'dista_O': 3.3215970238490664e-05, 'innovador_O': 4.9823955357735996e-05, 'publicitan_O': 3.3215970238490664e-05, 'quién_O': 3.3215970238490664e-05, 'advirtió_O': 3.3215970238490664e-05, 'luis_O': 4.9823955357735996e-05, 'éste_O': 6.643194047698133e-05, 'lunes_O': 3.3215970238490664e-05, 'pues_O': 0.00016607985119245332, 'tinto_I-positive': 9.52653138992093e-05, 'Viñas_O': 4.9823955357735996e-05, 'Vero_O': 4.9823955357735996e-05, 'Pago_O': 3.3215970238490664e-05, 'Ayles_O': 3.3215970238490664e-05, 'cafés_B-positive': 6.184865633794106e-05, 'chupito_B-positive': 6.184865633794106e-05, 'dentro_O': 9.964791071547199e-05, 'algunos_O': 0.000149471866073208, 'originales_O': 3.3215970238490664e-05, 'gazpacho_B-positive': 9.277298450691158e-05, 'cerezas_I-positive': 9.52653138992093e-05, 'Charly_I-positive': 6.351020926613953e-05, 'chipirones_B-positive': 6.184865633794106e-05, 'pasta_B-positive': 6.184865633794106e-05, 'carrillera_B-positive': 6.184865633794106e-05, 'lento_O': 0.00013286388095396266, 'grupos_O': 0.00013286388095396266, 'funcionan_O': 3.3215970238490664e-05, 'horario_O': 3.3215970238490664e-05, 'extricto_O': 3.3215970238490664e-05, 'con_I-positive': 0.00031755104633069764, 'almejas_I-positive': 6.351020926613953e-05, 'en_I-positive': 6.351020926613953e-05, 'salsa_I-positive': 9.52653138992093e-05, 'verde_I-positive': 9.52653138992093e-05, 'pocos_O': 0.000149471866073208, 'preciosa_O': 3.3215970238490664e-05, 'vistas_O': 8.303992559622666e-05, 'puerto_O': 4.9823955357735996e-05, 'opcion_O': 6.643194047698133e-05, 'disfrutar_O': 0.00019929582143094398, 'ofrecen_O': 9.964791071547199e-05, 'Flauta_O': 3.3215970238490664e-05, 'Asador_O': 4.9823955357735996e-05, 'Aranda_O': 9.964791071547199e-05, 'acordaros_O': 3.3215970238490664e-05, 'zaragoza_O': 8.303992559622666e-05, 'pueblo_O': 3.3215970238490664e-05, 'general_O': 0.0002657277619079253, 'individuales_O': 3.3215970238490664e-05, 'serán_O': 3.3215970238490664e-05, 'mismos_O': 4.9823955357735996e-05, 'Empecé_O': 4.9823955357735996e-05, 'pie_O': 4.9823955357735996e-05, 'continué_O': 3.3215970238490664e-05, 'escogiendo_O': 3.3215970238490664e-05, 'idóneo_O': 4.9823955357735996e-05, 'charlar_O': 3.3215970238490664e-05, 'atendida_O': 3.3215970238490664e-05, 'Es_O': 0.00046502358333886935, 'gozo_O': 3.3215970238490664e-05, 'deleite_O': 3.3215970238490664e-05, 'mesa_B-positive': 9.277298450691158e-05, 'mantel_O': 3.3215970238490664e-05, 'tí_O': 3.3215970238490664e-05, 'acompañante_O': 6.643194047698133e-05, '9R_B-positive': 6.184865633794106e-05, 'trataron_O': 4.9823955357735996e-05, 'Julio_B-positive': 6.184865633794106e-05, 'divertido_O': 6.643194047698133e-05, 'trató_O': 4.9823955357735996e-05, 'maravilla_O': 0.00013286388095396266, 'excesiva_O': 4.9823955357735996e-05, 'valoración_O': 4.9823955357735996e-05, 'desaconsejar_O': 3.3215970238490664e-05, 'sabiendo_O': 3.3215970238490664e-05, 'postres_B-positive': 0.0004020162661966169, 'deliciosos_O': 3.3215970238490664e-05, 'todos.Repetiremos_O': 3.3215970238490664e-05, 'Señor_O': 3.3215970238490664e-05, 'Rubira_O': 3.3215970238490664e-05, 'comento_O': 4.9823955357735996e-05, 'WC_B-positive': 6.184865633794106e-05, 'limpios_O': 4.9823955357735996e-05, 'espaciosos_O': 3.3215970238490664e-05, 'Lidl_O': 3.3215970238490664e-05, ';_O': 0.000348767687504152, 'anchoas_B-negative': 6.337537233031244e-05, 'marido_O': 6.643194047698133e-05, 'empezo_O': 3.3215970238490664e-05, 'datos_O': 3.3215970238490664e-05, 'efectivamente_O': 3.3215970238490664e-05, 'aparecia_O': 3.3215970238490664e-05, 'clarisima_O': 3.3215970238490664e-05, 'fantástico_O': 6.643194047698133e-05, 'marco_O': 3.3215970238490664e-05, 'terraza_B-positive': 0.00018554596901382316, 'Muelle_O': 3.3215970238490664e-05, 'España_O': 3.3215970238490664e-05, 'Anoche_O': 3.3215970238490664e-05, 'Casa_O': 4.9823955357735996e-05, 'Pelentós_O': 4.9823955357735996e-05, 'Sandiniés_O': 4.9823955357735996e-05, 'hicimos_O': 0.00011625589583471734, 'mayor_O': 0.00016607985119245332, 'fortuna_O': 3.3215970238490664e-05, 'ello_O': 6.643194047698133e-05, 'quisimos_O': 3.3215970238490664e-05, 'Volveremos_O': 0.00024911977678868, 'precios_B-positive': 6.184865633794106e-05, 'ajustados_I-positive': 6.351020926613953e-05, 'seguimos_O': 3.3215970238490664e-05, 'combinados_B-positive': 6.184865633794106e-05, 'fantastica_O': 3.3215970238490664e-05, 'muchas_O': 6.643194047698133e-05, 'familiares_O': 4.9823955357735996e-05, 'allegados_O': 3.3215970238490664e-05, 'Madre_O': 3.3215970238490664e-05, 'mía_O': 4.9823955357735996e-05, 'despropósito_O': 3.3215970238490664e-05, 'van_O': 0.00013286388095396266, 'cuantas_O': 4.9823955357735996e-05, 'mira_O': 3.3215970238490664e-05, 'entusiasmó_O': 3.3215970238490664e-05, 'comprendí_O': 3.3215970238490664e-05, 'concedisen_O': 3.3215970238490664e-05, 'michelín_O': 3.3215970238490664e-05, 'paro_O': 3.3215970238490664e-05, 'criterio_O': 4.9823955357735996e-05, 'terrible_O': 3.3215970238490664e-05, 'prejuicios_O': 3.3215970238490664e-05, 'flor_O': 4.9823955357735996e-05, 'piel_O': 4.9823955357735996e-05, 'valorarlo_O': 3.3215970238490664e-05, 'cachirulos_O': 4.9823955357735996e-05, 'después_O': 9.964791071547199e-05, 'sutiles_O': 3.3215970238490664e-05, 'buenísimos_O': 3.3215970238490664e-05, 'refrescantes_O': 3.3215970238490664e-05, 'excelentes_O': 0.00019929582143094398, 'atrevidos_O': 3.3215970238490664e-05, 'mayoría_O': 3.3215970238490664e-05, 'ellos_O': 0.00016607985119245332, 'cena_B-positive': 0.0003401676098586758, 'impecable_O': 0.00016607985119245332, 'fabulosa_O': 4.9823955357735996e-05, 'amplia_O': 9.964791071547199e-05, 'perfectos_O': 4.9823955357735996e-05, 'encantó_O': 8.303992559622666e-05, 'filete_B-positive': 9.277298450691158e-05, 'vedella_I-positive': 6.351020926613953e-05, 'al_I-positive': 0.00022228573243148836, 'foie_I-positive': 6.351020926613953e-05, 'doy_O': 9.964791071547199e-05, 'puntuación_O': 3.3215970238490664e-05, 'realmente_O': 0.0002657277619079253, 'Restaurante_O': 4.9823955357735996e-05, 'Puerto_O': 4.9823955357735996e-05, 'Santa_O': 3.3215970238490664e-05, 'María_O': 4.9823955357735996e-05, 'gaditana_O': 3.3215970238490664e-05, 'pescados_O': 4.9823955357735996e-05, 'carnes_O': 4.9823955357735996e-05, '35-45_O': 3.3215970238490664e-05, 'gambas_B-positive': 0.00015462164084485266, 'sobretodo_O': 9.964791071547199e-05, 'invitan_O': 3.3215970238490664e-05, 'aceptable_O': 0.000149471866073208, 'descompensada.se_O': 3.3215970238490664e-05, 'tendría_O': 3.3215970238490664e-05, 'cocinero_O': 4.9823955357735996e-05, 'pulpo_B-negative': 9.506305849546866e-05, 'calienta_O': 4.9823955357735996e-05, 'microondas_O': 4.9823955357735996e-05, 'pone_O': 6.643194047698133e-05, 'pimentón_O': 4.9823955357735996e-05, 'aceite_O': 0.00013286388095396266, 'quema_O': 3.3215970238490664e-05, 'reseca_O': 3.3215970238490664e-05, 'Contenta_O': 3.3215970238490664e-05, 'prestado_O': 3.3215970238490664e-05, 'ágil_O': 3.3215970238490664e-05, 'gentil_O': 3.3215970238490664e-05, 'adaptados_O': 3.3215970238490664e-05, 'menús_B-positive': 9.277298450691158e-05, 'parejas_O': 8.303992559622666e-05, 'Algo_O': 9.964791071547199e-05, 'encontrar_O': 0.00016607985119245332, 'alguna_O': 8.303992559622666e-05, 'sensacional_O': 3.3215970238490664e-05, 'pequeños_B-positive': 6.184865633794106e-05, 'detalles_I-positive': 6.351020926613953e-05, 'alta_O': 0.00011625589583471734, 'logrados_O': 4.9823955357735996e-05, 'ambos_O': 4.9823955357735996e-05, 'Ni_O': 3.3215970238490664e-05, 'sres_O': 3.3215970238490664e-05, 'parte_O': 0.00021590380655018933, 'pica_B-neutral': 6.40020480655381e-05, 'pica_I-neutral': 6.406149903907751e-05, 'Sólo_O': 6.643194047698133e-05, 'cómoda_O': 3.3215970238490664e-05, 'situación_B-positive': 0.00012369731267588213, 'trasladarse_O': 3.3215970238490664e-05, 'transporte_O': 3.3215970238490664e-05, 'público_O': 3.3215970238490664e-05, 'metro_O': 3.3215970238490664e-05, 'bus_O': 3.3215970238490664e-05, 'velada_O': 6.643194047698133e-05, 'Puedo_O': 4.9823955357735996e-05, 'entender_O': 6.643194047698133e-05, '30_O': 0.00019929582143094398, 'minutos_O': 6.643194047698133e-05, 'retraso_O': 3.3215970238490664e-05, 'fiestas_O': 3.3215970238490664e-05, 'Pilar_O': 3.3215970238490664e-05, 'hacen_O': 0.00016607985119245332, 'caja_O': 3.3215970238490664e-05, '1_O': 0.00013286388095396266, 'otro_O': 0.0002823357470271707, 'Q-Art_O': 3.3215970238490664e-05, 'Parecían_O': 3.3215970238490664e-05, 'taberna_O': 3.3215970238490664e-05, 'tenedores_O': 3.3215970238490664e-05, 'Servicio_O': 4.9823955357735996e-05, 'recomendaria_O': 3.3215970238490664e-05, 'raciones_B-negative': 0.00019012611699093733, 'cobran_O': 6.643194047698133e-05, 'ejemplo_O': 0.00013286388095396266, 'con_I-negative': 0.0003508995789205053, 'tres_I-negative': 9.56998851601378e-05, 'colmenillas_I-negative': 6.379992344009187e-05, 'foi_I-negative': 6.379992344009187e-05, 'corazones_I-negative': 6.379992344009187e-05, 'alcachofas_I-negative': 6.379992344009187e-05, 'rebozado_I-negative': 6.379992344009187e-05, 'en_I-negative': 6.379992344009187e-05, 'tempura_I-negative': 6.379992344009187e-05, 'sablean_O': 3.3215970238490664e-05, '22_O': 6.643194047698133e-05, 'solomillo_B-negative': 9.506305849546866e-05, 'llega_O': 6.643194047698133e-05, 'escasamente_O': 3.3215970238490664e-05, '60_O': 6.643194047698133e-05, 'gramos_O': 3.3215970238490664e-05, 'Sábado_O': 6.643194047698133e-05, 'reservamos_O': 8.303992559622666e-05, 'degustación_O': 0.00016607985119245332, 'B_O': 3.3215970238490664e-05, '26_O': 4.9823955357735996e-05, 'tenian_O': 0.00011625589583471734, 'dijeron_O': 9.964791071547199e-05, '29_O': 3.3215970238490664e-05, 'faltaban_O': 4.9823955357735996e-05, 'hizo_O': 4.9823955357735996e-05, '35_O': 3.3215970238490664e-05, 'copa_O': 0.00016607985119245332, 'incluida_O': 4.9823955357735996e-05, 'elaborados_O': 6.643194047698133e-05, 'alto_O': 0.000149471866073208, 'Ninguna_O': 6.643194047698133e-05, 'Realmente_O': 9.964791071547199e-05, 'eramos_O': 0.00011625589583471734, 'grupo_O': 0.00013286388095396266, 'decoracion_B-positive': 0.00012369731267588213, 'moderna_O': 3.3215970238490664e-05, 'economico_O': 3.3215970238490664e-05, 'low_O': 3.3215970238490664e-05, 'cost_O': 3.3215970238490664e-05, 'recomendaría_O': 4.9823955357735996e-05, 'fácil_O': 4.9823955357735996e-05, 'aguantar_O': 4.9823955357735996e-05, 'interesados_O': 3.3215970238490664e-05, 'estira_O': 3.3215970238490664e-05, 'levitas_O': 3.3215970238490664e-05, 'moscones_O': 3.3215970238490664e-05, 'sinvergüenzas_O': 4.9823955357735996e-05, 'cobardes_O': 3.3215970238490664e-05, 'apellidos_O': 4.9823955357735996e-05, 'posición_O': 3.3215970238490664e-05, 'mundillo_O': 3.3215970238490664e-05, 'insultan_O': 3.3215970238490664e-05, 'escriben_O': 3.3215970238490664e-05, 'intruso_O': 3.3215970238490664e-05, 'piensa_O': 3.3215970238490664e-05, 'libertad_O': 3.3215970238490664e-05, 'podrás_O': 3.3215970238490664e-05, 'telefoneó_O': 3.3215970238490664e-05, 'confirmar_O': 4.9823955357735996e-05, 'ajustaron_O': 3.3215970238490664e-05, 'requerido_O': 3.3215970238490664e-05, 'amables_O': 3.3215970238490664e-05, 'arroz_B-positive': 0.0002164702971827937, 'mitad_O': 6.643194047698133e-05, 'Limon_B-positive': 6.184865633794106e-05, ',_I-positive': 9.52653138992093e-05, 'mango_I-positive': 6.351020926613953e-05, 'y_I-positive': 0.0001905306277984186, 'moscatel_I-positive': 6.351020926613953e-05, 'fresas_I-positive': 6.351020926613953e-05, 'vino_I-positive': 9.52653138992093e-05, 'Terminamos_O': 3.3215970238490664e-05, 'Gin_O': 3.3215970238490664e-05, 'Tonic_O': 3.3215970238490664e-05, 'preparacion_O': 3.3215970238490664e-05, 'gourmet_O': 3.3215970238490664e-05, 'ponia_O': 3.3215970238490664e-05, 'cupon_O': 3.3215970238490664e-05, 'descuento_O': 0.000149471866073208, '):_O': 3.3215970238490664e-05, 'combinado_O': 3.3215970238490664e-05, 'prepado_O': 3.3215970238490664e-05, 'rebosar_O': 3.3215970238490664e-05, 'cómo_O': 6.643194047698133e-05, 'excepto_O': 3.3215970238490664e-05, 'lugar_B-negative': 6.337537233031244e-05, 'Navidad_O': 8.303992559622666e-05, 'carpa_O': 3.3215970238490664e-05, 'magret_O': 3.3215970238490664e-05, 'pato_O': 4.9823955357735996e-05, 'postre_B-negative': 0.00012675074466062487, 'recuerdo_O': 9.964791071547199e-05, 'fu_O': 3.3215970238490664e-05, 'fa_O': 3.3215970238490664e-05, 'pesar_O': 0.000149471866073208, 'salimos_O': 0.00016607985119245332, 'corriente_O': 4.9823955357735996e-05, 'comparada_O': 3.3215970238490664e-05, '45_O': 3.3215970238490664e-05, 'forma_O': 8.303992559622666e-05, 'comparte_O': 3.3215970238490664e-05, 'importa_O': 6.643194047698133e-05, 'servido_O': 0.00011625589583471734, 'Solicité_O': 3.3215970238490664e-05, 'satisfecha_O': 3.3215970238490664e-05, 'JoséHe_O': 3.3215970238490664e-05, 'bastantes_O': 6.643194047698133e-05, 'cavas_O': 4.9823955357735996e-05, 'aragoneses_O': 4.9823955357735996e-05, 'Bordejé_O': 4.9823955357735996e-05, 'chardonnay_O': 3.3215970238490664e-05, 'Monasterio_O': 4.9823955357735996e-05, 'Veruela_O': 3.3215970238490664e-05, 'Corona_O': 3.3215970238490664e-05, 'dejan_O': 3.3215970238490664e-05, 'humilde_O': 4.9823955357735996e-05, 'áun_O': 3.3215970238490664e-05, 'acuerdo_O': 0.00011625589583471734, 'foie_B-positive': 6.184865633794106e-05, 'mangret_B-positive': 6.184865633794106e-05, 'pato_I-positive': 9.52653138992093e-05, 'fresa_I-positive': 6.351020926613953e-05, 'ok_O': 4.9823955357735996e-05, 'tengo_O': 6.643194047698133e-05, 'Estilo_O': 3.3215970238490664e-05, 'realcion_O': 3.3215970238490664e-05, 'Mentira_O': 3.3215970238490664e-05, 'deberian_O': 3.3215970238490664e-05, 'cerrar_O': 3.3215970238490664e-05, 'quisieron_O': 4.9823955357735996e-05, 'venian_O': 3.3215970238490664e-05, 'dto_O': 4.9823955357735996e-05, '%_O': 0.00021590380655018933, 'mintieron_O': 3.3215970238490664e-05, 'descaradisimamente_O': 3.3215970238490664e-05, 'Saludos_O': 0.00013286388095396266, 'ehorabuena_O': 3.3215970238490664e-05, 'Mucha_O': 4.9823955357735996e-05, 'precio_B-neutral': 6.40020480655381e-05, 'estable_O': 3.3215970238490664e-05, 'susto_O': 3.3215970238490664e-05, 'heroismo_O': 3.3215970238490664e-05, 'pasarse_O': 3.3215970238490664e-05, 'raro_O': 6.643194047698133e-05, 'Carlos_O': 4.9823955357735996e-05, 'haya_O': 9.964791071547199e-05, 'puesto_O': 0.00018268783631169867, 'metros_O': 3.3215970238490664e-05, 'pegados_O': 3.3215970238490664e-05, 'negocios_O': 6.643194047698133e-05, 'cabezudos_O': 3.3215970238490664e-05, 'meccano_O': 3.3215970238490664e-05, 'tinglado_O': 3.3215970238490664e-05, 'ademas_O': 9.964791071547199e-05, 'comun_O': 3.3215970238490664e-05, 'veo_O': 4.9823955357735996e-05, 'sabrá_O': 3.3215970238490664e-05, 'ironía_O': 3.3215970238490664e-05, 'capaz_O': 3.3215970238490664e-05, 'faltar_O': 4.9823955357735996e-05, 'insultar_O': 3.3215970238490664e-05, 'moscatel_O': 4.9823955357735996e-05, 'olvidado_O': 3.3215970238490664e-05, 'preparada_O': 4.9823955357735996e-05, 'acordado_O': 3.3215970238490664e-05, 'vía_O': 4.9823955357735996e-05, 'Web_O': 4.9823955357735996e-05, 'previsto_O': 3.3215970238490664e-05, 'utilizo_O': 3.3215970238490664e-05, 'apropiado_O': 4.9823955357735996e-05, 'Fernando_O': 4.9823955357735996e-05, 'Sí_O': 8.303992559622666e-05, 'precisamente_O': 4.9823955357735996e-05, 'tendrían_O': 3.3215970238490664e-05, 'rigorosos_O': 3.3215970238490664e-05, 'limpieza_O': 6.643194047698133e-05, 'baños_B-negative': 6.337537233031244e-05, 'brasa_B-positive': 9.277298450691158e-05, 'junto_O': 0.00019929582143094398, 'equilibrio_O': 3.3215970238490664e-05, 'degustación_I-positive': 0.0002857959416976279, 'completo_O': 0.000149471866073208, 'comunicado_O': 3.3215970238490664e-05, 'comodo_O': 3.3215970238490664e-05, 'Al_O': 9.964791071547199e-05, 'renovación_O': 3.3215970238490664e-05, 'quedado_O': 6.643194047698133e-05, 'anticuado_O': 3.3215970238490664e-05, 'Su_O': 6.643194047698133e-05, 'exquisitos_O': 8.303992559622666e-05, 'aportan_O': 4.9823955357735996e-05, 'añadido_O': 3.3215970238490664e-05, 'difícilmente_O': 3.3215970238490664e-05, 'inigualable_O': 6.643194047698133e-05, 'estos_O': 0.00018268783631169867, 'momentos_O': 4.9823955357735996e-05, 'llegamos_O': 4.9823955357735996e-05, 'preguntar_O': 3.3215970238490664e-05, 'teníamos_O': 6.643194047698133e-05, 'boca_O': 4.9823955357735996e-05, 'trozo_O': 4.9823955357735996e-05, 'neumático_O': 3.3215970238490664e-05, 'aquellas_O': 3.3215970238490664e-05, 'elegancia_O': 3.3215970238490664e-05, 'quieran_O': 4.9823955357735996e-05, 'Igual_O': 3.3215970238490664e-05, 'fué_O': 9.964791071547199e-05, 'fe_O': 6.643194047698133e-05, 'él_O': 8.303992559622666e-05, 'sabia_O': 4.9823955357735996e-05, 'pudimos_O': 6.643194047698133e-05, 'postres_O': 0.00011625589583471734, 'Tiramisú_B-positive': 6.184865633794106e-05, 'Queso_B-positive': 6.184865633794106e-05, 'Gallego_I-positive': 6.351020926613953e-05, 'Tetilla_I-positive': 6.351020926613953e-05, 'Dulce_I-positive': 6.351020926613953e-05, 'Membrillo_I-positive': 6.351020926613953e-05, 'Así_O': 4.9823955357735996e-05, 'Había_O': 4.9823955357735996e-05, 'diversas_O': 4.9823955357735996e-05, '2004_O': 3.3215970238490664e-05, '2005_O': 3.3215970238490664e-05, 'comparable_O': 4.9823955357735996e-05, 'encontré_O': 6.643194047698133e-05, 'Enero_O': 3.3215970238490664e-05, '2009_O': 4.9823955357735996e-05, 'atención_B-negative': 0.00022181380315609354, 'pésima_O': 6.643194047698133e-05, 'justita_O': 3.3215970238490664e-05, 'esmerado_O': 3.3215970238490664e-05, 'Paella_O': 3.3215970238490664e-05, 'pescado_O': 0.00016607985119245332, 'marisco_O': 8.303992559622666e-05, 'suegros_O': 3.3215970238490664e-05, 'quedaron_O': 6.643194047698133e-05, 'encantados_O': 8.303992559622666e-05, 'velocidad_B-positive': 6.184865633794106e-05, 'puntos_B-positive': 9.277298450691158e-05, 'cocción_I-positive': 9.52653138992093e-05, 'Solo_O': 0.000149471866073208, 'peros_O': 3.3215970238490664e-05, 'vino_B-negative': 0.00022181380315609354, 'rancidumbre_O': 3.3215970238490664e-05, 'salón_O': 4.9823955357735996e-05, '3_O': 0.00021590380655018933, 'tienda_O': 3.3215970238490664e-05, 'Le_O': 8.303992559622666e-05, 'voy_O': 0.00016607985119245332, 'intentar_O': 3.3215970238490664e-05, 'porqué_O': 4.9823955357735996e-05, 'estrellato_O': 3.3215970238490664e-05, 'tipo_O': 0.000149471866073208, 'respetos_O': 3.3215970238490664e-05, 'posible_O': 0.000149471866073208, 'mundo_O': 6.643194047698133e-05, 'vida_O': 6.643194047698133e-05, 'sentarnos_O': 4.9823955357735996e-05, 'dicen_O': 9.964791071547199e-05, 'ternera_I-negative': 9.56998851601378e-05, 'terminado_O': 4.9823955357735996e-05, 'llegar_O': 6.643194047698133e-05, 'avisando_O': 3.3215970238490664e-05, 'darían_O': 3.3215970238490664e-05, 'chuleta_O': 3.3215970238490664e-05, 'ternera_O': 9.964791071547199e-05, 'chasco_O': 4.9823955357735996e-05, 'requería_O': 3.3215970238490664e-05, 'masticarlo_O': 3.3215970238490664e-05, 'muuucho_O': 3.3215970238490664e-05, 'Ademas_O': 3.3215970238490664e-05, 'pasaron_O': 3.3215970238490664e-05, 'horas_O': 6.643194047698133e-05, 'volando_O': 3.3215970238490664e-05, 'empalmamos_O': 3.3215970238490664e-05, 'sentiamos_O': 3.3215970238490664e-05, 'Asi_O': 6.643194047698133e-05, 'basicamente_O': 3.3215970238490664e-05, 'segundos_O': 0.00013286388095396266, 'ensaladas_O': 3.3215970238490664e-05, 'Precio_O': 8.303992559622666e-05, 'fui_O': 0.00011625589583471734, 'erabamos_O': 3.3215970238490664e-05, 'Merece_O': 4.9823955357735996e-05, '32,50_O': 3.3215970238490664e-05, 'Aparte_O': 3.3215970238490664e-05, '6_O': 0.00011625589583471734, 'estube_O': 3.3215970238490664e-05, '6-12-_O': 3.3215970238490664e-05, '08_O': 4.9823955357735996e-05, 'fasion_O': 3.3215970238490664e-05, 'toralmente_O': 3.3215970238490664e-05, 'sillas_O': 6.643194047698133e-05, 'modernas_O': 4.9823955357735996e-05, 'comodas_O': 4.9823955357735996e-05, 'ningun_O': 8.303992559622666e-05, 'cuadro_O': 3.3215970238490664e-05, 'carta_B-negative': 0.00022181380315609354, 'atractiva_O': 3.3215970238490664e-05, 'calamares_B-negative': 9.506305849546866e-05, 'a_I-negative': 9.56998851601378e-05, 'la_I-negative': 0.00012759984688018373, 'andaluza_I-negative': 6.379992344009187e-05, 'imposibles_O': 3.3215970238490664e-05, 'pimientos_B-negative': 6.337537233031244e-05, 'cristal_I-negative': 6.379992344009187e-05, 'anchoas_I-negative': 6.379992344009187e-05, 'nadando_O': 3.3215970238490664e-05, 'mediocre_O': 6.643194047698133e-05, 'crema_I-negative': 6.379992344009187e-05, 'puerros_I-negative': 6.379992344009187e-05, 'tapado_O': 3.3215970238490664e-05, 'exagerada_O': 3.3215970238490664e-05, 'llamémosla_O': 3.3215970238490664e-05, 'ensalada_O': 9.964791071547199e-05, 'rucula_O': 3.3215970238490664e-05, 'conjunto_O': 6.643194047698133e-05, 'paletilla_B-positive': 6.184865633794106e-05, 'cordero_I-positive': 6.351020926613953e-05, 'ejecución_O': 3.3215970238490664e-05, 'flojos_O': 4.9823955357735996e-05, 'estuviemos_O': 3.3215970238490664e-05, 'anoche_O': 3.3215970238490664e-05, '1/08_O': 3.3215970238490664e-05, '09_O': 4.9823955357735996e-05, 'ibamos_O': 6.643194047698133e-05, 'raciones_O': 0.00013286388095396266, 'andaluza_O': 4.9823955357735996e-05, 'sacaron_O': 8.303992559622666e-05, 'mientras_O': 6.643194047698133e-05, 'esperabamos_O': 3.3215970238490664e-05, 'acompañado_O': 3.3215970238490664e-05, 'Blanc_O': 3.3215970238490664e-05, 'Pescador_O': 3.3215970238490664e-05, 'quedamos_O': 8.303992559622666e-05, 'pensamos_O': 3.3215970238490664e-05, 'salir_O': 6.643194047698133e-05, 'ortiguillas_B-positive': 6.184865633794106e-05, 'mar_I-positive': 6.351020926613953e-05, 'gustaron_O': 3.3215970238490664e-05, 'sorpresa_O': 0.00016607985119245332, 'agradecida_O': 3.3215970238490664e-05, 'conocer_O': 6.643194047698133e-05, 'Cafe_B-positive': 6.184865633794106e-05, 'casa_I-positive': 9.52653138992093e-05, 'les_I-positive': 6.351020926613953e-05, 'lletres_I-positive': 6.351020926613953e-05, 'super_O': 9.964791071547199e-05, 'sencillo_O': 4.9823955357735996e-05, 'elegante_O': 8.303992559622666e-05, 'insisto_O': 3.3215970238490664e-05, 'corregir_O': 3.3215970238490664e-05, 'errores_O': 8.303992559622666e-05, 'satisfacción_O': 4.9823955357735996e-05, 'producto_O': 6.643194047698133e-05, 'tardaron_O': 6.643194047698133e-05, 'sacarnos_O': 3.3215970238490664e-05, 'habérsela_O': 3.3215970238490664e-05, 'camareras_B-negative': 9.506305849546866e-05, 'Dices_O': 3.3215970238490664e-05, 'ayer_O': 0.00013286388095396266, 'sabado_O': 3.3215970238490664e-05, 'aniversario_O': 6.643194047698133e-05, 'Inmejorables_O': 3.3215970238490664e-05, 'vistas_B-positive': 0.00012369731267588213, 'Este_O': 9.964791071547199e-05, 'goza_O': 3.3215970238490664e-05, 'homogeneidad_O': 8.303992559622666e-05, 'envidiable_O': 4.9823955357735996e-05, 'Espero_O': 6.643194047698133e-05, 'sigan_O': 4.9823955357735996e-05, 'afán_O': 3.3215970238490664e-05, 'superación_O': 3.3215970238490664e-05, 'razonables_O': 8.303992559622666e-05, 'lejos_O': 4.9823955357735996e-05, 'Torres_O': 3.3215970238490664e-05, 'debajo_O': 4.9823955357735996e-05, 'Insisto_O': 3.3215970238490664e-05, 'emplazamiento_B-negative': 6.337537233031244e-05, 'emplatan_O': 3.3215970238490664e-05, 'porciones_O': 3.3215970238490664e-05, 'arroces_O': 3.3215970238490664e-05, 'degustacion_I-positive': 9.52653138992093e-05, 'pareció_O': 0.00013286388095396266, 'variado_O': 4.9823955357735996e-05, 'dan_O': 0.00011625589583471734, 'siguiente.Nos_O': 3.3215970238490664e-05, 'pronto_O': 8.303992559622666e-05, '07.02_O': 3.3215970238490664e-05, '″_O': 3.3215970238490664e-05, 'lujo_O': 8.303992559622666e-05, 'tortillitas_B-positive': 6.184865633794106e-05, 'camarones_I-positive': 6.351020926613953e-05, 'huelva_I-positive': 6.351020926613953e-05, 'entrecot_B-positive': 9.277298450691158e-05, 'parrilla_I-positive': 6.351020926613953e-05, 'jerez_I-positive': 6.351020926613953e-05, 'regado_O': 4.9823955357735996e-05, 'Enate_B-positive': 6.184865633794106e-05, '-_I-positive': 6.351020926613953e-05, 'cabernet_I-positive': 6.351020926613953e-05, 'suavignon_I-positive': 6.351020926613953e-05, 'comida.El_B-positive': 6.184865633794106e-05, 'local_B-neutral': 0.00016000512016384525, 'bullicioso_O': 3.3215970238490664e-05, 'trato_B-neutral': 6.40020480655381e-05, 'Primeros_O': 3.3215970238490664e-05, 'Ensalada_O': 3.3215970238490664e-05, 'contemporánea_O': 3.3215970238490664e-05, 'verduritas_O': 3.3215970238490664e-05, 'hortalizas_O': 3.3215970238490664e-05, 'temporada_O': 3.3215970238490664e-05, 'q-art_O': 3.3215970238490664e-05, 'salí_O': 4.9823955357735996e-05, 'descontento_O': 3.3215970238490664e-05, 'zona_O': 0.00016607985119245332, 'rico_O': 6.643194047698133e-05, 'Obviamente_O': 3.3215970238490664e-05, '2,50_O': 3.3215970238490664e-05, 'cobrados_O': 3.3215970238490664e-05, 'agua_B-negative': 0.00012675074466062487, 'del_I-negative': 0.00015949980860022967, 'grifo_I-negative': 6.379992344009187e-05, 'perdido_O': 6.643194047698133e-05, 'salvedad_O': 3.3215970238490664e-05, 'diré_O': 4.9823955357735996e-05, 'jefe_B-positive': 6.184865633794106e-05, 'sala_I-positive': 6.351020926613953e-05, 'propietario_O': 3.3215970238490664e-05, 'desconozco_O': 3.3215970238490664e-05, 'educado_O': 8.303992559622666e-05, 'detallista_O': 3.3215970238490664e-05, 'resto_B-negative': 6.337537233031244e-05, 'servicio_I-negative': 6.379992344009187e-05, 'pinganillos_O': 3.3215970238490664e-05, 'inalámbricos_O': 3.3215970238490664e-05, 'incluidos_O': 3.3215970238490664e-05, 'respecta_O': 6.643194047698133e-05, 'capacidad_O': 4.9823955357735996e-05, 'habría_O': 3.3215970238490664e-05, 'cuidar_O': 3.3215970238490664e-05, 'aspectos_O': 3.3215970238490664e-05, 'retirar_O': 3.3215970238490664e-05, 'carrera_O': 3.3215970238490664e-05, 'terminar_O': 4.9823955357735996e-05, 'tutear_O': 3.3215970238490664e-05, 'defecto_O': 3.3215970238490664e-05, 'u_O': 4.9823955357735996e-05, 'ofrecer_O': 4.9823955357735996e-05, 'silencio_O': 4.9823955357735996e-05, 'respuesta_O': 4.9823955357735996e-05, 'duda_O': 0.000298943732146416, 'comensal_O': 3.3215970238490664e-05, 'opciones_O': 4.9823955357735996e-05, 'último_O': 0.000149471866073208, 'ostras_B-negative': 6.337537233031244e-05, 'sentaron_O': 3.3215970238490664e-05, 'camarero_B-negative': 0.00019012611699093733, 'sirvió_O': 4.9823955357735996e-05, 'servía_O': 3.3215970238490664e-05, 'chorreando_O': 3.3215970238490664e-05, 'pasará_O': 3.3215970238490664e-05, 'vuelva_O': 4.9823955357735996e-05, 'segundo_B-negative': 6.337537233031244e-05, 'habido_O': 3.3215970238490664e-05, 'digo_O': 9.964791071547199e-05, '‘_O': 4.9823955357735996e-05, 'mu_O': 3.3215970238490664e-05, 'hallazgo_O': 3.3215970238490664e-05, 'pleno_O': 6.643194047698133e-05, 'barcelona_O': 0.00011625589583471734, 'coma_O': 3.3215970238490664e-05, 'laborables_O': 3.3215970238490664e-05, 'festivo_O': 3.3215970238490664e-05, 'tradicional_O': 3.3215970238490664e-05, 'conceptos_O': 3.3215970238490664e-05, 'aliña_O': 3.3215970238490664e-05, 'procedimientos_B-positive': 6.184865633794106e-05, 'cocina_I-positive': 9.52653138992093e-05, 'modernos_O': 3.3215970238490664e-05, 'aplicados_O': 3.3215970238490664e-05, 'presentaciones_B-positive': 9.277298450691158e-05, 'creativas_O': 3.3215970238490664e-05, 'ounto_O': 3.3215970238490664e-05, '9reinas_B-positive': 9.277298450691158e-05, 'defrauda_O': 4.9823955357735996e-05, 'contestaron_O': 3.3215970238490664e-05, 'mandé_O': 3.3215970238490664e-05, 'e.mail_O': 3.3215970238490664e-05, 'diciéndoles_O': 3.3215970238490664e-05, 'llamaran_O': 3.3215970238490664e-05, 'verduras_B-positive': 0.00015462164084485266, 'calçots_B-positive': 9.277298450691158e-05, 'hechos_O': 4.9823955357735996e-05, 'sabrosos_O': 4.9823955357735996e-05, 'Fue_O': 0.000149471866073208, '6.5_O': 3.3215970238490664e-05, 'calsots_B-negative': 6.337537233031244e-05, 'costaban_O': 3.3215970238490664e-05, 'pelar_O': 3.3215970238490664e-05, 'raíz_O': 3.3215970238490664e-05, 'cortada_O': 3.3215970238490664e-05, 'estabamos_O': 4.9823955357735996e-05, 'comentando_O': 3.3215970238490664e-05, 'mirabamos_O': 3.3215970238490664e-05, 'debió_O': 3.3215970238490664e-05, 'oir_O': 3.3215970238490664e-05, 'quitó_O': 3.3215970238490664e-05, 'delante_O': 4.9823955357735996e-05, 'narices_O': 3.3215970238490664e-05, '11:00_O': 3.3215970238490664e-05, 'cocineros_O': 3.3215970238490664e-05, 'limpiar_O': 4.9823955357735996e-05, 'olor_B-negative': 9.506305849546866e-05, 'detergente_O': 3.3215970238490664e-05, 'invade_O': 3.3215970238490664e-05, 'parecio_O': 8.303992559622666e-05, 'jamas_O': 3.3215970238490664e-05, 'repercutido_O': 3.3215970238490664e-05, 'valoracion_O': 3.3215970238490664e-05, 'Ya_O': 0.00013286388095396266, 'comparar_O': 3.3215970238490664e-05, 'Bodega_O': 3.3215970238490664e-05, 'Chema_O': 3.3215970238490664e-05, 'Broto_O': 3.3215970238490664e-05, 'Filoxera_O': 3.3215970238490664e-05, 'DGusta_O': 3.3215970238490664e-05, 'Utebo_O': 3.3215970238490664e-05, 'saciado_O': 3.3215970238490664e-05, 'similar_O': 3.3215970238490664e-05, 'bello_O': 3.3215970238490664e-05, 'Toda_O': 8.303992559622666e-05, 'despropósitos_O': 3.3215970238490664e-05, 'conseguido_O': 4.9823955357735996e-05, 'navidad_O': 4.9823955357735996e-05, 'convertido_O': 6.643194047698133e-05, 'Pesadilla_O': 3.3215970238490664e-05, 'Cocina_O': 3.3215970238490664e-05, 'alegro_O': 4.9823955357735996e-05, 'coincidir_O': 3.3215970238490664e-05, 'gustos_O': 3.3215970238490664e-05, 'saliéramos_O': 4.9823955357735996e-05, 'decepcionados_O': 6.643194047698133e-05, 'supongo_O': 4.9823955357735996e-05, 'gustado_O': 0.00011625589583471734, 'contentísimos_O': 3.3215970238490664e-05, 'd_O': 8.303992559622666e-05, 'recomendarlo_O': 6.643194047698133e-05, 'doquier_O': 3.3215970238490664e-05, 'dinero_O': 8.303992559622666e-05, 'único_O': 0.00016607985119245332, 'garantías_O': 3.3215970238490664e-05, 'Camareros_B-positive': 9.277298450691158e-05, 'buenisima_O': 8.303992559622666e-05, 'Llamados_O': 3.3215970238490664e-05, 'ultima_O': 4.9823955357735996e-05, 'enero_O': 8.303992559622666e-05, 'compromiso_O': 3.3215970238490664e-05, 'laboral_O': 3.3215970238490664e-05, 'casos_O': 6.643194047698133e-05, 'apuesta_O': 4.9823955357735996e-05, 'segura_O': 3.3215970238490664e-05, 'montar_O': 4.9823955357735996e-05, 'comedor_O': 0.00013286388095396266, 'llena_O': 3.3215970238490664e-05, 'reservar_O': 6.643194047698133e-05, 'Bodas_O': 3.3215970238490664e-05, 'Plata_O': 4.9823955357735996e-05, 'destacaría_O': 4.9823955357735996e-05, 'croquetas_B-positive': 6.184865633794106e-05, 'deliciosas_O': 3.3215970238490664e-05, 'paella_B-positive': 0.00024739462535176425, 'quería_O': 6.643194047698133e-05, 'referencias_O': 4.9823955357735996e-05, 'defraudó_O': 3.3215970238490664e-05, 'del_I-positive': 0.00012702041853227905, 'personal_I-positive': 9.52653138992093e-05, 'Exquisitas_O': 3.3215970238490664e-05, 'ensaladas_B-positive': 6.184865633794106e-05, 'exagerado_O': 4.9823955357735996e-05, 'olor_O': 8.303992559622666e-05, 'restaurantes_B-negative': 6.337537233031244e-05, 'sevicio_B-positive': 6.184865633794106e-05, 'intachable_O': 3.3215970238490664e-05, 'tradicionales_O': 4.9823955357735996e-05, 'adecuado_O': 8.303992559622666e-05, 'jefe_O': 3.3215970238490664e-05, 'puntuales_O': 4.9823955357735996e-05, 'Estos_O': 3.3215970238490664e-05, 'meramente_O': 3.3215970238490664e-05, 'pongan_O': 3.3215970238490664e-05, 'pilas_O': 3.3215970238490664e-05, 'alguien_O': 9.964791071547199e-05, 'piede_O': 3.3215970238490664e-05, 'RECOMENDARÉ_O': 3.3215970238490664e-05, 'NI_O': 3.3215970238490664e-05, 'MI_O': 3.3215970238490664e-05, 'PEOR_O': 3.3215970238490664e-05, 'ENEMIGO_O': 3.3215970238490664e-05, 'valió_O': 3.3215970238490664e-05, 'ración_B-positive': 6.184865633794106e-05, 'jamón_I-positive': 6.351020926613953e-05, 'CUATRO_B-positive': 6.184865633794106e-05, 'banderillas_I-positive': 6.351020926613953e-05, 'discrepo_O': 3.3215970238490664e-05, 'comunicante_O': 3.3215970238490664e-05, 'suelo_O': 3.3215970238490664e-05, 'Frances_O': 3.3215970238490664e-05, 'bajado_O': 6.643194047698133e-05, 'acogedorLo_O': 3.3215970238490664e-05, 'echa_O': 3.3215970238490664e-05, 'jose_O': 4.9823955357735996e-05, 'dueño_O': 6.643194047698133e-05, 'hijos_O': 4.9823955357735996e-05, 'pierde_O': 3.3215970238490664e-05, 'euna_O': 3.3215970238490664e-05, 'generacion_O': 3.3215970238490664e-05, 'carçot_B-positive': 6.184865633794106e-05, 'buena.Los_O': 3.3215970238490664e-05, 'debilidad_O': 3.3215970238490664e-05, 'huevo_B-positive': 0.00015462164084485266, 'pochado_I-positive': 6.351020926613953e-05, 'Imperdible_O': 3.3215970238490664e-05, 'l_O': 4.9823955357735996e-05, '´_O': 4.9823955357735996e-05, 'llegaron_O': 6.643194047698133e-05, 'Sobre_O': 6.643194047698133e-05, 'tema_O': 6.643194047698133e-05, 'humo_B-neutral': 6.40020480655381e-05, 'lleno_O': 0.00013286388095396266, 'semilleno_O': 3.3215970238490664e-05, 'humo_O': 3.3215970238490664e-05, 'encanta_O': 9.964791071547199e-05, '65_I-positive': 6.351020926613953e-05, 'grados_I-positive': 6.351020926613953e-05, 'sashimi_B-positive': 6.184865633794106e-05, 'atun_I-positive': 6.351020926613953e-05, 'flipar_O': 3.3215970238490664e-05, 'torrija_B-positive': 6.184865633794106e-05, 'ganar_O': 3.3215970238490664e-05, 'miércoles_O': 4.9823955357735996e-05, 'amigas_O': 6.643194047698133e-05, 'tenemos_O': 9.964791071547199e-05, 'claro_O': 0.00011625589583471734, 'ah_O': 4.9823955357735996e-05, 'plan_O': 4.9823955357735996e-05, 'cena_B-negative': 6.337537233031244e-05, 'romantica_I-negative': 6.379992344009187e-05, 'incompatible_O': 3.3215970238490664e-05, 'actuación_O': 4.9823955357735996e-05, 'barra_O': 9.964791071547199e-05, 'bar_O': 6.643194047698133e-05, 'amistades_O': 3.3215970238490664e-05, 'artistas_O': 3.3215970238490664e-05, 'invadió_O': 3.3215970238490664e-05, 'ambientillo_O': 3.3215970238490664e-05, 'intimo_O': 4.9823955357735996e-05, 'apetecñia_O': 3.3215970238490664e-05, 'toma_O': 3.3215970238490664e-05, 'poneros_O': 3.3215970238490664e-05, 'fondo_O': 6.643194047698133e-05, 'trasiego_O': 3.3215970238490664e-05, 'escenario_O': 4.9823955357735996e-05, 'puerta_O': 3.3215970238490664e-05, 'lujo.el_O': 3.3215970238490664e-05, 'servi_B-positive': 6.184865633794106e-05, 'io_I-positive': 6.351020926613953e-05, 'absolutamente_O': 3.3215970238490664e-05, 'emplatados_O': 3.3215970238490664e-05, 'seis_O': 3.3215970238490664e-05, 'celebración_O': 4.9823955357735996e-05, 'las_I-positive': 6.351020926613953e-05, 'setas_I-positive': 6.351020926613953e-05, 'muchísimo_O': 3.3215970238490664e-05, 'cocinado_O': 9.964791071547199e-05, 'combinaciones_O': 3.3215970238490664e-05, 'sabores_O': 8.303992559622666e-05, 'exquisitas_O': 3.3215970238490664e-05, 'satisfechos_O': 6.643194047698133e-05, 'cerrara_O': 3.3215970238490664e-05, 'venía_O': 3.3215970238490664e-05, 'venir_O': 3.3215970238490664e-05, 'relajado_O': 3.3215970238490664e-05, 'Carta_O': 3.3215970238490664e-05, '+_O': 4.9823955357735996e-05, 'placer_O': 4.9823955357735996e-05, 'Fantástico_O': 8.303992559622666e-05, 'vas_O': 6.643194047698133e-05, 'teatro_O': 3.3215970238490664e-05, 'Poliorama_O': 3.3215970238490664e-05, 'Balneario_O': 8.303992559622666e-05, 'realizando_O': 4.9823955357735996e-05, 'envoltura_O': 4.9823955357735996e-05, 'algas_O': 4.9823955357735996e-05, '1ª_O': 3.3215970238490664e-05, 'parese_O': 3.3215970238490664e-05, 'Cuando_O': 9.964791071547199e-05, 'repetiré_O': 6.643194047698133e-05, 'pica_B-negative': 9.506305849546866e-05, 'pica_I-negative': 9.56998851601378e-05, 'congelados_O': 3.3215970238490664e-05, 'aceitosos_O': 4.9823955357735996e-05, 'camarero_B-positive': 0.00015462164084485266, 'amablemente_O': 3.3215970238490664e-05, 'sacó_O': 3.3215970238490664e-05, 'petición_O': 3.3215970238490664e-05, 'ración_O': 8.303992559622666e-05, 'croquetas_O': 9.964791071547199e-05, 'huevos_O': 6.643194047698133e-05, 'Charly_O': 4.9823955357735996e-05, 'Estupenda_O': 3.3215970238490664e-05, 'sábado_O': 0.00013286388095396266, 'bono_O': 4.9823955357735996e-05, 'Groupalia_O': 3.3215970238490664e-05, 'Espectacular_O': 3.3215970238490664e-05, 'entraña_B-positive': 6.184865633794106e-05, 'necesitaba_O': 3.3215970238490664e-05, 'vacaciones_O': 4.9823955357735996e-05, 'Cosa_O': 3.3215970238490664e-05, 'pequeñas_O': 6.643194047698133e-05, 'comparación_O': 4.9823955357735996e-05, 'constaba_O': 3.3215970238490664e-05, 'previamente_O': 6.643194047698133e-05, 'hecha_O': 6.643194047698133e-05, 'vetula_O': 3.3215970238490664e-05, 'hablando_O': 6.643194047698133e-05, 'enseñe_O': 3.3215970238490664e-05, 'porcierto_O': 3.3215970238490664e-05, 'conocía_O': 3.3215970238490664e-05, 'acontinuación_O': 3.3215970238490664e-05, 'puso_O': 3.3215970238490664e-05, 'contácto_O': 3.3215970238490664e-05, 'resolver_O': 3.3215970238490664e-05, 'Hortigillas_B-negative': 6.337537233031244e-05, 'fatal_O': 8.303992559622666e-05, 'parecía_O': 6.643194047698133e-05, 'etabas_O': 3.3215970238490664e-05, 'arena_O': 3.3215970238490664e-05, 'playa_O': 3.3215970238490664e-05, 'Salieron_O': 3.3215970238490664e-05, 'Pica_B-positive': 6.184865633794106e-05, 'pica_I-positive': 9.52653138992093e-05, 'correctas_O': 3.3215970238490664e-05, 'equipo_O': 4.9823955357735996e-05, 'tuvimos_O': 0.00013286388095396266, 'Sr_O': 9.964791071547199e-05, 'Jose_O': 8.303992559622666e-05, 'pusiera_O': 3.3215970238490664e-05, 'colaboracion_O': 3.3215970238490664e-05, 'eligido_O': 3.3215970238490664e-05, 'ti_O': 3.3215970238490664e-05, 'fatigoso_O': 3.3215970238490664e-05, 'ejercicio_O': 3.3215970238490664e-05, 'mental_O': 3.3215970238490664e-05, 'serlo_O': 3.3215970238490664e-05, 'Impresionante_O': 4.9823955357735996e-05, 'aclaración_O': 3.3215970238490664e-05, 'Pedro_O': 8.303992559622666e-05, 'utilizan_O': 3.3215970238490664e-05, 'tendencia_O': 3.3215970238490664e-05, 'presentación_O': 0.00018268783631169867, 'contra_O': 4.9823955357735996e-05, 'utilizaran_O': 3.3215970238490664e-05, 'alterados_O': 3.3215970238490664e-05, 'genéticamente_O': 3.3215970238490664e-05, 'mo_O': 3.3215970238490664e-05, 'haría_O': 3.3215970238490664e-05, 'tanta_O': 8.303992559622666e-05, 'mediodía_O': 3.3215970238490664e-05, 'pague_O': 6.643194047698133e-05, 'novatada_O': 3.3215970238490664e-05, 'valores_O': 4.9823955357735996e-05, 'seguros_O': 4.9823955357735996e-05, 'Respecto_O': 9.964791071547199e-05, 'niños_O': 4.9823955357735996e-05, 'escaleras_O': 4.9823955357735996e-05, 'creceran_O': 3.3215970238490664e-05, 'tocado_O': 3.3215970238490664e-05, 'pretos_O': 3.3215970238490664e-05, 'comodos_O': 3.3215970238490664e-05, 'modo_O': 4.9823955357735996e-05, 'vamos_O': 0.00016607985119245332, 'hago_O': 4.9823955357735996e-05, 'relajo_O': 3.3215970238490664e-05, 'agradezco_O': 4.9823955357735996e-05, 'pedirlo_O': 3.3215970238490664e-05, 'qu_O': 3.3215970238490664e-05, 'sonrisa_O': 6.643194047698133e-05, 'muchisimos_O': 3.3215970238490664e-05, 'comparacion_O': 3.3215970238490664e-05, 'callos_B-positive': 6.184865633794106e-05, 'reclamar_O': 4.9823955357735996e-05, 'Lamentablemente_O': 3.3215970238490664e-05, 'enemigo_O': 4.9823955357735996e-05, 'queja_O': 6.643194047698133e-05, 'reserva_B-negative': 6.337537233031244e-05, 'sino_O': 0.00011625589583471734, 'Recuerdo_O': 3.3215970238490664e-05, 'tempura_O': 6.643194047698133e-05, 'verduras_O': 3.3215970238490664e-05, 'soja_O': 3.3215970238490664e-05, 'esperamos_O': 4.9823955357735996e-05, 'antemano_O': 3.3215970238490664e-05, 'Impecable_O': 4.9823955357735996e-05, 'mejillones_B-negative': 6.337537233031244e-05, 'al_I-negative': 9.56998851601378e-05, 'vapor_I-negative': 6.379992344009187e-05, 'ración_B-negative': 0.00012675074466062487, 'gambas_I-negative': 6.379992344009187e-05, 'siquiera_O': 6.643194047698133e-05, 'cubiertos_O': 6.643194047698133e-05, 'Velada_O': 3.3215970238490664e-05, 'óptima_O': 3.3215970238490664e-05, 'Servicio_B-neutral': 0.0001280040961310762, 'simpatía_O': 3.3215970238490664e-05, 'diligente_O': 4.9823955357735996e-05, 'voluntad_O': 3.3215970238490664e-05, 'cumplir_O': 3.3215970238490664e-05, 'reserva_B-positive': 6.184865633794106e-05, 'ventanal_O': 3.3215970238490664e-05, 'Ramblas_O': 3.3215970238490664e-05, 'Paracuellos_O': 6.643194047698133e-05, 'Jiloca_O': 6.643194047698133e-05, 'ESTUPENDA_O': 4.9823955357735996e-05, 'ponen_O': 3.3215970238490664e-05, 'calidad.La_O': 3.3215970238490664e-05, 'Del_O': 6.643194047698133e-05, 'cafés_O': 6.643194047698133e-05, 'Siempre_O': 9.964791071547199e-05, 'excelso_O': 3.3215970238490664e-05, 'Trato_B-neutral': 6.40020480655381e-05, 'lamentablemente_O': 3.3215970238490664e-05, 'balalao_B-negative': 6.337537233031244e-05, 'gratinado_O': 3.3215970238490664e-05, 'crudo_O': 4.9823955357735996e-05, 'carrillera_B-negative': 6.337537233031244e-05, 'vacuno_O': 3.3215970238490664e-05, 'recalentada_O': 3.3215970238490664e-05, 'paso_O': 9.964791071547199e-05, 'calentamiento_O': 3.3215970238490664e-05, 'seca_O': 4.9823955357735996e-05, 'partes_O': 3.3215970238490664e-05, 'delgadas_O': 3.3215970238490664e-05, 'Normalito_O': 3.3215970238490664e-05, 'chuletones_B-neutral': 6.40020480655381e-05, 'estuviera_O': 6.643194047698133e-05, 'sirvieron_O': 8.303992559622666e-05, 'sardinas_B-positive': 6.184865633794106e-05, 'brasa_O': 9.964791071547199e-05, 'espectaculares_O': 3.3215970238490664e-05, 'Eso_O': 4.9823955357735996e-05, 'servidas_O': 4.9823955357735996e-05, 'pequeños_O': 9.964791071547199e-05, 'cuencos_O': 3.3215970238490664e-05, 'buenísimo_O': 9.964791071547199e-05, 'myuy_O': 3.3215970238490664e-05, 'dejarse_O': 3.3215970238490664e-05, 'aconsejar_O': 6.643194047698133e-05, 'ellos_B-positive': 6.184865633794106e-05, 'Cena_O': 6.643194047698133e-05, 'negocio_O': 4.9823955357735996e-05, 'pp_O': 4.9823955357735996e-05, 'longaniza_O': 4.9823955357735996e-05, 'vic_O': 3.3215970238490664e-05, 'dorada_B-positive': 6.184865633794106e-05, 'estilo_I-positive': 6.351020926613953e-05, '“_I-positive': 6.351020926613953e-05, 'el_I-positive': 0.00012702041853227905, 'puerto_I-positive': 6.351020926613953e-05, '”_I-positive': 6.351020926613953e-05, 'salsa_B-positive': 9.277298450691158e-05, 'terminada_O': 3.3215970238490664e-05, 'descamar_O': 3.3215970238490664e-05, 'incordiante_O': 3.3215970238490664e-05, 'ella_O': 3.3215970238490664e-05, 'ubicado_O': 3.3215970238490664e-05, 'par_O': 0.000149471866073208, 'dia_I-positive': 6.351020926613953e-05, 'restaurants_B-positive': 6.184865633794106e-05, 'encantador_O': 4.9823955357735996e-05, 'detalle_B-positive': 6.184865633794106e-05, 'llevarse_O': 3.3215970238490664e-05, 'sobraba_O': 4.9823955357735996e-05, 'corvina_O': 3.3215970238490664e-05, 'sobrante_O': 3.3215970238490664e-05, 'envase_O': 3.3215970238490664e-05, 'Continuamos_O': 3.3215970238490664e-05, 'lotería_O': 3.3215970238490664e-05, 'bajan_O': 3.3215970238490664e-05, 'guardia_O': 3.3215970238490664e-05, 'trabajen_O': 3.3215970238490664e-05, 'tratamientos_O': 3.3215970238490664e-05, 'masajes_O': 3.3215970238490664e-05, 'disfrutamos_O': 3.3215970238490664e-05, 'piscina_O': 3.3215970238490664e-05, 'solarium_O': 3.3215970238490664e-05, 'ES_O': 3.3215970238490664e-05, 'diferencia_O': 4.9823955357735996e-05, 'elaboración_B-negative': 6.337537233031244e-05, 'variedad_B-negative': 6.337537233031244e-05, 'desayunos_B-negative': 6.337537233031244e-05, 'importante_O': 4.9823955357735996e-05, 'medio_O': 4.9823955357735996e-05, 'pan_B-negative': 9.506305849546866e-05, 'variedades_O': 3.3215970238490664e-05, 'harinas_O': 3.3215970238490664e-05, 'frutos.comenzamos_O': 3.3215970238490664e-05, 'rara_O': 3.3215970238490664e-05, 'agua_O': 0.00013286388095396266, 'embotellada_O': 3.3215970238490664e-05, 'plastico_O': 3.3215970238490664e-05, 'imcomprensible_O': 3.3215970238490664e-05, 'estupendos_O': 3.3215970238490664e-05, 'changet_B-negative': 6.337537233031244e-05, 'salmuera_O': 4.9823955357735996e-05, 'cantidades_B-positive': 6.184865633794106e-05, 'aceptables_O': 3.3215970238490664e-05, 'Cualquier_O': 3.3215970238490664e-05, 'casera_O': 4.9823955357735996e-05, 'fresca_O': 4.9823955357735996e-05, 'Lamento_O': 3.3215970238490664e-05, 'quien_O': 0.00011625589583471734, 'combinados_O': 3.3215970238490664e-05, 'preparan_O': 3.3215970238490664e-05, 'vaso_O': 3.3215970238490664e-05, 'hielo_O': 4.9823955357735996e-05, 'rodajita_O': 3.3215970238490664e-05, 'limon_O': 3.3215970238490664e-05, 'lima_O': 3.3215970238490664e-05, 'ginebra_O': 3.3215970238490664e-05, 'tonica_O': 3.3215970238490664e-05, 'suspicacia_O': 3.3215970238490664e-05, 'practica_O': 3.3215970238490664e-05, 'habitual_O': 4.9823955357735996e-05, 'pasa_O': 6.643194047698133e-05, 'mente_O': 3.3215970238490664e-05, 'dudar_O': 3.3215970238490664e-05, 'ingredientes_O': 4.9823955357735996e-05, 'queda_O': 9.964791071547199e-05, 'metres_B-negative': 6.337537233031244e-05, 'espantapajaros_O': 3.3215970238490664e-05, 'fantástica_O': 6.643194047698133e-05, 'tienes_O': 9.964791071547199e-05, 'levantar_O': 3.3215970238490664e-05, 'voz_O': 3.3215970238490664e-05, 'oirte_O': 3.3215970238490664e-05, 'ambiente_B-negative': 0.00022181380315609354, 'espera_O': 3.3215970238490664e-05, 'iré_O': 4.9823955357735996e-05, 'íré_O': 3.3215970238490664e-05, 'quedaré_O': 3.3215970238490664e-05, 'Matilde_O': 6.643194047698133e-05, 'octubre_O': 4.9823955357735996e-05, 'ensalada_B-positive': 0.00012369731267588213, 'bonito_I-positive': 6.351020926613953e-05, 'escabechado_I-positive': 6.351020926613953e-05, 'elaboradas_O': 3.3215970238490664e-05, 'sobraban_O': 3.3215970238490664e-05, 'mereció_O': 3.3215970238490664e-05, 'Comes_O': 3.3215970238490664e-05, 'elaborar_O': 4.9823955357735996e-05, 'pones_O': 4.9823955357735996e-05, 'agradables_O': 3.3215970238490664e-05, 'entrantes_B-positive': 0.00018554596901382316, 'Estupendo_O': 3.3215970238490664e-05, 'cabrito_B-positive': 0.00015462164084485266, 'Vuelta_O': 3.3215970238490664e-05, 'oportuno_O': 3.3215970238490664e-05, 'acabamos_O': 3.3215970238490664e-05, 'experiencia.Excelente_O': 3.3215970238490664e-05, 'veraniego_I-positive': 6.351020926613953e-05, 'arroz_I-positive': 6.351020926613953e-05, 'menú_B-neutral': 6.40020480655381e-05, 'calçots_I-neutral': 6.406149903907751e-05, 'condiciones_O': 3.3215970238490664e-05, 'Habrá_O': 3.3215970238490664e-05, 'razón_O': 4.9823955357735996e-05, '53_O': 4.9823955357735996e-05, 'cinco_O': 4.9823955357735996e-05, 'están_O': 0.00013286388095396266, 'restauración_O': 4.9823955357735996e-05, 'cambiado_O': 6.643194047698133e-05, 'asequibles_O': 4.9823955357735996e-05, 'comenta_O': 3.3215970238490664e-05, 'emergencias_O': 3.3215970238490664e-05, 'Ingeniosos_O': 3.3215970238490664e-05, 'cambiando_O': 3.3215970238490664e-05, 'gama_O': 4.9823955357735996e-05, 'muchos_O': 0.00019929582143094398, 'misma_O': 4.9823955357735996e-05, 'incluso_O': 0.00013286388095396266, 'puding_I-positive': 6.351020926613953e-05, 'delicioso_O': 6.643194047698133e-05, 'bote_O': 4.9823955357735996e-05, 'llevaba_O': 3.3215970238490664e-05, 'mereceria_O': 3.3215970238490664e-05, 'plantearse_O': 3.3215970238490664e-05, 'presentacion_O': 4.9823955357735996e-05, 'acompañaron_O': 4.9823955357735996e-05, 'solicito_O': 3.3215970238490664e-05, 'bajen_O': 3.3215970238490664e-05, 'extraordinario_O': 4.9823955357735996e-05, 'excelencia_O': 4.9823955357735996e-05, 'gasta_O': 3.3215970238490664e-05, 'sacan_O': 6.643194047698133e-05, 'pidas_O': 4.9823955357735996e-05, 'insistas_O': 3.3215970238490664e-05, 'gritarnos_O': 3.3215970238490664e-05, 'exigirnos_O': 3.3215970238490664e-05, 'íbamos_O': 4.9823955357735996e-05, 'quince_O': 3.3215970238490664e-05, 'murmullo_O': 3.3215970238490664e-05, 'llegan_O': 3.3215970238490664e-05, 'cobrar_O': 6.643194047698133e-05, '36_O': 4.9823955357735996e-05, 'directamente_O': 4.9823955357735996e-05, 'denuncia_O': 3.3215970238490664e-05, 'podemos_O': 6.643194047698133e-05, 'nombrar_O': 3.3215970238490664e-05, 'docena_O': 3.3215970238490664e-05, 'global_O': 4.9823955357735996e-05, 'acompañamos_O': 3.3215970238490664e-05, 'Atteca_B-positive': 6.184865633794106e-05, 'Calatayud_I-positive': 6.351020926613953e-05, 'recomendación_O': 3.3215970238490664e-05, 'corta_O': 3.3215970238490664e-05, 'cobra_O': 6.643194047698133e-05, 'confeccionada_O': 3.3215970238490664e-05, "L'Olivé_O": 4.9823955357735996e-05, '9.5_O': 3.3215970238490664e-05, 'Genial_O': 4.9823955357735996e-05, 'cajera_B-negative': 6.337537233031244e-05, 'recibio_O': 3.3215970238490664e-05, 'catalan_O': 4.9823955357735996e-05, 'dandose_O': 3.3215970238490664e-05, 'entendiamos_O': 3.3215970238490664e-05, 'volveria_O': 3.3215970238490664e-05, 'Ole_O': 3.3215970238490664e-05, 'peazo_O': 3.3215970238490664e-05, 'Pto.Sta_O': 3.3215970238490664e-05, 'bueno.To_O': 3.3215970238490664e-05, 'elijes_O': 3.3215970238490664e-05, 'mariscada_B-positive': 6.184865633794106e-05, 'camarero_O': 8.303992559622666e-05, 'recomedó_O': 3.3215970238490664e-05, 'llegó_O': 4.9823955357735996e-05, 'mariscada_O': 3.3215970238490664e-05, 'costó_O': 3.3215970238490664e-05, 'podríamos_O': 3.3215970238490664e-05, 'marchamos_O': 3.3215970238490664e-05, 'Raciones_B-positive': 0.00012369731267588213, 'amplias_O': 3.3215970238490664e-05, 'abundantes_O': 6.643194047698133e-05, 'organizáis_O': 3.3215970238490664e-05, 'queréis_O': 4.9823955357735996e-05, 'mantiene_O': 3.3215970238490664e-05, 'altos_O': 6.643194047698133e-05, 'constó_O': 3.3215970238490664e-05, 'ocho_O': 3.3215970238490664e-05, 'También_O': 4.9823955357735996e-05, 'rapidez_O': 3.3215970238490664e-05, 'anormalmente_O': 3.3215970238490664e-05, 'sevicio_B-negative': 6.337537233031244e-05, 'atencionado_O': 3.3215970238490664e-05, 'habla_O': 3.3215970238490664e-05, 'idiomas_O': 3.3215970238490664e-05, 'Poca_O': 3.3215970238490664e-05, 'intimidad_B-negative': 6.337537233031244e-05, 'Ana_O': 3.3215970238490664e-05, 'Anselmo_O': 4.9823955357735996e-05, 'apoyo_O': 3.3215970238490664e-05, 'relacion_O': 0.000149471866073208, 'servicui_B-positive': 6.184865633794106e-05, '100_O': 8.303992559622666e-05, 'triunfado_O': 3.3215970238490664e-05, 'Seguidamente_O': 3.3215970238490664e-05, 'advertirnos_O': 3.3215970238490664e-05, 'pregunta_O': 8.303992559622666e-05, 'blanco_I-negative': 9.56998851601378e-05, 'ante_O': 3.3215970238490664e-05, 'comenté_O': 4.9823955357735996e-05, 'iban_O': 6.643194047698133e-05, 'servir_O': 9.964791071547199e-05, 'vinos_O': 9.964791071547199e-05, 'aparecían_O': 3.3215970238490664e-05, 'cupón_O': 3.3215970238490664e-05, 'respondió_O': 3.3215970238490664e-05, 'suministraba_O': 3.3215970238490664e-05, 'noviembre_O': 4.9823955357735996e-05, 'estupendo_O': 0.00016607985119245332, 'habían_O': 8.303992559622666e-05, 'considerado_O': 3.3215970238490664e-05, 'deberían_O': 3.3215970238490664e-05, 'ajustar_O': 3.3215970238490664e-05, 'comprado_O': 3.3215970238490664e-05, 'estafa_O': 3.3215970238490664e-05, 'ven_O': 4.9823955357735996e-05, 'margen_O': 8.303992559622666e-05, 'saquen_O': 3.3215970238490664e-05, 'ajusten_O': 3.3215970238490664e-05, 'parezca_O': 3.3215970238490664e-05, 'consumidor_O': 3.3215970238490664e-05, 'comprar.Pues_O': 3.3215970238490664e-05, 'acaba_O': 3.3215970238490664e-05, 'preguntamos_O': 3.3215970238490664e-05, 'cava_B-negative': 6.337537233031244e-05, 'servían_O': 3.3215970238490664e-05, 'creían_O': 3.3215970238490664e-05, 'sangria_B-positive': 9.277298450691158e-05, 'cava_I-positive': 0.00012702041853227905, 'quedarse_O': 3.3215970238490664e-05, 'dejandote_O': 3.3215970238490664e-05, 'manera_O': 9.964791071547199e-05, 'quitan_O': 3.3215970238490664e-05, 'ojo_O': 4.9823955357735996e-05, 'deseos_O': 3.3215970238490664e-05, 'favor_O': 6.643194047698133e-05, 'vergüenza_O': 4.9823955357735996e-05, 'torera_O': 3.3215970238490664e-05, 'escribir_O': 3.3215970238490664e-05, 'tontadas_O': 3.3215970238490664e-05, 'daño_O': 3.3215970238490664e-05, 'montados_O': 3.3215970238490664e-05, 'dignidad_O': 3.3215970238490664e-05, 'vete_O': 3.3215970238490664e-05, 'macarrones_O': 3.3215970238490664e-05, 'gustan_O': 3.3215970238490664e-05, 'Pagamos_O': 4.9823955357735996e-05, 'bebidas_O': 8.303992559622666e-05, 'Agricultura_O': 3.3215970238490664e-05, 'ecológica_O': 3.3215970238490664e-05, 'frio_O': 0.00011625589583471734, 'emocionalmente_O': 3.3215970238490664e-05, 'Sigan_O': 3.3215970238490664e-05, 'pastelitos_O': 3.3215970238490664e-05, 'pastelería_O': 3.3215970238490664e-05, 'manteniendo_O': 3.3215970238490664e-05, '4º_O': 3.3215970238490664e-05, 'consecutivo_O': 3.3215970238490664e-05, 'UNICO_O': 4.9823955357735996e-05, 'sol_O': 3.3215970238490664e-05, 'Repsol_O': 3.3215970238490664e-05, 'distingue_O': 3.3215970238490664e-05, 'retsaurante_O': 3.3215970238490664e-05, 'doble_O': 3.3215970238490664e-05, 'reconocimiento_O': 3.3215970238490664e-05, 'guías_O': 3.3215970238490664e-05, 'referente_O': 4.9823955357735996e-05, 'gastronomía_O': 4.9823955357735996e-05, 'Más_O': 3.3215970238490664e-05, 'recochineo_O': 3.3215970238490664e-05, 'días_O': 4.9823955357735996e-05, 'llamaron_O': 4.9823955357735996e-05, 'quë_O': 3.3215970238490664e-05, 'aun_O': 0.00013286388095396266, 'tipos_O': 4.9823955357735996e-05, 'olorosos_O': 3.3215970238490664e-05, 'amontillados_O': 3.3215970238490664e-05, 'PX_O': 3.3215970238490664e-05, 'oportos_O': 3.3215970238490664e-05, 'coñac_O': 3.3215970238490664e-05, 'licores_O': 3.3215970238490664e-05, 'incluyendo_O': 4.9823955357735996e-05, 'puros_O': 3.3215970238490664e-05, 'Supongo_O': 4.9823955357735996e-05, 'S_O': 4.9823955357735996e-05, 'pudieron_O': 3.3215970238490664e-05, 'contestar_O': 3.3215970238490664e-05, 'Trato_B-positive': 0.0002164702971827937, 'concertado_O': 3.3215970238490664e-05, 'decente_O': 4.9823955357735996e-05, 'embargo_O': 4.9823955357735996e-05, 'daba_O': 3.3215970238490664e-05, 'sitios_B-positive': 9.277298450691158e-05, 'quedan_O': 3.3215970238490664e-05, 'paga_O': 6.643194047698133e-05, 'función_O': 4.9823955357735996e-05, 'motivos_O': 6.643194047698133e-05, 'rápido_O': 9.964791071547199e-05, 'cerca_O': 0.00011625589583471734, 'Pl.Urquinaona_O': 3.3215970238490664e-05, 'quejo_O': 3.3215970238490664e-05, 'cocina_B-negative': 0.00012675074466062487, 'Ellos_O': 3.3215970238490664e-05, 'note_O': 3.3215970238490664e-05, 'cara_O': 6.643194047698133e-05, 'febrero_O': 4.9823955357735996e-05, '2007_O': 4.9823955357735996e-05, ');_O': 3.3215970238490664e-05, 'ruidosos_O': 3.3215970238490664e-05, 'impertinentes_O': 3.3215970238490664e-05, 'imposible_O': 6.643194047698133e-05, 'humo_B-negative': 6.337537233031244e-05, 'tabaco_I-negative': 6.379992344009187e-05, 'arruina_O': 3.3215970238490664e-05, 'triston_O': 3.3215970238490664e-05, 'olvide_O': 3.3215970238490664e-05, 'fues_O': 3.3215970238490664e-05, 'esplendida_O': 3.3215970238490664e-05, 'tuvieron_O': 4.9823955357735996e-05, 'porqeu_O': 3.3215970238490664e-05, 'itios_O': 3.3215970238490664e-05, 'encontramos_O': 4.9823955357735996e-05, '64_O': 3.3215970238490664e-05, 'euros.El_O': 3.3215970238490664e-05, 'cuatro_O': 9.964791071547199e-05, 'guitarra.Resumiendo_O': 3.3215970238490664e-05, 'servicio_B-neutral': 0.0002560081922621524, 'pero_B-positive': 6.184865633794106e-05, 'conjunto_I-positive': 6.351020926613953e-05, 'es_I-positive': 6.351020926613953e-05, 'bueno_I-positive': 6.351020926613953e-05, 'muy_I-positive': 9.52653138992093e-05, 'recomendable_I-positive': 6.351020926613953e-05, 'jarra_O': 3.3215970238490664e-05, 'sangria_O': 3.3215970238490664e-05, '15.01_O': 3.3215970238490664e-05, 'vieira_B-negative': 6.337537233031244e-05, 'sopa_I-negative': 6.379992344009187e-05, '21_O': 6.643194047698133e-05, 'excepcionales_O': 4.9823955357735996e-05, 'Él_O': 3.3215970238490664e-05, 'rápidamente_O': 4.9823955357735996e-05, 'hoja_O': 4.9823955357735996e-05, 'quiso_O': 3.3215970238490664e-05, 'creer_O': 4.9823955357735996e-05, 'habian_O': 3.3215970238490664e-05, '20.30_O': 3.3215970238490664e-05, 'Da_O': 4.9823955357735996e-05, 'acabarse_O': 3.3215970238490664e-05, 'patatas_B-positive': 0.00012369731267588213, 'Nada_O': 9.964791071547199e-05, 'objetar_O': 4.9823955357735996e-05, 'quiero_O': 9.964791071547199e-05, 'dejar_O': 6.643194047698133e-05, 'constancia_O': 4.9823955357735996e-05, 'merecería_O': 3.3215970238490664e-05, 'hola_O': 4.9823955357735996e-05, 'malo_O': 0.00013286388095396266, 'pruductos_O': 4.9823955357735996e-05, 'denominacion_O': 3.3215970238490664e-05, 'origen_O': 3.3215970238490664e-05, 'prepara_O': 4.9823955357735996e-05, 'deleciosos_O': 3.3215970238490664e-05, 'cuidado_O': 4.9823955357735996e-05, 'caseros_O': 3.3215970238490664e-05, 'quesa_O': 3.3215970238490664e-05, 'pasiega_O': 3.3215970238490664e-05, 'pudin_O': 3.3215970238490664e-05, 'frutas_O': 3.3215970238490664e-05, 'bolas_O': 4.9823955357735996e-05, 'melon_O': 3.3215970238490664e-05, 'maceradas_O': 3.3215970238490664e-05, 'pedro_O': 3.3215970238490664e-05, 'xmenez_O': 3.3215970238490664e-05, 'tarta_O': 8.303992559622666e-05, 'imperial_O': 3.3215970238490664e-05, 'almendras_O': 4.9823955357735996e-05, 'óptimo_O': 6.643194047698133e-05, 'Cuando_B-negative': 6.337537233031244e-05, 'está_I-negative': 6.379992344009187e-05, 'lleno_I-negative': 6.379992344009187e-05, ',_I-negative': 0.00012759984688018373, 'configuración_I-negative': 6.379992344009187e-05, 'local_I-negative': 6.379992344009187e-05, 'amplifica_I-negative': 6.379992344009187e-05, 'el_I-negative': 9.56998851601378e-05, 'ruido_I-negative': 9.56998851601378e-05, 'afirmar_O': 3.3215970238490664e-05, 'estava_O': 3.3215970238490664e-05, 'Ciudad_O': 3.3215970238490664e-05, 'Condal_O': 3.3215970238490664e-05, 'establecimientos_O': 3.3215970238490664e-05, 'izquierda_O': 4.9823955357735996e-05, 'vergonzoso_O': 3.3215970238490664e-05, 'tarden_O': 3.3215970238490664e-05, '10minutos_O': 3.3215970238490664e-05, 'emplatar_O': 4.9823955357735996e-05, 'pinchos_O': 3.3215970238490664e-05, 'templandolos_O': 3.3215970238490664e-05, 'salamandra_O': 3.3215970238490664e-05, 'napando_O': 3.3215970238490664e-05, 'tosca_O': 3.3215970238490664e-05, 'simple_O': 3.3215970238490664e-05, 'cebollino_O': 3.3215970238490664e-05, 'acorde_O': 9.964791071547199e-05, 'oferta_O': 0.000149471866073208, 'Valentín_O': 3.3215970238490664e-05, 'cabrito_O': 6.643194047698133e-05, 'encargar_O': 3.3215970238490664e-05, 'distancia_O': 6.643194047698133e-05, 'digna_O': 4.9823955357735996e-05, 'ventanales_O': 3.3215970238490664e-05, 'Restaurantes_O': 4.9823955357735996e-05, 'correctos_O': 9.964791071547199e-05, 'vuelves_O': 3.3215970238490664e-05, 'pasan_O': 3.3215970238490664e-05, 'desapercibidos_O': 3.3215970238490664e-05, 'pésimos_O': 3.3215970238490664e-05, 'La_B-negative': 6.337537233031244e-05, 'Scala_I-negative': 6.379992344009187e-05, 'atendió_O': 3.3215970238490664e-05, 'alegre_O': 3.3215970238490664e-05, 'simpático_O': 3.3215970238490664e-05, 'entrecot_B-negative': 9.506305849546866e-05, 'seco_O': 3.3215970238490664e-05, 'brocheta_B-negative': 6.337537233031244e-05, 'taquitos_O': 3.3215970238490664e-05, 'pelo_O': 4.9823955357735996e-05, 'Bastante_O': 3.3215970238490664e-05, 'rato_O': 4.9823955357735996e-05, 'llamo_O': 3.3215970238490664e-05, 'señor_O': 6.643194047698133e-05, 'diciendo_O': 3.3215970238490664e-05, 'volviamos_O': 3.3215970238490664e-05, 'invitaria_O': 3.3215970238490664e-05, 'deducimos_O': 3.3215970238490664e-05, 'evitar_O': 3.3215970238490664e-05, 'hagamos_O': 3.3215970238490664e-05, 'publicidad_O': 3.3215970238490664e-05, 'venimos_O': 3.3215970238490664e-05, 'Amigo_O': 3.3215970238490664e-05, 'José_O': 4.9823955357735996e-05, 'comentar_O': 6.643194047698133e-05, 'Ud_O': 3.3215970238490664e-05, 'judías_B-negative': 6.337537233031244e-05, 'verdes_I-negative': 6.379992344009187e-05, 'pequeña_O': 6.643194047698133e-05, 'vecinos_O': 4.9823955357735996e-05, 'Pensabamos_O': 3.3215970238490664e-05, 'despiste_O': 3.3215970238490664e-05, 'empezaron_O': 3.3215970238490664e-05, 'cuchichear_O': 3.3215970238490664e-05, 'cocinero_B-negative': 6.337537233031244e-05, 'criticandonos_O': 3.3215970238490664e-05, 'riendose_O': 3.3215970238490664e-05, 'descarada_O': 3.3215970238490664e-05, 'mofa_O': 3.3215970238490664e-05, 'completas_O': 3.3215970238490664e-05, 'alucinar_O': 3.3215970238490664e-05, 'Eramos_O': 3.3215970238490664e-05, 'bién_O': 6.643194047698133e-05, 'vaya_O': 4.9823955357735996e-05, 'calidad_B-positive': 0.00015462164084485266, 'carta_B-neutral': 6.40020480655381e-05, 'renovada_O': 3.3215970238490664e-05, 'invita_O': 3.3215970238490664e-05, 'haberte_O': 3.3215970238490664e-05, 'dulce_B-positive': 6.184865633794106e-05, 'leche_I-positive': 6.351020926613953e-05, 'divino_O': 4.9823955357735996e-05, 'decoracion_O': 4.9823955357735996e-05, 'gustaria_O': 3.3215970238490664e-05, 'minimalista.Del_O': 3.3215970238490664e-05, 'mobiliario_B-negative': 6.337537233031244e-05, 'incomodas_O': 3.3215970238490664e-05, 'tamaño_O': 4.9823955357735996e-05, 'Misericordia_O': 3.3215970238490664e-05, 'acercamos_O': 3.3215970238490664e-05, 'enfadado_O': 3.3215970238490664e-05, 'Definitivamente_O': 4.9823955357735996e-05, 'faltan_O': 4.9823955357735996e-05, 'típicas_O': 3.3215970238490664e-05, 'preguntas_O': 4.9823955357735996e-05, 'servidos_O': 8.303992559622666e-05, 'calificaría_O': 3.3215970238490664e-05, 'desastroso_O': 3.3215970238490664e-05, '15h30_O': 3.3215970238490664e-05, 'Jamon_O': 3.3215970238490664e-05, 'Preparamos_O': 3.3215970238490664e-05, 'encontarr_O': 3.3215970238490664e-05, 'abierto_O': 4.9823955357735996e-05, 'traído_O': 3.3215970238490664e-05, 'según_O': 6.643194047698133e-05, 'ingrediente_O': 3.3215970238490664e-05, 'magnífica_O': 4.9823955357735996e-05, 'Mejor_O': 3.3215970238490664e-05, 'largo.Es_O': 3.3215970238490664e-05, 'hostelera_O': 3.3215970238490664e-05, '28_O': 4.9823955357735996e-05, 'presenta_O': 4.9823955357735996e-05, 'estilo_B-positive': 6.184865633794106e-05, 'minimalista_O': 3.3215970238490664e-05, 'snob_O': 3.3215970238490664e-05, 'presentar_O': 4.9823955357735996e-05, 'tapa_O': 4.9823955357735996e-05, 'para_I-positive': 6.351020926613953e-05, 'los_I-positive': 9.52653138992093e-05, 'niños_I-positive': 6.351020926613953e-05, 'rapido_O': 8.303992559622666e-05, 'pasaba_O': 3.3215970238490664e-05, 'golpeaban_O': 3.3215970238490664e-05, 'llauna_O': 3.3215970238490664e-05, 'diminuto_O': 3.3215970238490664e-05, 'secas_O': 3.3215970238490664e-05, 'lata_O': 3.3215970238490664e-05, 'patatas_I-negative': 6.379992344009187e-05, 'fritas_I-negative': 6.379992344009187e-05, 'fast_O': 3.3215970238490664e-05, 'food_O': 3.3215970238490664e-05, 'Manolo_O': 3.3215970238490664e-05, 'explicado_O': 3.3215970238490664e-05, 'exigir_O': 3.3215970238490664e-05, 'Les_O': 3.3215970238490664e-05, 'caliu_I-positive': 6.351020926613953e-05, 'alcachofas_B-positive': 9.277298450691158e-05, 'escalibadas_B-positive': 6.184865633794106e-05, 'genial.El_O': 3.3215970238490664e-05, 'trabajan_O': 3.3215970238490664e-05, 'última_O': 9.964791071547199e-05, 'pediomos_O': 3.3215970238490664e-05, 'infantil_O': 3.3215970238490664e-05, 'peque_O': 3.3215970238490664e-05, 'compuesto_O': 4.9823955357735996e-05, 'hambueguesa_O': 3.3215970238490664e-05, 'patatas_O': 9.964791071547199e-05, 'recomendados_O': 3.3215970238490664e-05, 'costillares_O': 3.3215970238490664e-05, 'refrescos_O': 3.3215970238490664e-05, 'cerveza_O': 4.9823955357735996e-05, 'opinion_O': 6.643194047698133e-05, 'sobra_O': 3.3215970238490664e-05, 'dejo_O': 4.9823955357735996e-05, 'tipica_O': 3.3215970238490664e-05, 'franquicia_O': 3.3215970238490664e-05, 'maxima_O': 3.3215970238490664e-05, 'comodidad_O': 3.3215970238490664e-05, 'estuvieras_O': 3.3215970238490664e-05, 'compañia_O': 3.3215970238490664e-05, 'LA_O': 8.303992559622666e-05, 'CARTA_O': 3.3215970238490664e-05, 'coincido_O': 6.643194047698133e-05, 'supera_O': 3.3215970238490664e-05, 'ambientación_B-positive': 6.184865633794106e-05, 'sería_O': 8.303992559622666e-05, 'Recomendadisimo_O': 3.3215970238490664e-05, 'Fué_O': 4.9823955357735996e-05, 'experiencia_B-positive': 0.00012369731267588213, 'Decir_O': 3.3215970238490664e-05, 'tomate_B-positive': 6.184865633794106e-05, 'besugo_B-positive': 6.184865633794106e-05, 'hubo_O': 6.643194047698133e-05, 'cuadraban_O': 3.3215970238490664e-05, 'Pedir_O': 3.3215970238490664e-05, 'piso_O': 3.3215970238490664e-05, 'vayamos_O': 3.3215970238490664e-05, 'destacable_O': 4.9823955357735996e-05, 'puntos_O': 6.643194047698133e-05, 'cocción_O': 8.303992559622666e-05, 'breves_O': 3.3215970238490664e-05, 'ajustados_O': 3.3215970238490664e-05, 'línea_O': 6.643194047698133e-05, 'Martín_O': 3.3215970238490664e-05, 'Berasategui_O': 3.3215970238490664e-05, 'Degusté_O': 3.3215970238490664e-05, 'ortigillas_O': 3.3215970238490664e-05, 'mar_O': 6.643194047698133e-05, 'besugo_O': 4.9823955357735996e-05, 'espalda_O': 3.3215970238490664e-05, 'supone_O': 3.3215970238490664e-05, 'casero_O': 4.9823955357735996e-05, 'prefabricada_O': 3.3215970238490664e-05, 'descongelada_O': 3.3215970238490664e-05, 'escogimos_O': 4.9823955357735996e-05, 'entremeses_B-neutral': 6.40020480655381e-05, 'foto_O': 4.9823955357735996e-05, 'dejaba_O': 6.643194047698133e-05, 'huevos_B-negative': 6.337537233031244e-05, 'estrellados_I-negative': 6.379992344009187e-05, 'fritos_O': 8.303992559622666e-05, 'miramiento_O': 3.3215970238490664e-05, 'escrito_O': 4.9823955357735996e-05, 'eMail_O': 3.3215970238490664e-05, 'falso_O': 3.3215970238490664e-05, 'obstante_O': 6.643194047698133e-05, 'Internet_O': 3.3215970238490664e-05, 'rastro_O': 3.3215970238490664e-05, 'derecho_O': 6.643194047698133e-05, 'medidas_O': 3.3215970238490664e-05, 'legales_O': 3.3215970238490664e-05, 'utilice_O': 3.3215970238490664e-05, 'foro_O': 4.9823955357735996e-05, 'don_O': 3.3215970238490664e-05, 'dolo_O': 3.3215970238490664e-05, 'perjudicar_O': 3.3215970238490664e-05, 'vieja_O': 3.3215970238490664e-05, 'original_O': 4.9823955357735996e-05, 'ciñéndome_O': 3.3215970238490664e-05, 'elaboración_O': 8.303992559622666e-05, 'Sencillamente_O': 3.3215970238490664e-05, 'buenísima_O': 6.643194047698133e-05, 'invitados_O': 3.3215970238490664e-05, 'impresionados_O': 3.3215970238490664e-05, 'espinacas_O': 3.3215970238490664e-05, 'rissotto_O': 3.3215970238490664e-05, 'pulpo_O': 4.9823955357735996e-05, 'langostinos_O': 9.964791071547199e-05, 'Servicio_B-negative': 0.00015843843082578111, 'Vamos_O': 4.9823955357735996e-05, 'preferidos_O': 3.3215970238490664e-05, 'proxima_O': 3.3215970238490664e-05, 'pizzas_O': 3.3215970238490664e-05, 'viendo_O': 4.9823955357735996e-05, 'Servico_B-positive': 6.184865633794106e-05, 'factura_B-negative': 9.506305849546866e-05, 'elevada_O': 3.3215970238490664e-05, 'frescura_O': 3.3215970238490664e-05, 'incomparable_O': 3.3215970238490664e-05, 'Sobresaliente_O': 4.9823955357735996e-05, 'adaptarse_O': 3.3215970238490664e-05, 'tiempos_O': 9.964791071547199e-05, 'menús_O': 4.9823955357735996e-05, 'cerrados_O': 4.9823955357735996e-05, 'asequible_O': 3.3215970238490664e-05, 'reconocido_O': 3.3215970238490664e-05, 'parecieron_O': 3.3215970238490664e-05, 'dignos_O': 4.9823955357735996e-05, 'Maur_O': 3.3215970238490664e-05, 'lugar_O': 0.00018268783631169867, 'marisco_B-positive': 9.277298450691158e-05, 'pescado_B-positive': 0.00012369731267588213, 'entorno_B-positive': 0.00015462164084485266, 'privilegiado_O': 3.3215970238490664e-05, 'amplio_O': 6.643194047698133e-05, 'ruidoso_O': 8.303992559622666e-05, 'Han_O': 4.9823955357735996e-05, 'mejicana_O': 3.3215970238490664e-05, 'merecía_O': 3.3215970238490664e-05, 'italiana_I-neutral': 6.406149903907751e-05, 'básica_O': 3.3215970238490664e-05, 'servida_O': 8.303992559622666e-05, 'estilo_O': 6.643194047698133e-05, 'sirvieran_O': 3.3215970238490664e-05, 'congelada_O': 3.3215970238490664e-05, 'cocinar_O': 3.3215970238490664e-05, 'corren_O': 4.9823955357735996e-05, 'descuentos_O': 3.3215970238490664e-05, 'renunciar_O': 3.3215970238490664e-05, 'talla_O': 3.3215970238490664e-05, 'acertadas_O': 3.3215970238490664e-05, 'caldoso_I-positive': 0.00012702041853227905, 'Cordero_B-negative': 6.337537233031244e-05, 'atendieron_O': 0.00011625589583471734, 'faena_O': 4.9823955357735996e-05, 'restaurantem_O': 3.3215970238490664e-05, 'diez_O': 4.9823955357735996e-05, 'salúd_O': 3.3215970238490664e-05, 'bajas_O': 4.9823955357735996e-05, 'continuación_O': 3.3215970238490664e-05, 'texturas_O': 3.3215970238490664e-05, 'melocotón_O': 3.3215970238490664e-05, 'escogida_O': 3.3215970238490664e-05, 'adecuados_O': 4.9823955357735996e-05, 'tripa_O': 3.3215970238490664e-05, 'Kupela_O': 3.3215970238490664e-05, 'analogías_O': 3.3215970238490664e-05, 'Diverxo_O': 3.3215970238490664e-05, 'Madrid_O': 4.9823955357735996e-05, 'calificado_O': 3.3215970238490664e-05, '59_O': 3.3215970238490664e-05, 'salvando_O': 3.3215970238490664e-05, 'Gamberro_O': 4.9823955357735996e-05, 'veríamos_O': 3.3215970238490664e-05, 'ojos_O': 3.3215970238490664e-05, 'Desvirtúa_O': 3.3215970238490664e-05, 'esmero_O': 4.9823955357735996e-05, 'Poseso_O': 3.3215970238490664e-05, 'siento_O': 6.643194047698133e-05, 'engañado_O': 4.9823955357735996e-05, 'reserve_O': 3.3215970238490664e-05, '66_O': 3.3215970238490664e-05, 'podía_O': 3.3215970238490664e-05, 'cenado_O': 9.964791071547199e-05, 'engañaron_O': 3.3215970238490664e-05, 'disfruté_O': 3.3215970238490664e-05, 'ningúna_O': 3.3215970238490664e-05, 'Estaban_O': 6.643194047698133e-05, 'malos_O': 8.303992559622666e-05, 'maravilloso_O': 3.3215970238490664e-05, 'monton_O': 3.3215970238490664e-05, 'pude_O': 3.3215970238490664e-05, 'volvia_O': 3.3215970238490664e-05, 'maravilló_O': 3.3215970238490664e-05, 'dió_O': 3.3215970238490664e-05, 'Gonzalo_O': 6.643194047698133e-05, 'escasito_O': 3.3215970238490664e-05, 'Qué_O': 8.303992559622666e-05, 'sepan_O': 4.9823955357735996e-05, 'diferenciar_O': 3.3215970238490664e-05, 'desprende_O': 3.3215970238490664e-05, 'espina_O': 3.3215970238490664e-05, 'calor_O': 6.643194047698133e-05, 'Sali_O': 3.3215970238490664e-05, 'fascinada_O': 3.3215970238490664e-05, 'mañana_O': 6.643194047698133e-05, 'realize_O': 3.3215970238490664e-05, 'ratito_O': 3.3215970238490664e-05, 'gimnasio_O': 3.3215970238490664e-05, 'aperitivo_B-positive': 9.277298450691158e-05, 'entrante_B-positive': 6.184865633794106e-05, 'presentados_O': 4.9823955357735996e-05, 'miel_O': 3.3215970238490664e-05, 'acerté_O': 3.3215970238490664e-05, 'Croquetas_B-negative': 6.337537233031244e-05, 'oleosas_O': 3.3215970238490664e-05, 'salmón_B-negative': 6.337537233031244e-05, 'marinado_I-negative': 6.379992344009187e-05, 'insulso_O': 3.3215970238490664e-05, 'láminas_O': 3.3215970238490664e-05, 'transparentes_O': 3.3215970238490664e-05, 'alcachofas_B-negative': 6.337537233031244e-05, 'almejas_I-negative': 6.379992344009187e-05, 'duras_O': 3.3215970238490664e-05, 'sosas_O': 3.3215970238490664e-05, 'vivero_O': 3.3215970238490664e-05, 'carpaccio_B-negative': 6.337537233031244e-05, 'rape_I-negative': 6.379992344009187e-05, 'testimonial_O': 3.3215970238490664e-05, 'felicitar_O': 3.3215970238490664e-05, 'quizás_O': 4.9823955357735996e-05, 'preferido_O': 3.3215970238490664e-05, 'celebraciones_O': 4.9823955357735996e-05, 'respetado_O': 3.3215970238490664e-05, 'capa_O': 3.3215970238490664e-05, 'comedor_B-positive': 0.00012369731267588213, 'coordinación_O': 3.3215970238490664e-05, 'generosa_O': 3.3215970238490664e-05, 'abundancia_O': 3.3215970238490664e-05, 'complementos_O': 3.3215970238490664e-05, 'gambas_O': 3.3215970238490664e-05, 'mejillones_O': 3.3215970238490664e-05, 'almejas_O': 3.3215970238490664e-05, 'increíblemente_O': 4.9823955357735996e-05, 'decirle_O': 3.3215970238490664e-05, 'llegada_O': 3.3215970238490664e-05, 'crisis_O': 3.3215970238490664e-05, 'razonbles_O': 3.3215970238490664e-05, 'cristalera_O': 3.3215970238490664e-05, 'reservé_O': 6.643194047698133e-05, 'comentaron_O': 3.3215970238490664e-05, 'Seguimos_O': 4.9823955357735996e-05, 'chipirones_B-neutral': 6.40020480655381e-05, 'fritos_I-neutral': 6.406149903907751e-05, 'pasta_B-neutral': 6.40020480655381e-05, 'rellena_I-neutral': 6.406149903907751e-05, 'huevos_B-neutral': 6.40020480655381e-05, 'Charly_I-neutral': 6.406149903907751e-05, 'carrillera_B-neutral': 6.40020480655381e-05, 'ibérica_I-neutral': 6.406149903907751e-05, 'ESCASO_O': 4.9823955357735996e-05, 'comedora_O': 3.3215970238490664e-05, 'anunciado_O': 4.9823955357735996e-05, 'teléfono_O': 6.643194047698133e-05, 'Poma_B-positive': 6.184865633794106e-05, 'consigue_O': 3.3215970238490664e-05, 'probarlo_O': 4.9823955357735996e-05, 'felicitamos_O': 3.3215970238490664e-05, 'cocinero_B-positive': 9.277298450691158e-05, 'llamaba_O': 3.3215970238490664e-05, 'César_O': 3.3215970238490664e-05, 'miedo_O': 4.9823955357735996e-05, 'navajas_B-negative': 6.337537233031244e-05, 'musclos_B-negative': 6.337537233031244e-05, 'parecian_O': 3.3215970238490664e-05, 'tortillitas_O': 4.9823955357735996e-05, 'jamon_B-negative': 6.337537233031244e-05, '"_I-negative': 9.56998851601378e-05, 'iberico_I-negative': 6.379992344009187e-05, 'calificacion_O': 3.3215970238490664e-05, 'justito_O': 4.9823955357735996e-05, 'rematar_O': 3.3215970238490664e-05, 'lavabos_B-negative': 9.506305849546866e-05, 'sucios_O': 4.9823955357735996e-05, 'renfe_O': 3.3215970238490664e-05, 'música_B-positive': 6.184865633794106e-05, 'Bares_O': 3.3215970238490664e-05, 'música_O': 9.964791071547199e-05, 'carnes_B-neutral': 6.40020480655381e-05, 'calçotada_I-positive': 6.351020926613953e-05, 'trato_I-positive': 6.351020926613953e-05, 'comidas_O': 4.9823955357735996e-05, 'VOLVERÉ_O': 3.3215970238490664e-05, 'FELIZ_O': 3.3215970238490664e-05, 'AÑO_O': 3.3215970238490664e-05, 'Tomo_O': 3.3215970238490664e-05, 'plenamente_O': 3.3215970238490664e-05, 'SIn_O': 3.3215970238490664e-05, 'yahe_O': 3.3215970238490664e-05, 'número_O': 6.643194047698133e-05, 'disponible_O': 4.9823955357735996e-05, 'enfriase_O': 3.3215970238490664e-05, 'oh_O': 3.3215970238490664e-05, 'anegada_O': 3.3215970238490664e-05, 'tinta_O': 3.3215970238490664e-05, 'tempura_B-negative': 6.337537233031244e-05, 'treméndamente_O': 3.3215970238490664e-05, 'vulgar_O': 6.643194047698133e-05, 'olvidaba_O': 3.3215970238490664e-05, 'chuletón_B-positive': 6.184865633794106e-05, 'buey_I-positive': 9.52653138992093e-05, 'Navarra_O': 3.3215970238490664e-05, 'puedas_O': 6.643194047698133e-05, 'esperas_O': 4.9823955357735996e-05, 'calidad.No_O': 3.3215970238490664e-05, 'dejéis_O': 3.3215970238490664e-05, 'visitarlo_O': 3.3215970238490664e-05, 'agradeceréis_O': 3.3215970238490664e-05, '40_O': 4.9823955357735996e-05, 'hosteleria_O': 3.3215970238490664e-05, 'principalmente_O': 3.3215970238490664e-05, 'su_I-positive': 6.351020926613953e-05, 'propietaria_I-positive': 6.351020926613953e-05, 'recibirá_O': 3.3215970238490664e-05, 'brazos_O': 3.3215970238490664e-05, 'abiertos_O': 3.3215970238490664e-05, 'amabilidad_O': 6.643194047698133e-05, 'interés_O': 3.3215970238490664e-05, 'salga_O': 4.9823955357735996e-05, 'Postre_B-negative': 6.337537233031244e-05, 'gustosos_O': 3.3215970238490664e-05, 'elavorados_O': 3.3215970238490664e-05, 'saborea_O': 3.3215970238490664e-05, 'Cubiertos_B-negative': 6.337537233031244e-05, 'puestos_O': 3.3215970238490664e-05, 'mínima_O': 3.3215970238490664e-05, 'delicadeza_O': 3.3215970238490664e-05, 'veras_O': 3.3215970238490664e-05, 'cuente_O': 3.3215970238490664e-05, 'gaditano_O': 3.3215970238490664e-05, 'médula_O': 3.3215970238490664e-05, 'atendido_O': 4.9823955357735996e-05, 'próxima_O': 6.643194047698133e-05, 'pediré_O': 3.3215970238490664e-05, 'saldrá_O': 3.3215970238490664e-05, 'mejorará_O': 3.3215970238490664e-05, 'permitian_O': 3.3215970238490664e-05, 'fumar_O': 4.9823955357735996e-05, 'inconcebible_O': 3.3215970238490664e-05, 'decian_O': 3.3215970238490664e-05, 'comunicada_O': 3.3215970238490664e-05, 'salon_O': 3.3215970238490664e-05, 'toldos_O': 3.3215970238490664e-05, 'bajados_O': 3.3215970238490664e-05, 'entrada_I-negative': 6.379992344009187e-05, 'automatica_O': 3.3215970238490664e-05, 'recepción_O': 3.3215970238490664e-05, 'sentir_O': 6.643194047698133e-05, 'comoda_O': 3.3215970238490664e-05, 'Llegaba_O': 3.3215970238490664e-05, 'apetecía_O': 3.3215970238490664e-05, 'irse_O': 3.3215970238490664e-05, 'café_B-negative': 6.337537233031244e-05, 'Burger_O': 3.3215970238490664e-05, 'King_O': 3.3215970238490664e-05, 'platitos_B-negative': 6.337537233031244e-05, 'para_I-negative': 6.379992344009187e-05, 'pan_I-negative': 9.56998851601378e-05, 'resulta_O': 3.3215970238490664e-05, 'recibes_O': 3.3215970238490664e-05, 'sorbete_O': 3.3215970238490664e-05, 'llenísimo_O': 3.3215970238490664e-05, 'Interior_B-positive': 6.184865633794106e-05, 'k_O': 3.3215970238490664e-05, 'quereis_O': 3.3215970238490664e-05, 'Fenomenal_O': 3.3215970238490664e-05, 'esperais_O': 3.3215970238490664e-05, 'novio_O': 3.3215970238490664e-05, 'cumple_O': 3.3215970238490664e-05, 'samilos_O': 3.3215970238490664e-05, 'lindo_O': 3.3215970238490664e-05, 'podremos_O': 3.3215970238490664e-05, 'seguro.En_O': 3.3215970238490664e-05, 'personas.Unicamente_O': 3.3215970238490664e-05, 'seguramente_O': 3.3215970238490664e-05, 'brasas_B-negative': 6.337537233031244e-05, 'debian_O': 3.3215970238490664e-05, 'vaca_I-negative': 6.379992344009187e-05, 'ternasco_B-negative': 6.337537233031244e-05, 'presencia_O': 4.9823955357735996e-05, 'golpe_O': 3.3215970238490664e-05, 'doraditos_O': 3.3215970238490664e-05, 'amplitud_B-positive': 6.184865633794106e-05, 'adecuada_O': 3.3215970238490664e-05, 'complicaciones_O': 3.3215970238490664e-05, 'explicaciones_O': 8.303992559622666e-05, 'barbaridad_O': 3.3215970238490664e-05, '170_O': 3.3215970238490664e-05, 'tomaron_O': 3.3215970238490664e-05, 'cafes_O': 4.9823955357735996e-05, 'respeto_O': 6.643194047698133e-05, 'aprendido_O': 3.3215970238490664e-05, 'arte_O': 3.3215970238490664e-05, 'pueda_O': 4.9823955357735996e-05, 'conocerla_O': 3.3215970238490664e-05, 'matambre_B-positive': 6.184865633794106e-05, 'descripción_O': 8.303992559622666e-05, 'leído_O': 3.3215970238490664e-05, 'opiniones_O': 4.9823955357735996e-05, 'negativas_O': 3.3215970238490664e-05, 'positivas_O': 3.3215970238490664e-05, 'comentas_O': 3.3215970238490664e-05, 'Hablé_O': 3.3215970238490664e-05, 'gentílmente_O': 3.3215970238490664e-05, 'explicaron_O': 3.3215970238490664e-05, 'Relación_O': 4.9823955357735996e-05, 'estrellas_O': 4.9823955357735996e-05, 'recepción_B-positive': 6.184865633794106e-05, 'incorporación_O': 3.3215970238490664e-05, 'Alcachofas_B-negative': 6.337537233031244e-05, 'brasa_I-negative': 6.379992344009187e-05, 'aparece_O': 4.9823955357735996e-05, 'reestaurante_B-positive': 6.184865633794106e-05, 'volvieron_O': 3.3215970238490664e-05, 'degustacion_O': 3.3215970238490664e-05, 'diferentes_O': 4.9823955357735996e-05, 'Gran_B-positive': 6.184865633794106e-05, 'opcion_I-positive': 6.351020926613953e-05, 'rest_I-positive': 6.351020926613953e-05, 'pidais_O': 3.3215970238490664e-05, 'rape_B-negative': 9.506305849546866e-05, 'cazuela_O': 3.3215970238490664e-05, 'catalogan_O': 3.3215970238490664e-05, 'Hacía_O': 4.9823955357735996e-05, 'encontraba_O': 3.3215970238490664e-05, 'fallara_O': 3.3215970238490664e-05, 'mayoria_O': 3.3215970238490664e-05, 'acristalado_O': 3.3215970238490664e-05, 'podian_O': 4.9823955357735996e-05, 'asar_O': 3.3215970238490664e-05, 'pollos_O': 3.3215970238490664e-05, 'comedor_B-negative': 6.337537233031244e-05, 'necesidad_O': 4.9823955357735996e-05, 'alargarme_O': 3.3215970238490664e-05, 'Variedad_O': 4.9823955357735996e-05, 'Homogeneidad_B-positive': 6.184865633794106e-05, 'imbatible_O': 3.3215970238490664e-05, 'hacia_O': 6.643194047698133e-05, 'bordes_O': 3.3215970238490664e-05, 'malas_O': 4.9823955357735996e-05, 'disposición_B-positive': 6.184865633794106e-05, 'simpaticos_O': 3.3215970238490664e-05, 'profesionales_O': 6.643194047698133e-05, 'restaurant_B-positive': 6.184865633794106e-05, 'piña_B-negative': 6.337537233031244e-05, 'ajo_O': 4.9823955357735996e-05, 'verguenza_O': 3.3215970238490664e-05, 'Éramos_O': 4.9823955357735996e-05, '11_O': 6.643194047698133e-05, 'prepararon_O': 3.3215970238490664e-05, 'apartadita_O': 3.3215970238490664e-05, 'tarto_B-positive': 6.184865633794106e-05, 'EncantadoraRepetiremos_O': 3.3215970238490664e-05, 'indicar_O': 3.3215970238490664e-05, 'restaurnate_O': 3.3215970238490664e-05, '70_O': 6.643194047698133e-05, 'deprisa_O': 6.643194047698133e-05, 'servicial_O': 3.3215970238490664e-05, 'aprobado_O': 3.3215970238490664e-05, 'minucioso_O': 3.3215970238490664e-05, 'trabajador_O': 3.3215970238490664e-05, 'incansable_O': 3.3215970238490664e-05, 'director_O': 3.3215970238490664e-05, 'orquesta_O': 3.3215970238490664e-05, 'don_B-positive': 6.184865633794106e-05, 'Ramón_I-positive': 6.351020926613953e-05, 'Ferreira_I-positive': 6.351020926613953e-05, 'Aperitivos_O': 3.3215970238490664e-05, 'Tartar_O': 3.3215970238490664e-05, 'mono_O': 4.9823955357735996e-05, 'menú_B-negative': 0.00019012611699093733, 'probé_O': 4.9823955357735996e-05, 'suspenso_O': 3.3215970238490664e-05, 'favoritos_O': 6.643194047698133e-05, 'mencionada_O': 3.3215970238490664e-05, 'minimalistas_O': 3.3215970238490664e-05, 'tratando_O': 3.3215970238490664e-05, 'dejes_O': 3.3215970238490664e-05, 'ocupar_O': 3.3215970238490664e-05, 'Alberto_O': 3.3215970238490664e-05, 'Frisch_O': 3.3215970238490664e-05, 'uso_O': 3.3215970238490664e-05, 'cocción_B-positive': 6.184865633794106e-05, 'baja_I-positive': 6.351020926613953e-05, 'temperatura_I-positive': 6.351020926613953e-05, 'espetaculares_O': 3.3215970238490664e-05, 'trabajadas_O': 3.3215970238490664e-05, 'Tapas_B-positive': 6.184865633794106e-05, 'Bebidas_B-positive': 6.184865633794106e-05, 'Extraurinadio_O': 3.3215970238490664e-05, 'comi_O': 3.3215970238490664e-05, 'atendindo_O': 3.3215970238490664e-05, 'servito_O': 3.3215970238490664e-05, 'vuelto_O': 3.3215970238490664e-05, 'junio_O': 3.3215970238490664e-05, '2013_O': 3.3215970238490664e-05, 'conclusión_O': 3.3215970238490664e-05, 'aciertas_O': 4.9823955357735996e-05, 'esperado_O': 4.9823955357735996e-05, 'Tengo_O': 3.3215970238490664e-05, 'fundado_O': 3.3215970238490664e-05, 'encontró_O': 3.3215970238490664e-05, 'pimienta_I-positive': 6.351020926613953e-05, 'privados_O': 4.9823955357735996e-05, 'Tres_O': 3.3215970238490664e-05, 'rúcula_O': 3.3215970238490664e-05, 'carnes_B-negative': 6.337537233031244e-05, 'grado_O': 3.3215970238490664e-05, 'comernsal_O': 3.3215970238490664e-05, 'cene_O': 3.3215970238490664e-05, 'colette_O': 3.3215970238490664e-05, 'retaurante_B-positive': 6.184865633794106e-05, 'Cenados_O': 3.3215970238490664e-05, 'decepcionó_O': 4.9823955357735996e-05, 'risotto_B-positive': 9.277298450691158e-05, 'acceptable_O': 3.3215970238490664e-05, 'pizza_B-negative': 6.337537233031244e-05, 'iluminación_B-positive': 6.184865633794106e-05, 'pactado_O': 3.3215970238490664e-05, 'antelación_O': 3.3215970238490664e-05, 'correspondido_O': 3.3215970238490664e-05, 'error_O': 6.643194047698133e-05, 'Tomamos_O': 4.9823955357735996e-05, 'broma_O': 3.3215970238490664e-05, 'salió_O': 6.643194047698133e-05, 'Euros_O': 4.9823955357735996e-05, 'Totalmente_O': 8.303992559622666e-05, 'Ayerbe_O': 3.3215970238490664e-05, 'postres_B-neutral': 6.40020480655381e-05, 'simples_O': 3.3215970238490664e-05, 'cafes_B-negative': 6.337537233031244e-05, 'pettite_O': 3.3215970238490664e-05, 'fours_O': 3.3215970238490664e-05, '175_O': 3.3215970238490664e-05, 'cobrarnos_O': 4.9823955357735996e-05, 'leen_O': 3.3215970238490664e-05, 'sugerencias_B-negative': 6.337537233031244e-05, 'Desde_O': 9.964791071547199e-05, 'afortunadamente_O': 3.3215970238490664e-05, 'cobrado_O': 6.643194047698133e-05, 'Jefe_B-positive': 6.184865633794106e-05, 'Sala_I-positive': 6.351020926613953e-05, 'recuerda_O': 4.9823955357735996e-05, 'saluda_O': 3.3215970238490664e-05, 'habituales_O': 3.3215970238490664e-05, 'meses_O': 6.643194047698133e-05, 'visitaron_O': 3.3215970238490664e-05, 'esposa_O': 3.3215970238490664e-05, 'ciertamente_O': 3.3215970238490664e-05, 'revisar_O': 3.3215970238490664e-05, 'descubrir_O': 3.3215970238490664e-05, 'llevó_O': 3.3215970238490664e-05, 'disgusto_O': 3.3215970238490664e-05, 'considerábamos_O': 3.3215970238490664e-05, 'refugios_O': 3.3215970238490664e-05, 'pirenaicos_O': 3.3215970238490664e-05, 'vivimos_O': 3.3215970238490664e-05, 'regalaron_O': 3.3215970238490664e-05, 'cuchara_O': 3.3215970238490664e-05, 'madera_O': 3.3215970238490664e-05, 'Febrero_O': 3.3215970238490664e-05, 'califación_O': 3.3215970238490664e-05, 'dejeis_O': 3.3215970238490664e-05, 'comportaros_O': 3.3215970238490664e-05, 'penas_O': 3.3215970238490664e-05, 'tenías_O': 3.3215970238490664e-05, 'Posiblemente_O': 3.3215970238490664e-05, 'Finalmente_O': 3.3215970238490664e-05, 'gustaría_O': 4.9823955357735996e-05, 'agradecerle_O': 3.3215970238490664e-05, 'enriquece_O': 3.3215970238490664e-05, 'suya_O': 3.3215970238490664e-05, 'ofrecido_O': 8.303992559622666e-05, 'jamás_O': 3.3215970238490664e-05, 'jmarcoj_O': 3.3215970238490664e-05, 'dedir_O': 3.3215970238490664e-05, 'presa_O': 3.3215970238490664e-05, 'ibérica_O': 3.3215970238490664e-05, 'Can_O': 3.3215970238490664e-05, 'Roca_O': 3.3215970238490664e-05, 'espectacular_O': 0.00013286388095396266, 'L_O': 3.3215970238490664e-05, 'Olivé_O': 3.3215970238490664e-05, 'Muntaner_O': 4.9823955357735996e-05, 'acondicionado_O': 3.3215970238490664e-05, 'desagradable_O': 3.3215970238490664e-05, 'fernando_B-negative': 6.337537233031244e-05, 'nieto_I-negative': 6.379992344009187e-05, 'tratar_O': 3.3215970238490664e-05, 'alusion_O': 3.3215970238490664e-05, 'utilizando_O': 4.9823955357735996e-05, 'criticas_O': 3.3215970238490664e-05, 'cazón_O': 6.643194047698133e-05, 'adobo_O': 3.3215970238490664e-05, 'chocos_O': 6.643194047698133e-05, 'tortilla_O': 3.3215970238490664e-05, 'camarones_O': 3.3215970238490664e-05, 'boquerones_O': 3.3215970238490664e-05, 'cazon_O': 3.3215970238490664e-05, 'boqueron_O': 3.3215970238490664e-05, 'juegan_O': 3.3215970238490664e-05, 'hablan_O': 3.3215970238490664e-05, '38_O': 4.9823955357735996e-05, 'bodega_O': 3.3215970238490664e-05, 'incluida.Los_O': 3.3215970238490664e-05, 'platos_B-neutral': 6.40020480655381e-05, 'estuvieron_O': 3.3215970238490664e-05, 'jamón_O': 3.3215970238490664e-05, 'pimientos_O': 3.3215970238490664e-05, 'padrón_O': 3.3215970238490664e-05, 'confitadas_O': 4.9823955357735996e-05, 'meloso_O': 3.3215970238490664e-05, 'Decepcionante_O': 4.9823955357735996e-05, 'experiencia_B-negative': 0.00012675074466062487, 'vinos_I-negative': 9.56998851601378e-05, 'internacional_O': 3.3215970238490664e-05, 'disponen_O': 3.3215970238490664e-05, 'hermoso_O': 3.3215970238490664e-05, 'vista_B-positive': 6.184865633794106e-05, 'encarado_O': 3.3215970238490664e-05, 'distraido_O': 3.3215970238490664e-05, 'seran_O': 3.3215970238490664e-05, 'Empiezo_O': 3.3215970238490664e-05, 'trabajamos_O': 3.3215970238490664e-05, 'rape_O': 3.3215970238490664e-05, 'gamba_O': 3.3215970238490664e-05, 'sur_O': 3.3215970238490664e-05, 'Bahía_O': 3.3215970238490664e-05, 'Langostinos_O': 3.3215970238490664e-05, 'Sanlucar_O': 3.3215970238490664e-05, 'pescaitos_O': 3.3215970238490664e-05, 'mandan_O': 3.3215970238490664e-05, 'Cádiz_O': 6.643194047698133e-05, 'marinera_O': 3.3215970238490664e-05, 'destacando_O': 4.9823955357735996e-05, 'Estrella_O': 3.3215970238490664e-05, 'Michelín_O': 4.9823955357735996e-05, 'paciencia_O': 3.3215970238490664e-05, 'presentes_O': 3.3215970238490664e-05, '21/01_O': 3.3215970238490664e-05, '2014Fuimos_O': 3.3215970238490664e-05, 'Pareja_O': 4.9823955357735996e-05, 'CenarEl_O': 3.3215970238490664e-05, 'CorrectoLas_O': 3.3215970238490664e-05, 'Tortillitas_B-positive': 6.184865633794106e-05, 'Tomate_B-positive': 6.184865633794106e-05, 'Choco_B-positive': 6.184865633794106e-05, 'ExcelentesDe_O': 3.3215970238490664e-05, 'Segundo_O': 4.9823955357735996e-05, 'Rodaballo_O': 4.9823955357735996e-05, 'Presa_O': 3.3215970238490664e-05, 'Jamón_B-positive': 6.184865633794106e-05, 'J_O': 3.3215970238490664e-05, 'Bueno_O': 6.643194047698133e-05, 'PuntoA_O': 3.3215970238490664e-05, 'Limpiaron_O': 3.3215970238490664e-05, 'Notaba_O': 3.3215970238490664e-05, 'Estaba_O': 6.643194047698133e-05, 'Acostumbrado_O': 3.3215970238490664e-05, 'Hacerlo_O': 3.3215970238490664e-05, 'Rápida_O': 3.3215970238490664e-05, 'Profesional_O': 3.3215970238490664e-05, 'Camarera_B-positive': 6.184865633794106e-05, 'LimpioA_O': 3.3215970238490664e-05, 'Nosotros_O': 8.303992559622666e-05, 'Cobraron_O': 3.3215970238490664e-05, 'Pan_O': 3.3215970238490664e-05, 'Tal_O': 6.643194047698133e-05, 'Vez_O': 3.3215970238490664e-05, 'Estabamos_O': 3.3215970238490664e-05, 'Zona_O': 3.3215970238490664e-05, 'Bar_O': 3.3215970238490664e-05, 'Primer_O': 3.3215970238490664e-05, 'Plato_O': 3.3215970238490664e-05, 'Tomate_O': 3.3215970238490664e-05, 'Habiamos_O': 3.3215970238490664e-05, 'Comido_O': 3.3215970238490664e-05, 'Cesta_O': 3.3215970238490664e-05, 'Entera_O': 3.3215970238490664e-05, 'UntandoDe_O': 3.3215970238490664e-05, 'Postre_O': 4.9823955357735996e-05, 'Tocino_B-positive': 6.184865633794106e-05, 'De_I-positive': 9.52653138992093e-05, 'Cielo_I-positive': 6.351020926613953e-05, 'Francamente_O': 3.3215970238490664e-05, 'Probar_O': 3.3215970238490664e-05, 'Muchos_O': 3.3215970238490664e-05, 'Estoy_O': 3.3215970238490664e-05, 'Casi_O': 3.3215970238490664e-05, 'Convencido_O': 3.3215970238490664e-05, 'CaseroUn_O': 3.3215970238490664e-05, 'Detalle_O': 3.3215970238490664e-05, 'Café_O': 3.3215970238490664e-05, 'Pusieron_O': 3.3215970238490664e-05, 'Degustación_O': 4.9823955357735996e-05, 'Chupitos_O': 3.3215970238490664e-05, 'Gratis_O': 3.3215970238490664e-05, 'El_B-positive': 9.277298450691158e-05, 'Chupito_I-positive': 6.351020926613953e-05, 'Pedro_I-positive': 6.351020926613953e-05, 'Ximenez_I-positive': 6.351020926613953e-05, 'Buenísimo_O': 3.3215970238490664e-05, 'Precio_I-positive': 6.351020926613953e-05, '48,00_I-positive': 6.351020926613953e-05, '€_I-positive': 6.351020926613953e-05, 'Los_I-positive': 6.351020926613953e-05, 'Dos_I-positive': 6.351020926613953e-05, 'No_I-positive': 6.351020926613953e-05, 'Esta_I-positive': 6.351020926613953e-05, 'Mal_I-positive': 6.351020926613953e-05, 'Para_I-positive': 6.351020926613953e-05, 'Lo_I-positive': 6.351020926613953e-05, 'Que_I-positive': 6.351020926613953e-05, 'Comimos_I-positive': 6.351020926613953e-05, 'Probamos_O': 4.9823955357735996e-05, 'Cosas_O': 3.3215970238490664e-05, 'Vas_O': 3.3215970238490664e-05, 'Andalucía_O': 3.3215970238490664e-05, 'Difícil_O': 3.3215970238490664e-05, 'ComasEn_O': 3.3215970238490664e-05, 'Conclusión_O': 3.3215970238490664e-05, 'P_O': 4.9823955357735996e-05, 'fijo_O': 3.3215970238490664e-05, 'cocina.el_O': 3.3215970238490664e-05, 'separación_O': 3.3215970238490664e-05, 'genero_B-positive': 6.184865633794106e-05, 'sorprendente_O': 6.643194047698133e-05, 'puedes_O': 6.643194047698133e-05, 'encontrarlo_O': 3.3215970238490664e-05, 'sorprendía_O': 3.3215970238490664e-05, 'Gran_O': 6.643194047698133e-05, 'numerosos_O': 3.3215970238490664e-05, 'detalles_B-positive': 6.184865633794106e-05, 'calidad.El_O': 3.3215970238490664e-05, 'tuvo_O': 3.3215970238490664e-05, 'segunda_O': 3.3215970238490664e-05, 'quedaba_O': 3.3215970238490664e-05, 'panes_O': 3.3215970238490664e-05, 'pizza_O': 4.9823955357735996e-05, 'oliva_O': 6.643194047698133e-05, 'verde_O': 3.3215970238490664e-05, 'negra_O': 4.9823955357735996e-05, 'cuenquito_O': 3.3215970238490664e-05, 'mozzarella_O': 3.3215970238490664e-05, 'albahaca_O': 3.3215970238490664e-05, 'relacion_B-positive': 6.184865633794106e-05, 'calidad_I-positive': 9.52653138992093e-05, 'precio_I-positive': 9.52653138992093e-05, 'buena.Buena_O': 3.3215970238490664e-05, 'elwccion_O': 3.3215970238490664e-05, 'reducto_B-positive': 6.184865633794106e-05, 'llantar_O': 3.3215970238490664e-05, 'rematado_O': 3.3215970238490664e-05, 'cigarro_O': 3.3215970238490664e-05, 'habano_O': 3.3215970238490664e-05, 'Intentaré_O': 3.3215970238490664e-05, 'mejorando_O': 3.3215970238490664e-05, 'extranjeros_O': 3.3215970238490664e-05, 'parecia_O': 3.3215970238490664e-05, 'introducir_O': 3.3215970238490664e-05, 'tipico_O': 3.3215970238490664e-05, 'Rossello_O': 3.3215970238490664e-05, 'fresquísimas_O': 3.3215970238490664e-05, 'entrante_B-negative': 6.337537233031244e-05, 'Arroz_B-positive': 6.184865633794106e-05, 'extrañada_O': 3.3215970238490664e-05, 'abajo_O': 4.9823955357735996e-05, 'vienen_O': 3.3215970238490664e-05, 'llevamos_O': 6.643194047698133e-05, 'querido_O': 3.3215970238490664e-05, 'integra_O': 3.3215970238490664e-05, 'siguen_O': 3.3215970238490664e-05, 'futuro_O': 3.3215970238490664e-05, 'Ustedes_O': 3.3215970238490664e-05, 'valoren_O': 3.3215970238490664e-05, 'Tras_O': 3.3215970238490664e-05, 'interesarme_O': 3.3215970238490664e-05, 'extra-coste_O': 3.3215970238490664e-05, 'consideran_O': 3.3215970238490664e-05, 'asado_B-positive': 6.184865633794106e-05, 'citar_O': 3.3215970238490664e-05, 'coeficientes_O': 3.3215970238490664e-05, 'criterios_O': 3.3215970238490664e-05, 'valorar_O': 8.303992559622666e-05, 'espero_O': 6.643194047698133e-05, 'hayas_O': 4.9823955357735996e-05, 'teneido_O': 3.3215970238490664e-05, 'decencia_O': 3.3215970238490664e-05, 'aplicarle_O': 3.3215970238490664e-05, 'resultado_O': 8.303992559622666e-05, 'gastronómico_O': 3.3215970238490664e-05, 'coeficiente_O': 3.3215970238490664e-05, 'JOse_O': 3.3215970238490664e-05, 'varías_O': 3.3215970238490664e-05, 'Pelentos_O': 3.3215970238490664e-05, 'opino_O': 3.3215970238490664e-05, 'tercera_O': 3.3215970238490664e-05, 'faltaba_O': 4.9823955357735996e-05, 'sinceramente_O': 9.964791071547199e-05, 'argentinos_O': 4.9823955357735996e-05, 'inviertiendo_O': 3.3215970238490664e-05, 'exitosamente_O': 3.3215970238490664e-05, 'complicado_O': 3.3215970238490664e-05, 'imagen_B-positive': 6.184865633794106e-05, 'web_I-positive': 6.351020926613953e-05, 'pase_O': 3.3215970238490664e-05, 'básico_O': 3.3215970238490664e-05, 'díscolos_O': 3.3215970238490664e-05, 'rebajar_O': 3.3215970238490664e-05, 'notable_O': 4.9823955357735996e-05, 'fuí_O': 3.3215970238490664e-05, 'Resumen_O': 3.3215970238490664e-05, 'decía_O': 6.643194047698133e-05, 'título_O': 3.3215970238490664e-05, 'sabes_O': 3.3215970238490664e-05, 'juega_O': 3.3215970238490664e-05, 'idea_O': 4.9823955357735996e-05, 'mecanizado_O': 3.3215970238490664e-05, 'exceso_O': 4.9823955357735996e-05, 'resienten_O': 3.3215970238490664e-05, 'Precisamente_O': 3.3215970238490664e-05, 'artículo_O': 3.3215970238490664e-05, 'intentado_O': 4.9823955357735996e-05, 'incongruente_O': 3.3215970238490664e-05, 'negativa_O': 3.3215970238490664e-05, 'logrado_O': 3.3215970238490664e-05, 'sencilla_O': 3.3215970238490664e-05, 'lineal_O': 3.3215970238490664e-05, 'liar_O': 3.3215970238490664e-05, 'geenral_O': 3.3215970238490664e-05, 'cuidasen_O': 3.3215970238490664e-05, 'sufrimos_O': 3.3215970238490664e-05, 'gritar_O': 4.9823955357735996e-05, 'Pesquera_O': 3.3215970238490664e-05, 'crianza_O': 3.3215970238490664e-05, 'posos_O': 3.3215970238490664e-05, 'pica_O': 3.3215970238490664e-05, 'exceccional_O': 3.3215970238490664e-05, 'impresionantes_O': 3.3215970238490664e-05, 'recien_O': 3.3215970238490664e-05, 'traido_O': 3.3215970238490664e-05, 'SEBASTIAN_O': 3.3215970238490664e-05, 'comes_O': 4.9823955357735996e-05, 'sidreria.volveria_O': 3.3215970238490664e-05, 'pudiese_O': 3.3215970238490664e-05, 'descubres_O': 3.3215970238490664e-05, 'formar_O': 3.3215970238490664e-05, 'decoración_O': 4.9823955357735996e-05, 'pica_B-positive': 6.184865633794106e-05, 'típicos_O': 3.3215970238490664e-05, 'catalanes_O': 4.9823955357735996e-05, 'Actuaban_O': 3.3215970238490664e-05, 'las_B-positive': 6.184865633794106e-05, 'Posesas_I-positive': 6.351020926613953e-05, 'show_B-positive': 6.184865633794106e-05, 'entretenido_O': 3.3215970238490664e-05, 'haciendose_O': 3.3215970238490664e-05, 'simpáticas_O': 3.3215970238490664e-05, 'quisiera_O': 3.3215970238490664e-05, 'propuesta_O': 3.3215970238490664e-05, 'mejora_O': 3.3215970238490664e-05, 'estresados_O': 3.3215970238490664e-05, 'atendernos_O': 3.3215970238490664e-05, 'pusieran_O': 4.9823955357735996e-05, 'sistema_B-negative': 6.337537233031244e-05, 'logístico_I-negative': 6.379992344009187e-05, 'cocina-sala_I-negative': 6.379992344009187e-05, 'diseñado_O': 3.3215970238490664e-05, 'estaríamos_O': 3.3215970238490664e-05, 'prontitud_O': 3.3215970238490664e-05, 'coincidimos_O': 3.3215970238490664e-05, 'dinamismo_O': 3.3215970238490664e-05, 'simpatia_O': 3.3215970238490664e-05, 'caldoso_O': 4.9823955357735996e-05, 'chipirones_B-negative': 6.337537233031244e-05, 'podían_O': 3.3215970238490664e-05, 'amoniaco_O': 3.3215970238490664e-05, 'ensalada_B-neutral': 6.40020480655381e-05, 'sorbete_B-positive': 9.277298450691158e-05, 'mandarina_I-positive': 6.351020926613953e-05, '30euros_O': 3.3215970238490664e-05, 'calidad_B-negative': 9.506305849546866e-05, 'desproporcionado_O': 3.3215970238490664e-05, 'jamon_B-positive': 6.184865633794106e-05, 'sidreria_O': 3.3215970238490664e-05, 'pondria_O': 4.9823955357735996e-05, 'jamoneria_O': 3.3215970238490664e-05, 'mivida_O': 3.3215970238490664e-05, 'tome_O': 3.3215970238490664e-05, 'jamonun_O': 3.3215970238490664e-05, 'sugerido_O': 3.3215970238490664e-05, 'tomate_I-negative': 9.56998851601378e-05, '700_O': 3.3215970238490664e-05, 'invitaran_O': 3.3215970238490664e-05, 'chupito_O': 4.9823955357735996e-05, 'vecino_O': 3.3215970238490664e-05, 'Privacidad_B-negative': 6.337537233031244e-05, 'cero_O': 3.3215970238490664e-05, 'pequeñito_O': 4.9823955357735996e-05, 'x_O': 3.3215970238490664e-05, 'molesta_O': 3.3215970238490664e-05, 'excesivamente_O': 8.303992559622666e-05, 'conversar_O': 4.9823955357735996e-05, 'volúmen_O': 3.3215970238490664e-05, 'Fueron_O': 3.3215970238490664e-05, 'lentos_O': 3.3215970238490664e-05, 'descanso_O': 3.3215970238490664e-05, 'allá_O': 3.3215970238490664e-05, 'actos_O': 3.3215970238490664e-05, 'ciega_O': 3.3215970238490664e-05, 'mercado_O': 6.643194047698133e-05, 'Comunion_O': 3.3215970238490664e-05, 'Nuestra_O': 3.3215970238490664e-05, 'Hija_O': 3.3215970238490664e-05, 'Dia_O': 3.3215970238490664e-05, 'Lluvia_O': 3.3215970238490664e-05, 'Importo_O': 3.3215970238490664e-05, 'Porque_O': 3.3215970238490664e-05, 'Preparativos_O': 3.3215970238490664e-05, 'Demas_O': 3.3215970238490664e-05, 'Atencion_B-positive': 6.184865633794106e-05, 'Camareros_O': 4.9823955357735996e-05, 'Destacable_O': 3.3215970238490664e-05, 'Genial.El_O': 3.3215970238490664e-05, 'Ternasco_B-positive': 6.184865633794106e-05, 'Salio_O': 3.3215970238490664e-05, 'Bordado.Muchas_O': 3.3215970238490664e-05, 'Sois_O': 3.3215970238490664e-05, 'Mejores_O': 3.3215970238490664e-05, 'melon_B-positive': 6.184865633794106e-05, 'jamon_I-positive': 6.351020926613953e-05, 'estarían_O': 3.3215970238490664e-05, 'ricos_O': 4.9823955357735996e-05, 'forraron_O': 3.3215970238490664e-05, 'huevas_B-negative': 6.337537233031244e-05, 'pescado_I-negative': 6.379992344009187e-05, 'color_I-negative': 6.379992344009187e-05, 'naranja_I-negative': 9.56998851601378e-05, 'fuerte_O': 3.3215970238490664e-05, 'conservadas_O': 3.3215970238490664e-05, 'sal_O': 9.964791071547199e-05, 'énmascaraban_O': 3.3215970238490664e-05, 'originarl_O': 3.3215970238490664e-05, 'platos.No_O': 3.3215970238490664e-05, 'casualidad_O': 3.3215970238490664e-05, 'añadir_O': 3.3215970238490664e-05, 'huevas_O': 4.9823955357735996e-05, 'latita_O': 3.3215970238490664e-05, 'adición_O': 3.3215970238490664e-05, 'salado_O': 3.3215970238490664e-05, 'mencionado_O': 3.3215970238490664e-05, 'Felicitar_O': 3.3215970238490664e-05, 'aqii_O': 3.3215970238490664e-05, 'direccion_O': 3.3215970238490664e-05, 'elaboracion_O': 3.3215970238490664e-05, 'esmerada_O': 4.9823955357735996e-05, 'medida_O': 4.9823955357735996e-05, 'autores_O': 3.3215970238490664e-05, 'salen_O': 3.3215970238490664e-05, 'ovación_O': 3.3215970238490664e-05, 'cerrada_O': 3.3215970238490664e-05, 'vuelta_O': 3.3215970238490664e-05, 'ruedo_O': 3.3215970238490664e-05, 'orejas_O': 3.3215970238490664e-05, 'Repito_O': 3.3215970238490664e-05, 'apreciación_O': 4.9823955357735996e-05, 'compartimos_O': 3.3215970238490664e-05, 'carpaccio_B-positive': 6.184865633794106e-05, 'Kobe_I-positive': 6.351020926613953e-05, 'sabrosísimo_O': 3.3215970238490664e-05, 'comenillas_O': 3.3215970238490664e-05, 'rellenas_O': 3.3215970238490664e-05, 'alchofas_O': 3.3215970238490664e-05, 'cola_B-neutral': 6.40020480655381e-05, 'entera_I-neutral': 6.406149903907751e-05, 'bogavante_I-neutral': 6.406149903907751e-05, 'la_I-neutral': 6.406149903907751e-05, 'plancha_I-neutral': 6.406149903907751e-05, 'salsa_I-neutral': 6.406149903907751e-05, 'su_I-neutral': 6.406149903907751e-05, 'coral_I-neutral': 6.406149903907751e-05, 'sabrosa_O': 6.643194047698133e-05, 'Antes_O': 3.3215970238490664e-05, 'ofreció_O': 3.3215970238490664e-05, 'pinco_O': 3.3215970238490664e-05, 'queso_O': 6.643194047698133e-05, 'palo_O': 3.3215970238490664e-05, 'canela_O': 4.9823955357735996e-05, 'encendido_O': 3.3215970238490664e-05, 'Hemosido_O': 3.3215970238490664e-05, 'Menú_B-positive': 6.184865633794106e-05, 'Jornadas_O': 3.3215970238490664e-05, 'huerta_O': 3.3215970238490664e-05, 'posibilidad_O': 3.3215970238490664e-05, 'tomado_O': 4.9823955357735996e-05, 'pensado_O': 4.9823955357735996e-05, 'caer_O': 4.9823955357735996e-05, 'pesadez_O': 3.3215970238490664e-05, 'Dicen_O': 3.3215970238490664e-05, 'Menos_O': 3.3215970238490664e-05, 'acertada.felicidades_O': 3.3215970238490664e-05, 'sr_B-positive': 6.184865633794106e-05, '._I-positive': 6.351020926613953e-05, 'Aranda_I-positive': 6.351020926613953e-05, 'paellas_B-positive': 9.277298450691158e-05, 'magret_B-negative': 6.337537233031244e-05, 'fino_O': 4.9823955357735996e-05, 'pelitos_O': 3.3215970238490664e-05, 'ideal_O': 4.9823955357735996e-05, 'restaurantes_B-positive': 9.277298450691158e-05, 'quise_O': 3.3215970238490664e-05, 'circo_O': 3.3215970238490664e-05, 'libro_O': 3.3215970238490664e-05, 'reclamaciones_O': 3.3215970238490664e-05, 'sentí_O': 3.3215970238490664e-05, 'estafada.En_O': 3.3215970238490664e-05, 'aconsejo_O': 3.3215970238490664e-05, 'animo_O': 3.3215970238490664e-05, 'compre_O': 3.3215970238490664e-05, 'cupones_O': 3.3215970238490664e-05, 'arriesgan_O': 3.3215970238490664e-05, 'sirvan_O': 3.3215970238490664e-05, 'consideren_O': 3.3215970238490664e-05, 'independientemente_O': 3.3215970238490664e-05, 'anterioridad_O': 3.3215970238490664e-05, 'acerque_O': 3.3215970238490664e-05, 'quedará_O': 3.3215970238490664e-05, 'insatisfecho_O': 3.3215970238490664e-05, 'Evidentemente_O': 3.3215970238490664e-05, 'creemos_O': 3.3215970238490664e-05, 'invitacion_O': 3.3215970238490664e-05, 'comedores_O': 3.3215970238490664e-05, 'llenos_O': 3.3215970238490664e-05, 'Central_O': 3.3215970238490664e-05, 'Reservas_O': 3.3215970238490664e-05, 'homogéneo_O': 3.3215970238490664e-05, 'defraudará_O': 3.3215970238490664e-05, 'Apuntada_O': 3.3215970238490664e-05, 'LA_B-negative': 6.337537233031244e-05, 'DESPENSA_I-negative': 6.379992344009187e-05, 'lea_O': 3.3215970238490664e-05, 'critica_O': 3.3215970238490664e-05, 'defenderse_O': 3.3215970238490664e-05, 'defender_O': 6.643194047698133e-05, 'rie_O': 3.3215970238490664e-05, 'sabra_O': 3.3215970238490664e-05, 'pudiera_O': 3.3215970238490664e-05, 'seria_O': 3.3215970238490664e-05, 'DE_O': 9.964791071547199e-05, 'VERGÜENZA_O': 3.3215970238490664e-05, 'tostaditas_O': 3.3215970238490664e-05, 'bravas_O': 3.3215970238490664e-05, 'deshecho_O': 3.3215970238490664e-05, 'incomible_O': 3.3215970238490664e-05, 'cigala_O': 3.3215970238490664e-05, 'hay_B-negative': 6.337537233031244e-05, 'mucho_I-negative': 6.379992344009187e-05, 'hacía_O': 4.9823955357735996e-05, 'veníamos_O': 3.3215970238490664e-05, 'Born_O': 3.3215970238490664e-05, 'concreto_O': 3.3215970238490664e-05, 'pulpo_B-positive': 6.184865633794106e-05, 'langostinos_I-positive': 6.351020926613953e-05, 'quejar_O': 3.3215970238490664e-05, 'buenisimo_O': 6.643194047698133e-05, 'corresponde_O': 3.3215970238490664e-05, 'oyes_O': 3.3215970238490664e-05, 'quere_O': 3.3215970238490664e-05, 'conversaciones_O': 6.643194047698133e-05, 'Cantidad_O': 3.3215970238490664e-05, 'insuficiente_O': 3.3215970238490664e-05, 'admiten_O': 3.3215970238490664e-05, 'perros_O': 3.3215970238490664e-05, 'países_O': 3.3215970238490664e-05, 'Pocas_O': 3.3215970238490664e-05, 'coincide_O': 3.3215970238490664e-05, 'criticos_O': 3.3215970238490664e-05, 'podido_O': 3.3215970238490664e-05, 'mayo_O': 4.9823955357735996e-05, '3,5_O': 4.9823955357735996e-05, 'croqueta_B-negative': 6.337537233031244e-05, 'gasto_O': 3.3215970238490664e-05, 'bolsillo_O': 3.3215970238490664e-05, 'verá_O': 3.3215970238490664e-05, 'molestan_O': 3.3215970238490664e-05, 'enuncian_O': 3.3215970238490664e-05, 'servirlos.Por_O': 3.3215970238490664e-05, 'responden.Pero_O': 3.3215970238490664e-05, 'trata_O': 3.3215970238490664e-05, 'disfruten_O': 3.3215970238490664e-05, 'ambiente_O': 4.9823955357735996e-05, 'parace_O': 3.3215970238490664e-05, 'habitaciones_O': 3.3215970238490664e-05, 'aguas_O': 4.9823955357735996e-05, 'beneficiosas_O': 3.3215970238490664e-05, 'problemas_O': 3.3215970238490664e-05, 'respiratorios_O': 3.3215970238490664e-05, 'sorprende_O': 3.3215970238490664e-05, 'promoción_O': 4.9823955357735996e-05, 'región_O': 3.3215970238490664e-05, 'visto_O': 3.3215970238490664e-05, 'apostar_O': 3.3215970238490664e-05, 'restaurante.Los_O': 3.3215970238490664e-05, 'satisfactorio_O': 3.3215970238490664e-05, 'Intento_O': 3.3215970238490664e-05, 'buscar_O': 3.3215970238490664e-05, 'libre_O': 8.303992559622666e-05, 'Tempo_B-positive': 6.184865633794106e-05, 'haga_O': 3.3215970238490664e-05, 'seré_O': 3.3215970238490664e-05, 'defenderlo_O': 3.3215970238490664e-05, 'Dios_O': 3.3215970238490664e-05, 'flaco_O': 3.3215970238490664e-05, 'recomedación_O': 3.3215970238490664e-05, 'Prefiero_O': 4.9823955357735996e-05, 'desengaño_O': 3.3215970238490664e-05, 'tuve_O': 4.9823955357735996e-05, 'nubes_O': 3.3215970238490664e-05, 'aumento_O': 3.3215970238490664e-05, 'consumo_O': 3.3215970238490664e-05, 'politiqueos_O': 3.3215970238490664e-05, '8,5_O': 3.3215970238490664e-05, 'Estube_O': 3.3215970238490664e-05, 'bal_O': 3.3215970238490664e-05, 'osera_O': 3.3215970238490664e-05, 'disculpan_O': 3.3215970238490664e-05, 'percheros_B-negative': 6.337537233031244e-05, 'provisonales_O': 3.3215970238490664e-05, 'hubieran_O': 3.3215970238490664e-05, 'encantador.Al_O': 3.3215970238490664e-05, 'resulto_O': 3.3215970238490664e-05, 'agradableLa_O': 3.3215970238490664e-05, 'unica_O': 3.3215970238490664e-05, 'telefono_B-negative': 6.337537233031244e-05, 'pagina_O': 3.3215970238490664e-05, 'tendriamos_O': 3.3215970238490664e-05, 'Total_O': 4.9823955357735996e-05, 'disfutamos_O': 3.3215970238490664e-05, 'muchisimo_O': 3.3215970238490664e-05, 'volvereoms_O': 3.3215970238490664e-05, 'aplicaron_O': 4.9823955357735996e-05, 'Vacío_O': 3.3215970238490664e-05, 'Entrañas_O': 3.3215970238490664e-05, 'especialidades_O': 3.3215970238490664e-05, 'conocidas_O': 3.3215970238490664e-05, 'blando_O': 4.9823955357735996e-05, 'Entramos_O': 4.9823955357735996e-05, '12.45_O': 3.3215970238490664e-05, 'tomamos_O': 4.9823955357735996e-05, 'compensado_O': 3.3215970238490664e-05, 'nula_O': 3.3215970238490664e-05, 'educación_B-negative': 6.337537233031244e-05, 'credenciales_O': 3.3215970238490664e-05, 'acelerados_O': 3.3215970238490664e-05, 'estuvieran_O': 3.3215970238490664e-05, 'apagando_O': 3.3215970238490664e-05, 'conato_O': 3.3215970238490664e-05, 'incendio_O': 3.3215970238490664e-05, 'fantastico_O': 3.3215970238490664e-05, '3_I-positive': 6.351020926613953e-05, 'quesos_I-positive': 6.351020926613953e-05, 'cabra_B-positive': 6.184865633794106e-05, 'caliente_I-positive': 6.351020926613953e-05, 'sobre_I-positive': 6.351020926613953e-05, 'pure_I-positive': 6.351020926613953e-05, 'manzana_I-positive': 6.351020926613953e-05, 'positiva_O': 3.3215970238490664e-05, 'Teniendo_O': 3.3215970238490664e-05, 'mayores_O': 4.9823955357735996e-05, 'acomodarnos_O': 3.3215970238490664e-05, 'comentado_O': 3.3215970238490664e-05, 'maravillas_O': 3.3215970238490664e-05, 'riggatoni_B-negative': 6.337537233031244e-05, 'bolognesa_I-negative': 6.379992344009187e-05, 'escueto_O': 3.3215970238490664e-05, 'suplio_O': 3.3215970238490664e-05, 'saciados_O': 3.3215970238490664e-05, 'Felicidades_O': 4.9823955357735996e-05, 'completos_O': 3.3215970238490664e-05, 'especialidad_O': 4.9823955357735996e-05, 'sirveiron_O': 3.3215970238490664e-05, 'rebozados_O': 3.3215970238490664e-05, 'sardinitas_O': 3.3215970238490664e-05, 'refritas_O': 3.3215970238490664e-05, 'immundo_O': 3.3215970238490664e-05, 'pésimo_O': 3.3215970238490664e-05, 'enteraba_O': 3.3215970238490664e-05, 'riquísimos_O': 3.3215970238490664e-05, 'Calor_O': 3.3215970238490664e-05, 'colmo_O': 4.9823955357735996e-05, 'amenizaron_O': 3.3215970238490664e-05, 'musica_B-negative': 9.506305849546866e-05, 'volumen_O': 3.3215970238490664e-05, 'musica_O': 3.3215970238490664e-05, 'latinoamericana_O': 3.3215970238490664e-05, 'incongruencia_O': 3.3215970238490664e-05, 'llama_O': 3.3215970238490664e-05, 'italiana_O': 3.3215970238490664e-05, 'Pasable_O': 3.3215970238490664e-05, 'reforma_B-positive': 9.277298450691158e-05, 'estrecho_O': 3.3215970238490664e-05, 'alargado_O': 3.3215970238490664e-05, 'comunicarse_O': 3.3215970238490664e-05, 'interior_B-negative': 6.337537233031244e-05, 'sentirse_O': 3.3215970238490664e-05, 'incómodo_O': 3.3215970238490664e-05, 'prefiero_O': 3.3215970238490664e-05, 'encuentro_O': 3.3215970238490664e-05, 'agusto_O': 3.3215970238490664e-05, 'fuisteis_O': 3.3215970238490664e-05, '32_O': 6.643194047698133e-05, 'disfrutásteis_O': 3.3215970238490664e-05, 'gustara_O': 4.9823955357735996e-05, 'pasó_O': 3.3215970238490664e-05, 'salierais_O': 3.3215970238490664e-05, 'Paella_B-negative': 6.337537233031244e-05, 'coloreada_O': 3.3215970238490664e-05, 'bañada_O': 4.9823955357735996e-05, 'gamba_B-negative': 6.337537233031244e-05, 'immunda_O': 3.3215970238490664e-05, 'entienden_O': 3.3215970238490664e-05, 'discúlpeme_O': 3.3215970238490664e-05, 'responda_O': 3.3215970238490664e-05, 'destacaria_O': 3.3215970238490664e-05, 'bacalao_B-positive': 9.277298450691158e-05, 'gustoso_O': 3.3215970238490664e-05, 'bogavante_I-positive': 6.351020926613953e-05, 'recordaremos_O': 3.3215970238490664e-05, 'alegra_O': 3.3215970238490664e-05, 'bravas_B-negative': 6.337537233031244e-05, 'picantes_O': 3.3215970238490664e-05, '22:30_O': 3.3215970238490664e-05, 'permitía_O': 3.3215970238490664e-05, '23_O': 3.3215970238490664e-05, 'éramos_O': 3.3215970238490664e-05, '21:00_O': 3.3215970238490664e-05, 'di_O': 3.3215970238490664e-05, 'Candelas_O': 4.9823955357735996e-05, 'estamos_O': 3.3215970238490664e-05, 'habituadas_O': 3.3215970238490664e-05, 'montón_O': 3.3215970238490664e-05, 'categorías_O': 3.3215970238490664e-05, 'cueste_O': 3.3215970238490664e-05, 'diario_I-positive': 6.351020926613953e-05, 'cuidada_O': 3.3215970238490664e-05, '16_O': 3.3215970238490664e-05, 'autentica_O': 3.3215970238490664e-05, 'joya_O': 3.3215970238490664e-05, 'ilustrada_I-positive': 6.351020926613953e-05, 'morcilla_O': 3.3215970238490664e-05, 'chorizo_O': 6.643194047698133e-05, 'ó_O': 3.3215970238490664e-05, 'Aragón_I-positive': 6.351020926613953e-05, 'sencillamente_O': 3.3215970238490664e-05, 'Seguramente_O': 3.3215970238490664e-05, 'Encima_O': 3.3215970238490664e-05, 'pantalla_O': 3.3215970238490664e-05, 'televisor_O': 3.3215970238490664e-05, 'sonido_B-negative': 6.337537233031244e-05, 'oido_O': 3.3215970238490664e-05, 'izquierdo_O': 3.3215970238490664e-05, 'sepuede_O': 3.3215970238490664e-05, 'expectación_O': 3.3215970238490664e-05, 'Mesas_B-negative': 6.337537233031244e-05, 'trato_B-negative': 6.337537233031244e-05, 'bebe_O': 3.3215970238490664e-05, 'intentamos_O': 3.3215970238490664e-05, 'concurso_O': 4.9823955357735996e-05, 'celebra_O': 3.3215970238490664e-05, 'maneras_O': 3.3215970238490664e-05, 'comia_O': 3.3215970238490664e-05, 'hacian_O': 3.3215970238490664e-05, 'propaganda_O': 3.3215970238490664e-05, 'viene_O': 3.3215970238490664e-05, 'querían_O': 3.3215970238490664e-05, 'sentar_O': 3.3215970238490664e-05, 'carrito_O': 3.3215970238490664e-05, 'ocurrió_O': 3.3215970238490664e-05, 'comisteis_O': 3.3215970238490664e-05, '27_O': 4.9823955357735996e-05, 'reservaste_O': 3.3215970238490664e-05, '32-27_O': 3.3215970238490664e-05, '=_O': 3.3215970238490664e-05, 'equitativa_O': 3.3215970238490664e-05, 'proponen_O': 3.3215970238490664e-05, 'bogavante_O': 3.3215970238490664e-05, 'cigalas_O': 3.3215970238490664e-05, 'recordé_O': 3.3215970238490664e-05, 'dignó_O': 3.3215970238490664e-05, 'contestarme_O': 3.3215970238490664e-05, 'causandome_O': 3.3215970238490664e-05, 'combinaciones_B-positive': 6.184865633794106e-05, 'equilibradas_O': 3.3215970238490664e-05, 'tomate_B-neutral': 6.40020480655381e-05, 'el_I-neutral': 6.406149903907751e-05, 'atún_I-neutral': 6.406149903907751e-05, 'marinado_I-neutral': 6.406149903907751e-05, 'bogavante_B-positive': 6.184865633794106e-05, 'poché_I-positive': 6.351020926613953e-05, 'Nunca_O': 6.643194047698133e-05, 'cambiamos_O': 3.3215970238490664e-05, 'anunciaba_O': 3.3215970238490664e-05, 'limpia_O': 3.3215970238490664e-05, 'lis_O': 3.3215970238490664e-05, 'crees_O': 3.3215970238490664e-05, 'cosiderar_O': 3.3215970238490664e-05, 'aragonés_O': 4.9823955357735996e-05, 'enofílicamente_O': 3.3215970238490664e-05, 'ínfima_O': 3.3215970238490664e-05, 'continuas_O': 3.3215970238490664e-05, 'consideras_O': 3.3215970238490664e-05, 'pichi_O': 3.3215970238490664e-05, 'usas_O': 3.3215970238490664e-05, 'exponente_O': 3.3215970238490664e-05, 'populista_O': 3.3215970238490664e-05, 'terminas_O': 3.3215970238490664e-05, 'Recomiendo_O': 9.964791071547199e-05, 'falte_O': 3.3215970238490664e-05, 'seguid_O': 3.3215970238490664e-05, 'disfrutando_O': 3.3215970238490664e-05, 'establecimiento_O': 3.3215970238490664e-05, 'Precios_O': 3.3215970238490664e-05, 'Camareros_B-negative': 6.337537233031244e-05, 'despistados_O': 3.3215970238490664e-05, 'apuntaron_O': 3.3215970238490664e-05, 'trajeron_O': 6.643194047698133e-05, 'discusiones_O': 3.3215970238490664e-05, 'correctísimo_O': 3.3215970238490664e-05, 'Besugo_B-negative': 6.337537233031244e-05, 'orrio_I-negative': 6.379992344009187e-05, 'Incomestible_O': 3.3215970238490664e-05, 'vinagre_O': 3.3215970238490664e-05, 'soportar.TODO_O': 3.3215970238490664e-05, 'ESTO_O': 3.3215970238490664e-05, 'DECIMOS_O': 3.3215970238490664e-05, 'BOCA_O': 3.3215970238490664e-05, 'LLENA_O': 3.3215970238490664e-05, 'SIN_O': 4.9823955357735996e-05, 'TAPUJOS_O': 3.3215970238490664e-05, 'SOMOS_O': 3.3215970238490664e-05, 'HUELVA_O': 3.3215970238490664e-05, 'ALMERÍA_O': 3.3215970238490664e-05, 'BADAJOZ_O': 3.3215970238490664e-05, 'TODO_O': 3.3215970238490664e-05, 'QUE_O': 3.3215970238490664e-05, 'PEDIMOS_O': 3.3215970238490664e-05, 'CONOCEMOS_O': 3.3215970238490664e-05, 'PERFECCIÓN_O': 3.3215970238490664e-05, 'SERVIDO_O': 3.3215970238490664e-05, 'SE_O': 3.3215970238490664e-05, 'PARECE_O': 3.3215970238490664e-05, 'REALIDAD_O': 3.3215970238490664e-05, 'COMO_O': 3.3215970238490664e-05, 'UN_O': 4.9823955357735996e-05, 'HUEVO_O': 3.3215970238490664e-05, 'CASTAÑA_O': 3.3215970238490664e-05, 'súper_O': 3.3215970238490664e-05, 'Solomillo_O': 3.3215970238490664e-05, 'Pirineo_O': 3.3215970238490664e-05, 'tallarines_O': 3.3215970238490664e-05, 'pimiento_O': 3.3215970238490664e-05, 'rojo_O': 4.9823955357735996e-05, 'patata_O': 4.9823955357735996e-05, 'cremosa_O': 3.3215970238490664e-05, '21,45_O': 3.3215970238490664e-05, 'h_O': 4.9823955357735996e-05, '23,30_O': 3.3215970238490664e-05, 'dónde_O': 3.3215970238490664e-05, 'cena-espectáculo_B-negative': 6.337537233031244e-05, 'anuncian_O': 3.3215970238490664e-05, 'maitre_B-positive': 9.277298450691158e-05, 'acertados_O': 3.3215970238490664e-05, 'traves_O': 3.3215970238490664e-05, 'Justito_O': 3.3215970238490664e-05, 'probaron_O': 3.3215970238490664e-05, 'sentó_O': 3.3215970238490664e-05, 'entrañable_O': 3.3215970238490664e-05, 'íntima_O': 3.3215970238490664e-05, 'cosmopolita_O': 3.3215970238490664e-05, 'Mire_O': 3.3215970238490664e-05, 'Arzak_O': 3.3215970238490664e-05, 'sobremesa.Los_O': 3.3215970238490664e-05, 'aliciente_O': 3.3215970238490664e-05, 'cine_O': 4.9823955357735996e-05, 'reirte_O': 3.3215970238490664e-05, 'llorar_O': 3.3215970238490664e-05, 'dormirte_O': 3.3215970238490664e-05, 'vale_B-negative': 6.337537233031244e-05, 'descuento_I-negative': 6.379992344009187e-05, 'arroz_B-negative': 6.337537233031244e-05, 'caldoso_I-negative': 6.379992344009187e-05, 'devuelto_O': 3.3215970238490664e-05, 'posterior_O': 3.3215970238490664e-05, 'anecdótico_O': 3.3215970238490664e-05, 'ampoco_O': 3.3215970238490664e-05, 'tocar_O': 4.9823955357735996e-05, 'incomestible_O': 3.3215970238490664e-05, 'aversión_O': 3.3215970238490664e-05, 'morro_O': 4.9823955357735996e-05, 'devuelvo_O': 3.3215970238490664e-05, 'conveniente_O': 3.3215970238490664e-05, 'daré_O': 3.3215970238490664e-05, 'propia_O': 3.3215970238490664e-05, 'iniciativa_O': 3.3215970238490664e-05, 'Cumplió_O': 3.3215970238490664e-05, 'molesto_O': 3.3215970238490664e-05, 'contratamos_O': 3.3215970238490664e-05, 'vimos_O': 6.643194047698133e-05, 'prácticamente_O': 4.9823955357735996e-05, 'Notable_O': 3.3215970238490664e-05, 'Servian_O': 3.3215970238490664e-05, 'regando_O': 3.3215970238490664e-05, 'salvo_O': 8.303992559622666e-05, 'gano_O': 3.3215970238490664e-05, 'convertirlo_O': 3.3215970238490664e-05, 'satisfactoria_O': 3.3215970238490664e-05, 'Esplendido_O': 3.3215970238490664e-05, 'parellada_O': 3.3215970238490664e-05, 'artoz_O': 3.3215970238490664e-05, 'hervido_O': 3.3215970238490664e-05, 'Quería_O': 3.3215970238490664e-05, 'afluencia_O': 4.9823955357735996e-05, 'noches_O': 3.3215970238490664e-05, 'den_O': 3.3215970238490664e-05, 'iva_O': 3.3215970238490664e-05, 'aportar_O': 3.3215970238490664e-05, 'ocupado_O': 3.3215970238490664e-05, 'leí_O': 3.3215970238490664e-05, 'confundieron_O': 3.3215970238490664e-05, 'Arroz_B-negative': 6.337537233031244e-05, 'Señorito_I-negative': 6.379992344009187e-05, 'verdadero_O': 3.3215970238490664e-05, 'desastre_O': 4.9823955357735996e-05, 'aplemazado_O': 3.3215970238490664e-05, 'cayó_O': 3.3215970238490664e-05, 'vera_O': 4.9823955357735996e-05, 'ahumado_O': 3.3215970238490664e-05, 'IMPOSIBLE_O': 3.3215970238490664e-05, 'COMERLO_O': 3.3215970238490664e-05, 'encarecidamente_O': 3.3215970238490664e-05, 'optar_O': 3.3215970238490664e-05, 'Wok_B-positive': 6.184865633794106e-05, 'Via_O': 3.3215970238490664e-05, 'Universitas_O': 3.3215970238490664e-05, 'experiencias_O': 3.3215970238490664e-05, 'últimos_O': 3.3215970238490664e-05, 'cAfe_O': 3.3215970238490664e-05, 'Miguel_O': 3.3215970238490664e-05, 'postre_B-neutral': 6.40020480655381e-05, 'platano_O': 3.3215970238490664e-05, 'mousse_O': 3.3215970238490664e-05, 'galleta_O': 3.3215970238490664e-05, 'Hicimos_O': 3.3215970238490664e-05, 'variados_O': 4.9823955357735996e-05, 'excusa_O': 3.3215970238490664e-05, 'justitas_O': 3.3215970238490664e-05, 'presentación_B-positive': 0.00012369731267588213, 'combinación_B-positive': 6.184865633794106e-05, 'propio_O': 3.3215970238490664e-05, 'recomendaciones_O': 4.9823955357735996e-05, 'impresiones_O': 3.3215970238490664e-05, 'personales_O': 3.3215970238490664e-05, 'defraude_O': 3.3215970238490664e-05, 'creativa_O': 3.3215970238490664e-05, 'éxito_O': 3.3215970238490664e-05, 'encargaré_O': 3.3215970238490664e-05, 'han_B-negative': 6.337537233031244e-05, 'subido_I-negative': 6.379992344009187e-05, 'bastante_I-negative': 6.379992344009187e-05, 'los_I-negative': 6.379992344009187e-05, 'precios_I-negative': 6.379992344009187e-05, 'acompaña_O': 3.3215970238490664e-05, 'pedirla_O': 3.3215970238490664e-05, 'puntuacion_O': 3.3215970238490664e-05, 'Fueron_B-positive': 6.184865633794106e-05, 'todos_I-positive': 6.351020926613953e-05, 'amables_I-positive': 6.351020926613953e-05, 'atentos_I-positive': 6.351020926613953e-05, 'servilleta_B-negative': 6.337537233031244e-05, 'carbonilla_O': 3.3215970238490664e-05, 'Demasiado_O': 3.3215970238490664e-05, 'asiatico_O': 3.3215970238490664e-05, 'laborales_O': 3.3215970238490664e-05, 'sigo_O': 3.3215970238490664e-05, 'pensando_O': 3.3215970238490664e-05, 'aprovechadas_O': 4.9823955357735996e-05, 'Seguiremos_O': 3.3215970238490664e-05, 'vengamos_O': 3.3215970238490664e-05, 'Barna_O': 3.3215970238490664e-05, 'pudo_O': 4.9823955357735996e-05, 'tiramos_O': 3.3215970238490664e-05, 'atún_O': 4.9823955357735996e-05, 'teriyaki_O': 3.3215970238490664e-05, 'steak_O': 3.3215970238490664e-05, 'tartare_O': 3.3215970238490664e-05, 'ferrero_O': 3.3215970238490664e-05, 'rocher_O': 3.3215970238490664e-05, 'torrija_O': 3.3215970238490664e-05, 'chilindron_O': 3.3215970238490664e-05, 'agarrado_O': 3.3215970238490664e-05, 'albondigas_B-negative': 6.337537233031244e-05, 'dejaban_O': 3.3215970238490664e-05, 'albondigas_O': 3.3215970238490664e-05, 'cucharita_O': 3.3215970238490664e-05, 'brandada_O': 3.3215970238490664e-05, 'caldo_O': 3.3215970238490664e-05, 'ardo_O': 3.3215970238490664e-05, 'espuma_O': 3.3215970238490664e-05, 'delicado_O': 3.3215970238490664e-05, 'SORPRENDENTE_O': 3.3215970238490664e-05, 'temporada_I-positive': 6.351020926613953e-05, 'sales_O': 3.3215970238490664e-05, 'deberías_O': 3.3215970238490664e-05, 'infinitamente_O': 3.3215970238490664e-05, 'Quizá_O': 3.3215970238490664e-05, 'podría_O': 3.3215970238490664e-05, 'distribución_B-negative': 6.337537233031244e-05, 'mesas_I-negative': 6.379992344009187e-05, 'pasando_O': 8.303992559622666e-05, 'contínuamente_O': 3.3215970238490664e-05, 'Disfrutamos_O': 3.3215970238490664e-05, 'Lugar_B-positive': 6.184865633794106e-05, 'media-alta_O': 3.3215970238490664e-05, 'xato_I-positive': 6.351020926613953e-05, 'magret_B-positive': 6.184865633794106e-05, 'Bailar_O': 3.3215970238490664e-05, 'extensísima_O': 3.3215970238490664e-05, 'instalaciones_B-positive': 6.184865633794106e-05, 'cuidadísimas_O': 3.3215970238490664e-05, 'ensalzan_O': 3.3215970238490664e-05, 'stick_B-positive': 6.184865633794106e-05, 'tartar_I-positive': 6.351020926613953e-05, 'dura_O': 3.3215970238490664e-05, 'definitiva_O': 4.9823955357735996e-05, 'moderación_O': 3.3215970238490664e-05, 'bebida_O': 4.9823955357735996e-05, '55_O': 3.3215970238490664e-05, 'homenaje_O': 3.3215970238490664e-05, 'quieras_O': 3.3215970238490664e-05, 'elaborada_O': 3.3215970238490664e-05, 'Tuvimos_O': 4.9823955357735996e-05, 'restando_O': 3.3215970238490664e-05, 'subiendo_O': 3.3215970238490664e-05, 'daría_O': 3.3215970238490664e-05, 'clientela_O': 4.9823955357735996e-05, 'tonta_O': 3.3215970238490664e-05, 'orejones_B-positive': 6.184865633794106e-05, 'Otro_O': 4.9823955357735996e-05, 'intente_O': 3.3215970238490664e-05, 'dulce_O': 3.3215970238490664e-05, 'decepcionante_O': 8.303992559622666e-05, 'suciedad_O': 3.3215970238490664e-05, 'trasnochado_O': 3.3215970238490664e-05, 'textura_O': 3.3215970238490664e-05, 'industrial_O': 3.3215970238490664e-05, 'diario_O': 3.3215970238490664e-05, '10,0_O': 3.3215970238490664e-05, 'menu_B-neutral': 6.40020480655381e-05, 'agregaria_O': 3.3215970238490664e-05, 'conejo_O': 3.3215970238490664e-05, 'estofado_O': 3.3215970238490664e-05, 'pavo_O': 3.3215970238490664e-05, 'pollo_O': 3.3215970238490664e-05, 'felicitos_O': 3.3215970238490664e-05, 'referencia_O': 6.643194047698133e-05, 'visitar_O': 3.3215970238490664e-05, 'restaurant_O': 4.9823955357735996e-05, '14_O': 4.9823955357735996e-05, 'molt_O': 3.3215970238490664e-05, 'be_O': 3.3215970238490664e-05, 'millor_O': 3.3215970238490664e-05, 'cancelar_O': 3.3215970238490664e-05, 'comunicando_O': 3.3215970238490664e-05, 'oportunamente_O': 3.3215970238490664e-05, 'intercambiar_O': 3.3215970238490664e-05, 'ideas_O': 3.3215970238490664e-05, 'quienes_O': 3.3215970238490664e-05, 'compartes_O': 3.3215970238490664e-05, 'tono_O': 3.3215970238490664e-05, 'Veraton_O': 3.3215970238490664e-05, 'descubrí_O': 3.3215970238490664e-05, 'ólive_O': 3.3215970238490664e-05, 'Seguid_O': 3.3215970238490664e-05, 'seguiréis_O': 3.3215970238490664e-05, 'llenando_O': 3.3215970238490664e-05, 'Perfectamente_O': 3.3215970238490664e-05, 'cartel_O': 3.3215970238490664e-05, 'BCN_O': 3.3215970238490664e-05, 'crujiente_O': 3.3215970238490664e-05, 'manzana_O': 4.9823955357735996e-05, 'vainilla_O': 6.643194047698133e-05, 'coulante_O': 3.3215970238490664e-05, 'marron_O': 3.3215970238490664e-05, 'glacé_O': 3.3215970238490664e-05, 'Pasamos_O': 3.3215970238490664e-05, 'celebrando_O': 4.9823955357735996e-05, 'cou_O': 3.3215970238490664e-05, 'TODOS_O': 3.3215970238490664e-05, 'Elegí_O': 3.3215970238490664e-05, 'defraudo_O': 3.3215970238490664e-05, 'tuvieran_O': 3.3215970238490664e-05, 'cuarta_O': 3.3215970238490664e-05, 'solicitamos_O': 3.3215970238490664e-05, 'amargaros_O': 3.3215970238490664e-05, 'probad_O': 3.3215970238490664e-05, 'pides_O': 3.3215970238490664e-05, 'curiosidad_O': 3.3215970238490664e-05, 'etiquetada_O': 3.3215970238490664e-05, 'inglés_O': 3.3215970238490664e-05, 'francés_O': 3.3215970238490664e-05, 'técnico_O': 3.3215970238490664e-05, 'dominio_O': 3.3215970238490664e-05, 'combinaciones_B-neutral': 6.40020480655381e-05, 'estéticamente_O': 3.3215970238490664e-05, 'conservadoras_O': 3.3215970238490664e-05, 'Todas_O': 3.3215970238490664e-05, 'pagando_O': 3.3215970238490664e-05, 'L_B-negative': 6.337537233031244e-05, '´_I-negative': 6.379992344009187e-05, 'Olive_I-negative': 6.379992344009187e-05, 'Escriba_O': 6.643194047698133e-05, 'hermano_O': 3.3215970238490664e-05, 'Salamanca_O': 3.3215970238490664e-05, 'sienten_O': 3.3215970238490664e-05, 'comí_O': 4.9823955357735996e-05, 'prudencia_O': 3.3215970238490664e-05, 'bocatería_O': 3.3215970238490664e-05, 'logísticos_O': 3.3215970238490664e-05, 'busque_O': 3.3215970238490664e-05, 'pensaba_O': 3.3215970238490664e-05, 'comería_O': 3.3215970238490664e-05, 'ambientacion_O': 3.3215970238490664e-05, 'marisco_I-positive': 6.351020926613953e-05, 'Tipico_O': 3.3215970238490664e-05, 'turistas_O': 3.3215970238490664e-05, 'griterio_B-negative': 6.337537233031244e-05, 'ensordecedor_O': 3.3215970238490664e-05, 'somo_O': 3.3215970238490664e-05, 'Lástima_O': 3.3215970238490664e-05, 'acompaño_O': 3.3215970238490664e-05, 'bajaron_O': 3.3215970238490664e-05, 'Pido_O': 3.3215970238490664e-05, 'Raimat_O': 3.3215970238490664e-05, 'Chardonay_O': 3.3215970238490664e-05, 'Pienso_O': 3.3215970238490664e-05, 'ponerlo_O': 3.3215970238490664e-05, 'cené_O': 4.9823955357735996e-05, 'chardonay_O': 3.3215970238490664e-05, 'cepa_O': 3.3215970238490664e-05, 'uva_O': 3.3215970238490664e-05, 'blanca_O': 4.9823955357735996e-05, 'gastronomico_O': 3.3215970238490664e-05, 'discreta_O': 3.3215970238490664e-05, 'refiere_O': 3.3215970238490664e-05, 'considerarse_O': 3.3215970238490664e-05, 'paella_O': 3.3215970238490664e-05, 'mostrarla_O': 3.3215970238490664e-05, 'devolución_O': 3.3215970238490664e-05, 'Preparación_O': 3.3215970238490664e-05, 'pescado_B-neutral': 6.40020480655381e-05, 'Ultimamente_O': 3.3215970238490664e-05, 'pilar_O': 3.3215970238490664e-05, 'ect_O': 3.3215970238490664e-05, 'ofertas_O': 3.3215970238490664e-05, 'anticipado_O': 3.3215970238490664e-05, 'carlos_O': 3.3215970238490664e-05, 'celebracion_O': 3.3215970238490664e-05, 'HOSTERIA_O': 3.3215970238490664e-05, 'BOTIN_O': 3.3215970238490664e-05, 'DEL_O': 3.3215970238490664e-05, 'CORREGIDOR_O': 3.3215970238490664e-05, 'c_O': 3.3215970238490664e-05, 'Josefa_O': 3.3215970238490664e-05, 'Amar_O': 3.3215970238490664e-05, 'Borbom_O': 3.3215970238490664e-05, 'confirmacion_O': 3.3215970238490664e-05, 'hija_O': 3.3215970238490664e-05, 'favorable_O': 3.3215970238490664e-05, 'relació_O': 3.3215970238490664e-05, 'Mal_B-negative': 6.337537233031244e-05, 'sabor_I-negative': 6.379992344009187e-05, 'boca_I-negative': 6.379992344009187e-05, 'tras_I-negative': 6.379992344009187e-05, 'haber_I-negative': 6.379992344009187e-05, 'leido_I-negative': 6.379992344009187e-05, 'las_I-negative': 6.379992344009187e-05, 'criticas_I-negative': 6.379992344009187e-05, 'sobre_I-negative': 6.379992344009187e-05, 'todo_I-negative': 6.379992344009187e-05, 'por_I-negative': 6.379992344009187e-05, 'salir_I-negative': 6.379992344009187e-05, 'sin_I-negative': 6.379992344009187e-05, 'comer_I-negative': 6.379992344009187e-05, 'Comida_O': 3.3215970238490664e-05, 'pinchos_B-negative': 6.337537233031244e-05, 'cigalas_I-positive': 6.351020926613953e-05, 'presentacion_B-positive': 6.184865633794106e-05, 'Paella_B-positive': 6.184865633794106e-05, 'mixta_I-positive': 6.351020926613953e-05, 'Local_B-negative': 6.337537233031244e-05, 'jamón_B-positive': 9.277298450691158e-05, 'langostinos_B-positive': 6.184865633794106e-05, 'rodaballo_B-positive': 6.184865633794106e-05, 'recordarlo_O': 3.3215970238490664e-05, 'Quizás_O': 3.3215970238490664e-05, 'caros_O': 4.9823955357735996e-05, 'degustación_I-negative': 9.56998851601378e-05, 'corto_O': 3.3215970238490664e-05, 'necesitas_O': 3.3215970238490664e-05, 'eleva_O': 3.3215970238490664e-05, 'justas_O': 3.3215970238490664e-05, 'francamente_O': 6.643194047698133e-05, 'tostada_O': 3.3215970238490664e-05, 'codorniz_O': 3.3215970238490664e-05, 'braseada_O': 3.3215970238490664e-05, 'Rapidez_O': 3.3215970238490664e-05, 'gestión_B-positive': 6.184865633794106e-05, 'pedido_I-positive': 6.351020926613953e-05, 'encargado_B-positive': 9.277298450691158e-05, 'sucintas_O': 3.3215970238490664e-05, 'pertinentes_O': 3.3215970238490664e-05, 'agradecen_O': 3.3215970238490664e-05, 'Necesitan_O': 3.3215970238490664e-05, 'quieren_O': 3.3215970238490664e-05, 'cobrando_O': 4.9823955357735996e-05, 'retaurante_O': 3.3215970238490664e-05, 'Pasta_O': 3.3215970238490664e-05, 'City_O': 3.3215970238490664e-05, 'linea_O': 3.3215970238490664e-05, 'olive_O': 3.3215970238490664e-05, 'calçotades_B-positive': 6.184865633794106e-05, 'dadas_O': 3.3215970238490664e-05, 'Heraldo_O': 3.3215970238490664e-05, 'merecimientos_O': 3.3215970238490664e-05, 'constante_O': 3.3215970238490664e-05, 'canelones_B-positive': 6.184865633794106e-05, 'gallina_I-positive': 6.351020926613953e-05, 'trufados_I-positive': 6.351020926613953e-05, 'pasamos_O': 4.9823955357735996e-05, 'cayos_B-positive': 6.184865633794106e-05, 'recomendaremos_O': 3.3215970238490664e-05, 'foros_O': 3.3215970238490664e-05, 'Generoso_O': 3.3215970238490664e-05, 'productos_B-negative': 6.337537233031244e-05, 'acabaron_O': 3.3215970238490664e-05, 'agradarme_O': 3.3215970238490664e-05, 'Acaso_O': 3.3215970238490664e-05, 'Aragoneses_O': 3.3215970238490664e-05, 'alquel_O': 3.3215970238490664e-05, 'desprecie_O': 3.3215970238490664e-05, 'servidor_O': 3.3215970238490664e-05, 'llevará_O': 3.3215970238490664e-05, 'Personal_B-positive': 6.184865633794106e-05, 'floja_O': 3.3215970238490664e-05, 'elejir_O': 3.3215970238490664e-05, 'Debe_O': 3.3215970238490664e-05, 'economia_O': 3.3215970238490664e-05, 'euracos_O': 3.3215970238490664e-05, 'ensalada_B-negative': 6.337537233031244e-05, 'segun_O': 3.3215970238490664e-05, '80_O': 3.3215970238490664e-05, 'garnacha.Onoro_O': 3.3215970238490664e-05, 'volvimos_O': 4.9823955357735996e-05, 'aparcó_O': 3.3215970238490664e-05, 'ofrecimiento_O': 3.3215970238490664e-05, 'grosería_O': 3.3215970238490664e-05, 'contento_O': 3.3215970238490664e-05, 'ultimo_O': 3.3215970238490664e-05, 'recurso_O': 3.3215970238490664e-05, 'partido_O': 3.3215970238490664e-05, 'todavia_O': 3.3215970238490664e-05, 'clara_O': 3.3215970238490664e-05, 'PD_O': 3.3215970238490664e-05, 'pesará_O': 3.3215970238490664e-05, 'calificación_O': 3.3215970238490664e-05, 'Novodabo_O': 3.3215970238490664e-05, 'pesó_O': 3.3215970238490664e-05, 'Cachirulo_O': 3.3215970238490664e-05, 'Dgusta_O': 3.3215970238490664e-05, 'tuviese_O': 3.3215970238490664e-05, 'espacios_O': 3.3215970238490664e-05, 'reducidos_O': 3.3215970238490664e-05, 'Especialmente_O': 3.3215970238490664e-05, 'buñuelos_B-positive': 6.184865633794106e-05, 'bacalao_I-positive': 6.351020926613953e-05, 'vicio_O': 3.3215970238490664e-05, 'particularmente_O': 3.3215970238490664e-05, 'frente_O': 3.3215970238490664e-05, 'ralacion_O': 3.3215970238490664e-05, 'Jefa_O': 3.3215970238490664e-05, 'Sala_O': 3.3215970238490664e-05, 'Ramon_O': 3.3215970238490664e-05, 'Estamos_O': 3.3215970238490664e-05, 'situado_O': 3.3215970238490664e-05, 'cogollo_O': 3.3215970238490664e-05, 'barcelonés_O': 3.3215970238490664e-05, 'reflejar_O': 3.3215970238490664e-05, 'auqnue_O': 3.3215970238490664e-05, 'cloaca_O': 3.3215970238490664e-05, 'limitó_O': 3.3215970238490664e-05, 'picotear_O': 3.3215970238490664e-05, 'probarlos_O': 3.3215970238490664e-05, 'Albericoque_O': 3.3215970238490664e-05, 'yogurt_O': 3.3215970238490664e-05, 'oveja_O': 3.3215970238490664e-05, 'caramelo_O': 3.3215970238490664e-05, 'aroma_O': 6.643194047698133e-05, 'saúco_O': 3.3215970238490664e-05, 'pechugas_O': 3.3215970238490664e-05, 'buffet_O': 3.3215970238490664e-05, 'sacar_O': 3.3215970238490664e-05, 'deseábamos_O': 4.9823955357735996e-05, 'intima_O': 3.3215970238490664e-05, 'Tacita_O': 3.3215970238490664e-05, 'barrio_O': 3.3215970238490664e-05, 'espacioso_O': 3.3215970238490664e-05, 'calsotada_O': 4.9823955357735996e-05, 'distendida_O': 3.3215970238490664e-05, 'consiste_O': 3.3215970238490664e-05, 'fantásticamente_O': 3.3215970238490664e-05, 'tiempo_B-neutral': 6.40020480655381e-05, 'espera_I-neutral': 6.406149903907751e-05, 'perdón_O': 3.3215970238490664e-05, 'ortográficos_O': 3.3215970238490664e-05, 'fruto_O': 3.3215970238490664e-05, 'prisas_O': 3.3215970238490664e-05, 'dispongo_O': 3.3215970238490664e-05, 'agradecería_O': 3.3215970238490664e-05, 'reportará_O': 3.3215970238490664e-05, 'encontrara_O': 3.3215970238490664e-05, 'restaurante.La_O': 3.3215970238490664e-05, 'aparecido_O': 3.3215970238490664e-05, 'normalmente_O': 3.3215970238490664e-05, 'compartido_O': 3.3215970238490664e-05, 'deseaba.Lo_O': 3.3215970238490664e-05, 'cafe_O': 3.3215970238490664e-05, 'mediterranea_O': 3.3215970238490664e-05, 'mini_O': 3.3215970238490664e-05, 'tapas_B-negative': 6.337537233031244e-05, 'bandejas_O': 3.3215970238490664e-05, 'dificilmente_O': 3.3215970238490664e-05, 'usar_O': 3.3215970238490664e-05, 'mismas_O': 3.3215970238490664e-05, 'subsistir_O': 3.3215970238490664e-05, 'nueve_O': 3.3215970238490664e-05, 'BUEN_O': 3.3215970238490664e-05, 'ENTENDEDOR_O': 3.3215970238490664e-05, 'POCAS_O': 3.3215970238490664e-05, 'PALABRAS_O': 3.3215970238490664e-05, 'BASTAN_O': 3.3215970238490664e-05, 'paladares_O': 3.3215970238490664e-05, 'selectos_O': 3.3215970238490664e-05, 'La_B-positive': 6.184865633794106e-05, 'Poma_I-positive': 6.351020926613953e-05, 'Esperamos_O': 3.3215970238490664e-05, 'veros_O': 3.3215970238490664e-05, 'invito_O': 3.3215970238490664e-05, 'prueben_O': 3.3215970238490664e-05, 'opinen_O': 3.3215970238490664e-05, 'rapidísimo_O': 3.3215970238490664e-05, 'entretenimiento_O': 3.3215970238490664e-05, 'migas_O': 3.3215970238490664e-05, 'fresas_O': 3.3215970238490664e-05, 'deshidratadas_O': 3.3215970238490664e-05, 'sopa_O': 3.3215970238490664e-05, 'copita_O': 3.3215970238490664e-05, 'secos_O': 3.3215970238490664e-05, 'hiciera_O': 3.3215970238490664e-05, 'efecto_O': 3.3215970238490664e-05, 'volcán_O': 3.3215970238490664e-05, 'burbujeante_O': 3.3215970238490664e-05, 'cogote_B-neutral': 6.40020480655381e-05, 'bacalao_B-neutral': 6.40020480655381e-05, 'situación_O': 3.3215970238490664e-05, 'estás_O': 3.3215970238490664e-05, 'Soy_O': 3.3215970238490664e-05, 'asidua_O': 3.3215970238490664e-05, 'convierte_O': 3.3215970238490664e-05, 'luz_B-negative': 6.337537233031244e-05, 'fluorescente_O': 3.3215970238490664e-05, 'cantina_O': 3.3215970238490664e-05, 'conejo_B-positive': 6.184865633794106e-05, 'ajillo_I-positive': 6.351020926613953e-05, 'aceitito_O': 3.3215970238490664e-05, 'haberlo_O': 3.3215970238490664e-05, 'cuento_O': 3.3215970238490664e-05, '113,00_O': 3.3215970238490664e-05, 'inteligencia_O': 3.3215970238490664e-05, 'lectores_O': 4.9823955357735996e-05, 'piensan.Hace_O': 3.3215970238490664e-05, 'fechas_O': 3.3215970238490664e-05, 'stock_O': 3.3215970238490664e-05, 'pagarlo_O': 3.3215970238490664e-05, 'límites_O': 3.3215970238490664e-05, 'vengan_O': 3.3215970238490664e-05, 'Señores_O': 3.3215970238490664e-05, 'colocaron_O': 3.3215970238490664e-05, 'romántico_O': 3.3215970238490664e-05, 'rincón_O': 3.3215970238490664e-05, 'obsequiaron_O': 3.3215970238490664e-05, 'tendrìa_O': 3.3215970238490664e-05, 'oían_O': 3.3215970238490664e-05, 'molestaban_O': 3.3215970238490664e-05, 'incumplimiento_O': 3.3215970238490664e-05, 'preguntarnos_O': 3.3215970238490664e-05, 'queríamos_O': 3.3215970238490664e-05, 'constar_O': 3.3215970238490664e-05, 'alimento_O': 3.3215970238490664e-05, 'dijimos_O': 4.9823955357735996e-05, 'elaboraran_O': 3.3215970238490664e-05, 'sal.En_O': 3.3215970238490664e-05, 'gorda_O': 3.3215970238490664e-05, 'observación_O': 3.3215970238490664e-05, 'disculparon_O': 3.3215970238490664e-05, 'rehacerlo_O': 3.3215970238490664e-05, 'evidenciar_O': 3.3215970238490664e-05, 'presta_O': 3.3215970238490664e-05, 'solicitudes_O': 3.3215970238490664e-05, 'distribuidos_O': 3.3215970238490664e-05, 'cutrez_O': 3.3215970238490664e-05, 'rectificar_O': 3.3215970238490664e-05, 'golosos_O': 3.3215970238490664e-05, 'cremas_B-positive': 6.184865633794106e-05, 'catalanas_I-positive': 6.351020926613953e-05, 'tiramisú_B-positive': 6.184865633794106e-05, 'antología_O': 3.3215970238490664e-05, 'tranquilos_O': 3.3215970238490664e-05, 'intimidad_O': 3.3215970238490664e-05, 'Hubiera_O': 3.3215970238490664e-05, 'mejorado_O': 3.3215970238490664e-05, 'predilecto_O': 3.3215970238490664e-05, 'mesa_B-negative': 6.337537233031244e-05, 'confortable_O': 3.3215970238490664e-05, 'chicas_O': 3.3215970238490664e-05, 'comedoras_O': 3.3215970238490664e-05, 'croquetas_B-negative': 6.337537233031244e-05, 'buñuelos_B-negative': 6.337537233031244e-05, 'Primer_B-negative': 6.337537233031244e-05, 'plato_I-negative': 6.379992344009187e-05, 'simplon_O': 3.3215970238490664e-05, 'cuartelero_O': 3.3215970238490664e-05, 'apariencia_B-negative': 6.337537233031244e-05, 'modernidad_O': 3.3215970238490664e-05, 'Apto_O': 3.3215970238490664e-05, 'edades_O': 3.3215970238490664e-05, 'aconsejable_O': 4.9823955357735996e-05, 'adelante_O': 3.3215970238490664e-05, 'sean_O': 3.3215970238490664e-05, 'tengan_O': 3.3215970238490664e-05, 'sed_O': 3.3215970238490664e-05, 'pesimo_O': 3.3215970238490664e-05, 'Recordamos_O': 3.3215970238490664e-05, 'cualquiera_O': 3.3215970238490664e-05, 'judicial_O': 3.3215970238490664e-05, 'obtener_O': 3.3215970238490664e-05, 'identidad_O': 3.3215970238490664e-05, 'envía_O': 3.3215970238490664e-05, 'información_O': 3.3215970238490664e-05, 'internet.Aunque_O': 3.3215970238490664e-05, 'tengamos_O': 3.3215970238490664e-05, 'queremos_O': 3.3215970238490664e-05, 'disculpas_O': 3.3215970238490664e-05, 'Argente_O': 3.3215970238490664e-05, 'molestado_O': 3.3215970238490664e-05, 'individuo_O': 3.3215970238490664e-05, 'objeto_O': 3.3215970238490664e-05, 'honesto_O': 3.3215970238490664e-05, 'respetuoso_O': 3.3215970238490664e-05, 'fines_O': 3.3215970238490664e-05, 'solas_O': 3.3215970238490664e-05, 'premio_O': 3.3215970238490664e-05, 'aprobada_O': 3.3215970238490664e-05, 'Hice_O': 4.9823955357735996e-05, 'llamando_O': 3.3215970238490664e-05, '22:00_O': 3.3215970238490664e-05, 'comentan_O': 3.3215970238490664e-05, 'comienza_O': 3.3215970238490664e-05, 'llevan_O': 3.3215970238490664e-05, 'rodaje_O': 3.3215970238490664e-05, 'agradar_O': 3.3215970238490664e-05, 'ganarse_O': 3.3215970238490664e-05, 'puesta_O': 3.3215970238490664e-05, 'escena_O': 3.3215970238490664e-05, 'vale_O': 4.9823955357735996e-05, 'topico_O': 3.3215970238490664e-05, 'irte_O': 3.3215970238490664e-05, 'bocadillo_O': 3.3215970238490664e-05, 'Pan_B-negative': 6.337537233031244e-05, 'untado_O': 4.9823955357735996e-05, 'convencio_O': 3.3215970238490664e-05, 'hacermelo_O': 3.3215970238490664e-05, 'prohibitivo_O': 3.3215970238490664e-05, 'ganen_O': 3.3215970238490664e-05, 'repita_O': 3.3215970238490664e-05, 'Bacalao_B-neutral': 6.40020480655381e-05, 'al_I-neutral': 6.406149903907751e-05, 'pil_I-neutral': 9.609224855861628e-05, 'aireado_I-neutral': 6.406149903907751e-05, 'objecion_O': 3.3215970238490664e-05, 'Carnes_B-positive': 6.184865633794106e-05, 'pastas_B-positive': 6.184865633794106e-05, 'buenísimas_O': 3.3215970238490664e-05, 'sorrentinos_B-positive': 6.184865633794106e-05, 'frescos_O': 3.3215970238490664e-05, 'gallinas_O': 3.3215970238490664e-05, 'Pescados_B-positive': 6.184865633794106e-05, 'elevados_O': 3.3215970238490664e-05, 'lubina_O': 3.3215970238490664e-05, 'reforma_O': 3.3215970238490664e-05, 'privacidad_O': 3.3215970238490664e-05, 'oyendo_O': 3.3215970238490664e-05, 'cuentas_O': 3.3215970238490664e-05, 'limpieza_B-negative': 6.337537233031244e-05, 'Hotel_O': 4.9823955357735996e-05, 'espacio_B-positive': 9.277298450691158e-05, '15/03_O': 3.3215970238490664e-05, 'creia_O': 3.3215970238490664e-05, 'seguia_O': 3.3215970238490664e-05, 'Sugerencias_B-positive': 6.184865633794106e-05, 'principales_O': 4.9823955357735996e-05, 'claras_O': 3.3215970238490664e-05, 'concisas_O': 3.3215970238490664e-05, '110_O': 3.3215970238490664e-05, 'menudo_O': 3.3215970238490664e-05, 'tira_B-neutral': 6.40020480655381e-05, 'asado_I-neutral': 6.406149903907751e-05, 'Calçots_B-neutral': 6.40020480655381e-05, 'gruesos_O': 3.3215970238490664e-05, 'preparación_O': 3.3215970238490664e-05, 'preparadas_O': 3.3215970238490664e-05, 'defectos_O': 3.3215970238490664e-05, 'bebemos_O': 3.3215970238490664e-05, 'cava_B-positive': 6.184865633794106e-05, 'contrario_O': 3.3215970238490664e-05, 'grata_O': 3.3215970238490664e-05, 'Algunos_O': 4.9823955357735996e-05, 'teniendo_O': 3.3215970238490664e-05, 'presentada_O': 4.9823955357735996e-05, 'traerme_O': 3.3215970238490664e-05, 'cuenta_B-neutral': 6.40020480655381e-05, 'aplicado_O': 3.3215970238490664e-05, 'decirselo_O': 3.3215970238490664e-05, 'arreglada_O': 3.3215970238490664e-05, 'desee_O': 3.3215970238490664e-05, 'ponga_O': 3.3215970238490664e-05, 'manos_O': 3.3215970238490664e-05, 'maitre_O': 3.3215970238490664e-05, 'Restaurente_B-positive': 6.184865633794106e-05, 'montecito_O': 3.3215970238490664e-05, 'botella_B-negative': 6.337537233031244e-05, 'vino_I-negative': 6.379992344009187e-05, '500cc_I-negative': 6.379992344009187e-05, 'escritos_O': 3.3215970238490664e-05, 'gastas_O': 3.3215970238490664e-05, 'cuarentaitantos_O': 3.3215970238490664e-05, 'lambrusco_O': 3.3215970238490664e-05, 'parque_O': 3.3215970238490664e-05, 'Chef_O': 3.3215970238490664e-05, 'cocina.La_O': 3.3215970238490664e-05, 'factura_O': 3.3215970238490664e-05, 'cazón_B-negative': 6.337537233031244e-05, 'bienmesabe_O': 4.9823955357735996e-05, 'TIBURÓN_O': 3.3215970238490664e-05, 'MULTIVITAMÍNICO_O': 3.3215970238490664e-05, 'tomadura_O': 3.3215970238490664e-05, 'reservaba_O': 3.3215970238490664e-05, 'señorita_B-positive': 6.184865633794106e-05, 'estarr_O': 3.3215970238490664e-05, 'JoséEn_O': 3.3215970238490664e-05, 'ventilación_B-positive': 6.184865633794106e-05, 'ocuriría_O': 3.3215970238490664e-05, 'compulsivos_O': 3.3215970238490664e-05, 'lúgubre_O': 3.3215970238490664e-05, 'falto_O': 3.3215970238490664e-05, 'luminosidad_O': 3.3215970238490664e-05, 'pesado_O': 3.3215970238490664e-05, 'urgencia_O': 3.3215970238490664e-05, 'vaciar_O': 3.3215970238490664e-05, 'compartidos_O': 3.3215970238490664e-05, 'queriendo_O': 3.3215970238490664e-05, 'continuamente_O': 3.3215970238490664e-05, 'abudantes_O': 3.3215970238490664e-05, 'apunto_O': 3.3215970238490664e-05, 'agenda_O': 3.3215970238490664e-05, 'agrado_O': 3.3215970238490664e-05, 'horno_I-positive': 6.351020926613953e-05, 'trabajadores_B-positive': 6.184865633794106e-05, 'Rioja_B-positive': 6.184865633794106e-05, 'cosecha_O': 3.3215970238490664e-05, 'Ningún_O': 3.3215970238490664e-05, 'WEB_O': 3.3215970238490664e-05, 'resaltar_O': 3.3215970238490664e-05, 'incluyen_O': 3.3215970238490664e-05, 'IVA_O': 3.3215970238490664e-05, 'obligatorio_O': 3.3215970238490664e-05, 'Extraordinario_O': 3.3215970238490664e-05, 'sabor_B-positive': 6.184865633794106e-05, 'textura_B-positive': 6.184865633794106e-05, 'taco_B-positive': 6.184865633794106e-05, 'rape_I-positive': 6.351020926613953e-05, 'cocido_O': 3.3215970238490664e-05, 'Nefastos_O': 3.3215970238490664e-05, 'recomiendaselo_O': 3.3215970238490664e-05, 'caiga_O': 3.3215970238490664e-05, 'Correcto_O': 3.3215970238490664e-05, 'chef_O': 3.3215970238490664e-05, 'encarga_O': 3.3215970238490664e-05, 'generar_O': 3.3215970238490664e-05, 'amigable_O': 3.3215970238490664e-05, 'actuación_B-positive': 6.184865633794106e-05, 'drag_O': 3.3215970238490664e-05, 'queen_O': 3.3215970238490664e-05, 'entretanida_O': 3.3215970238490664e-05, 'descartamos_O': 3.3215970238490664e-05, 'hermanos_O': 3.3215970238490664e-05, 'teresa_O': 3.3215970238490664e-05, 'san_O': 3.3215970238490664e-05, 'evohe_O': 3.3215970238490664e-05, 'jota_O': 3.3215970238490664e-05, 'originalidad_O': 3.3215970238490664e-05, 'merecen_O': 3.3215970238490664e-05, 'Despensa_O': 3.3215970238490664e-05, 'demuestra_O': 3.3215970238490664e-05, 'Poco_O': 3.3215970238490664e-05, 'serio_O': 3.3215970238490664e-05, 'confirmen_O': 3.3215970238490664e-05, 'disponibilidad_O': 3.3215970238490664e-05, 'cancelarla_O': 3.3215970238490664e-05, 'derivarte_O': 3.3215970238490664e-05, 'Grupo_O': 3.3215970238490664e-05, 'impecables_O': 3.3215970238490664e-05, 'cocina-comedor-servicios_O': 3.3215970238490664e-05, 'desapercibido_O': 3.3215970238490664e-05, 'tercero_O': 3.3215970238490664e-05, 'dia.hemos_O': 3.3215970238490664e-05, 'argentino_O': 3.3215970238490664e-05, 'sobresaliente_O': 4.9823955357735996e-05, 'bien_B-positive': 6.184865633794106e-05, 'situado_I-positive': 6.351020926613953e-05, 'Uy_O': 3.3215970238490664e-05, 'Cena_B-positive': 6.184865633794106e-05, 'insistimos_O': 3.3215970238490664e-05, 'disimulo_O': 3.3215970238490664e-05, 'mirando_O': 3.3215970238490664e-05, 'papeles_O': 3.3215970238490664e-05, 'imprimida_O': 3.3215970238490664e-05, 'Solamente_O': 3.3215970238490664e-05, 'rosado_O': 3.3215970238490664e-05, 'mediano_O': 3.3215970238490664e-05, 'En_B-negative': 6.337537233031244e-05, 'fin_I-negative': 6.379992344009187e-05, 'yo_I-negative': 6.379992344009187e-05, 'no_I-negative': 6.379992344009187e-05, 'volveré_I-negative': 6.379992344009187e-05, 'fabuloso_O': 3.3215970238490664e-05, 'calentar_O': 3.3215970238490664e-05, '180º_O': 3.3215970238490664e-05, 'calientas_O': 3.3215970238490664e-05, 'echas_O': 3.3215970238490664e-05, 'compensa_O': 3.3215970238490664e-05, 'patatas_I-positive': 6.351020926613953e-05, 'preguntaran_O': 3.3215970238490664e-05, 'sorprendió_O': 3.3215970238490664e-05, 'gas_I-negative': 6.379992344009187e-05, 'dijeran_O': 3.3215970238490664e-05, 'acabado_O': 3.3215970238490664e-05, 'responsables_O': 3.3215970238490664e-05, 'restauante_O': 3.3215970238490664e-05, 'refiero_O': 3.3215970238490664e-05, 'pincelada_O': 4.9823955357735996e-05, 'mostaza_O': 3.3215970238490664e-05, 'acompañaba_O': 3.3215970238490664e-05, 'carrillera_O': 4.9823955357735996e-05, 'juntaba_O': 3.3215970238490664e-05, 'mataba_O': 3.3215970238490664e-05, 'sabor_B-negative': 6.337537233031244e-05, 'protagonistas_O': 3.3215970238490664e-05, 'Un_B-positive': 6.184865633794106e-05, '"_I-positive': 9.52653138992093e-05, 'Blue_I-positive': 6.351020926613953e-05, 'chip_I-positive': 6.351020926613953e-05, 'restauración_I-positive': 6.351020926613953e-05, 'Grata_O': 3.3215970238490664e-05, 'retocado_O': 3.3215970238490664e-05, 'Seleccioné_O': 3.3215970238490664e-05, 'pediremos_O': 3.3215970238490664e-05, 'alejados_O': 3.3215970238490664e-05, 'pq_O': 3.3215970238490664e-05, 'jaleo_B-negative': 6.337537233031244e-05, 'aconsejaron_O': 3.3215970238490664e-05, 'Habia_O': 3.3215970238490664e-05, 'preocupándose_O': 3.3215970238490664e-05, 'faltase_O': 3.3215970238490664e-05, 'sintiera_O': 3.3215970238490664e-05, 'defraudar_O': 3.3215970238490664e-05, 'abril_O': 3.3215970238490664e-05, 'Luego_O': 3.3215970238490664e-05, 'calentita_O': 3.3215970238490664e-05, 'relación_B-positive': 6.184865633794106e-05, 'llegaba_O': 3.3215970238490664e-05, 'agobiante_O': 3.3215970238490664e-05, 'diría_O': 3.3215970238490664e-05, 'amante_O': 4.9823955357735996e-05, 'vinos_B-neutral': 6.40020480655381e-05, 'Rioja_I-neutral': 6.406149903907751e-05, 'Riojas_O': 3.3215970238490664e-05, 'madre.han_O': 3.3215970238490664e-05, 'realizado_O': 3.3215970238490664e-05, 'satisfacer_O': 3.3215970238490664e-05, '¨_O': 3.3215970238490664e-05, 'Diario_O': 3.3215970238490664e-05, 'recomendaré_O': 3.3215970238490664e-05, 'atun_B-negative': 6.337537233031244e-05, 'hecho.Aplicaron_O': 3.3215970238490664e-05, 'descuento_B-positive': 6.184865633794106e-05, 'promocional_I-positive': 6.351020926613953e-05, 'ENTRAR_O': 3.3215970238490664e-05, 'VALORAR_O': 3.3215970238490664e-05, 'LOS_O': 3.3215970238490664e-05, 'BOSTEZOS_O': 3.3215970238490664e-05, 'CAMARERA_B-negative': 6.337537233031244e-05, 'discusiones_B-negative': 6.337537233031244e-05, 'tratos_B-negative': 6.337537233031244e-05, 'repartidores_O': 3.3215970238490664e-05, 'representantes_O': 3.3215970238490664e-05, 'respetar_O': 3.3215970238490664e-05, 'barra.Ademas_O': 3.3215970238490664e-05, 'deben_O': 3.3215970238490664e-05, '5,5_O': 3.3215970238490664e-05, 'pincho_O': 3.3215970238490664e-05, 'distan_O': 3.3215970238490664e-05, 'miniatura_O': 3.3215970238490664e-05, 'habiendo_O': 3.3215970238490664e-05, 'confirmado_O': 3.3215970238490664e-05, 'propios_O': 3.3215970238490664e-05, 'sinceros_O': 3.3215970238490664e-05, 'cordiales_O': 3.3215970238490664e-05, 'amiga_O': 3.3215970238490664e-05, 'lugar_B-neutral': 6.40020480655381e-05, 'Mala_O': 3.3215970238490664e-05, 'iluminacion_B-negative': 6.337537233031244e-05, 'insípida_O': 3.3215970238490664e-05, 'toque_O': 4.9823955357735996e-05, 'changet_O': 3.3215970238490664e-05, 'lineas_O': 3.3215970238490664e-05, 'generales_O': 3.3215970238490664e-05, 'gambas_B-negative': 6.337537233031244e-05, 'advertimos_O': 3.3215970238490664e-05, 'gustaba_O': 3.3215970238490664e-05, 'sutil_O': 3.3215970238490664e-05, 'sabía_O': 3.3215970238490664e-05, 'riojana_O': 3.3215970238490664e-05, 'López_O': 3.3215970238490664e-05, 'Madrazo.Estoy_O': 3.3215970238490664e-05, 'apreciaciones_O': 3.3215970238490664e-05, 'preguntándo_O': 3.3215970238490664e-05, 'retórica_O': 3.3215970238490664e-05, 'epseranod_O': 3.3215970238490664e-05, 'cocinada_O': 3.3215970238490664e-05, 'reseñable_O': 3.3215970238490664e-05, '750cc_O': 3.3215970238490664e-05, 'calidad-precioLo_O': 3.3215970238490664e-05, 'Conocimos_O': 3.3215970238490664e-05, 'veraniega_O': 3.3215970238490664e-05, 'volveríamos_O': 3.3215970238490664e-05, 'new_O': 3.3215970238490664e-05, 'york_O': 3.3215970238490664e-05, 'paris_O': 3.3215970238490664e-05, 'fascino_O': 3.3215970238490664e-05, 'chicos_O': 3.3215970238490664e-05, 'haciéndolo_O': 3.3215970238490664e-05, 'creerme_O': 3.3215970238490664e-05, 'Zumo_B-negative': 6.337537233031244e-05, 'precio_B-negative': 6.337537233031244e-05, 'Sevicio_B-negative': 6.337537233031244e-05, 'gente_B-negative': 6.337537233031244e-05, 'tranquila_O': 3.3215970238490664e-05, 'imagino_O': 3.3215970238490664e-05, 'influyó_O': 3.3215970238490664e-05, 'generosas_O': 3.3215970238490664e-05, 'actualmente_O': 3.3215970238490664e-05, 'tomanos_O': 3.3215970238490664e-05, 'sintomático_O': 3.3215970238490664e-05, 'pense_O': 3.3215970238490664e-05, 'comprarla_O': 3.3215970238490664e-05, 'gin-tonic_O': 3.3215970238490664e-05, 'imbatible.Eso_O': 3.3215970238490664e-05, 'gin_O': 3.3215970238490664e-05, 'tonic_O': 3.3215970238490664e-05, 'violeta_O': 4.9823955357735996e-05, 'caramelos_O': 3.3215970238490664e-05, 'esté_O': 3.3215970238490664e-05, 'permitido_O': 3.3215970238490664e-05, 'reservada_O': 3.3215970238490664e-05, 'subido_O': 3.3215970238490664e-05, 'sola_O': 3.3215970238490664e-05, 'entramos_O': 3.3215970238490664e-05, 'salida_O': 3.3215970238490664e-05, 'Luis_O': 3.3215970238490664e-05, 'aproveche_O': 3.3215970238490664e-05, 'Comida_B-neutral': 6.40020480655381e-05, 'correctita_O': 3.3215970238490664e-05, 'Confío_O': 3.3215970238490664e-05, 'Empecemos_O': 3.3215970238490664e-05, 'Siento_O': 3.3215970238490664e-05, 'darme_O': 3.3215970238490664e-05, 'informacion_O': 3.3215970238490664e-05, 'tapeo_O': 3.3215970238490664e-05, 'hare_O': 3.3215970238490664e-05, 'poneis_O': 3.3215970238490664e-05, 'salon_B-negative': 6.337537233031244e-05, 'infiernos_O': 3.3215970238490664e-05, 'adjetivos_O': 3.3215970238490664e-05, 'gas_O': 4.9823955357735996e-05, 'Vos_O': 3.3215970238490664e-05, 'Somos_O': 3.3215970238490664e-05, 'Plato_B-negative': 6.337537233031244e-05, 'jamón_I-negative': 6.379992344009187e-05, '5J_O': 3.3215970238490664e-05, 'entendemos_O': 3.3215970238490664e-05, 'perdió_O': 3.3215970238490664e-05, 'Tortitas_B-negative': 6.337537233031244e-05, 'Camarón_I-negative': 6.379992344009187e-05, 'parecidas_O': 3.3215970238490664e-05, 'Huelva_O': 3.3215970238490664e-05, 'seguna_O': 3.3215970238490664e-05, 'malentendido_O': 3.3215970238490664e-05, 'localización_B-positive': 6.184865633794106e-05, 'facilmente_O': 3.3215970238490664e-05, 'braseado_I-negative': 6.379992344009187e-05, 'Alguno_O': 3.3215970238490664e-05, 'pese_O': 3.3215970238490664e-05, 'Estaria_O': 3.3215970238490664e-05, 'argentina_I-negative': 6.379992344009187e-05, 'Deben_O': 3.3215970238490664e-05, 'control_O': 3.3215970238490664e-05, 'base_B-negative': 6.337537233031244e-05, 'pizz_I-negative': 6.379992344009187e-05, 'utlilizan_O': 3.3215970238490664e-05, 'lujos_O': 3.3215970238490664e-05, 'pasable_O': 3.3215970238490664e-05, 'bravas_I-positive': 6.351020926613953e-05, 'jueves_O': 3.3215970238490664e-05, 'unicos_O': 3.3215970238490664e-05, 'lentísimo_O': 3.3215970238490664e-05, 'Platos_B-negative': 6.337537233031244e-05, 'sobrevolando_O': 3.3215970238490664e-05, 'cabezas_O': 3.3215970238490664e-05, 'limón_O': 3.3215970238490664e-05, 'querian_O': 3.3215970238490664e-05, 'estafar_O': 3.3215970238490664e-05, 'relajante_O': 3.3215970238490664e-05, 'colores_O': 3.3215970238490664e-05, 'sobrio_O': 3.3215970238490664e-05, 'rincones_O': 3.3215970238490664e-05, 'Pescado_B-positive': 6.184865633794106e-05, 'conocemos_O': 3.3215970238490664e-05, 'pensabamos_O': 3.3215970238490664e-05, 'Apruebo_O': 3.3215970238490664e-05, 'mejoran_O': 3.3215970238490664e-05, 'calsot_B-negative': 6.337537233031244e-05, 'constructiva_O': 3.3215970238490664e-05, 'acertadisimas_O': 3.3215970238490664e-05, 'riquísimo_O': 3.3215970238490664e-05, 'Fiel_O': 3.3215970238490664e-05, 'tradición_O': 3.3215970238490664e-05, 'dejé_O': 3.3215970238490664e-05, 'dieran_O': 3.3215970238490664e-05, 'dirección_B-positive': 6.184865633794106e-05, 'serviría_O': 3.3215970238490664e-05, 'TROCITO_O': 3.3215970238490664e-05, 'CÁDIZ_O': 3.3215970238490664e-05, 'EN_O': 3.3215970238490664e-05, 'CORAZÓN_O': 3.3215970238490664e-05, 'ZARAGOZA_O': 3.3215970238490664e-05, 'parametros_O': 3.3215970238490664e-05, 'normales_O': 3.3215970238490664e-05, 'Todos_O': 3.3215970238490664e-05, 'jefa_O': 3.3215970238490664e-05, 'Eduardo_B-positive': 6.184865633794106e-05, 'Orús_I-positive': 6.351020926613953e-05, 'fogones_O': 3.3215970238490664e-05, 'Agosto_O': 3.3215970238490664e-05, 'fideus_O': 3.3215970238490664e-05, 'rossejats_O': 3.3215970238490664e-05, 'tronco_O': 3.3215970238490664e-05, 'donostiarra_O': 3.3215970238490664e-05, 'famoso_O': 3.3215970238490664e-05, 'tiramisú_O': 3.3215970238490664e-05, 'caramelizada_O': 3.3215970238490664e-05, 'iría_O': 3.3215970238490664e-05, 'novia_O': 3.3215970238490664e-05, 'brasas_B-positive': 6.184865633794106e-05, 'parriladas_B-positive': 6.184865633794106e-05, 'Atentos_O': 3.3215970238490664e-05, 'educados_O': 3.3215970238490664e-05, 'discretos_O': 3.3215970238490664e-05, 'cuantos_O': 3.3215970238490664e-05, 'despedirnos_O': 3.3215970238490664e-05, 'acudieron_O': 3.3215970238490664e-05, 'darnos_O': 3.3215970238490664e-05, 'adios_O': 3.3215970238490664e-05, 'Prometimos_O': 3.3215970238490664e-05, 'habremos_O': 3.3215970238490664e-05, 'informado_O': 3.3215970238490664e-05, 'almuerzo_O': 3.3215970238490664e-05, 'redondeó_O': 3.3215970238490664e-05, 'inolvidable_O': 3.3215970238490664e-05, 'limpio_B-positive': 6.184865633794106e-05, 'menus_B-positive': 6.184865633794106e-05, '#UNK#O': 1.6607985119245332e-05, '#UNK#B-positive': 3.092432816897053e-05, '#UNK#B-negative': 3.168768616515622e-05, '#UNK#B-neutral': 3.200102403276905e-05, '#UNK#I-positive': 3.175510463306976e-05, '#UNK#I-negative': 3.189996172004593e-05, '#UNK#I-neutral': 3.2030749519538757e-05}

new_emission_para = {}
for key, value in emission_para.items():
    if key.startswith("#UNK#"):
        new_key = ("#UNK#", key.split("#UNK#")[1])
    else:
        new_key = tuple(key.split('_'))
    new_emission_para[new_key] = value

# print(new_emission_para)

In [ ]:
transition_para = {('O', 'START'): 0.9289176090468497, ('O', 'O'): 0.885720190121926, ('B-positive', 'O'): 0.03651646685958385, ('O', 'B-positive'): 0.871551724137931, ('STOP', 'O'): 0.06344067504735663, ('START', 'STOP'): 1.0, ('B-negative', 'O'): 0.012350403562482605, ('O', 'B-negative'): 0.8110236220472441, ('B-positive', 'START'): 0.0522911051212938, ('B-neutral', 'O'): 0.002170336402142332, ('I-neutral', 'B-neutral'): 0.22388059701492538, ('I-neutral', 'I-neutral'): 0.6829268292682927, ('O', 'I-neutral'): 0.3488372093023256, ('I-positive', 'B-positive'): 0.1171875, ('I-positive', 'I-positive'): 0.5774193548387097, ('O', 'I-positive'): 0.4267515923566879, ('O', 'B-neutral'): 0.7916666666666666, ('I-negative', 'B-negative'): 0.17989417989417988, ('O', 'I-negative'): 0.39766081871345027, ('B-negative', 'START'): 0.014223194748358862, ('I-negative', 'I-negative'): 0.6058823529411764, ('B-neutral', 'START'): 0.005002779321845469, ('STOP', 'B-positive'): 0.009523809523809525, ('STOP', 'B-negative'): 0.01084010840108401, ('B-neutral', 'B-positive'): 0.0036231884057971015, ('STOP', 'I-positive'): 0.006756756756756757, ('B-positive', 'B-positive'): 0.0029411764705882353}

transition_dict = {}

for (to_state, from_state), probability in transition_para.items():
    if from_state not in transition_dict:
        transition_dict[from_state] = {}
    transition_dict[from_state][to_state] = probability

# print(transition_dict)

In [ ]:
def viterbi(data, t_params, e_params, word_set):
    # print("Beginning viterbi for", data)
    n = len(data)
    # Includes only possible labels for the words in our dataset: ie. excludes 'START' and 'STOP'
    labels = ['O', 'B-positive', 'B-neutral', 'B-negative', 'I-positive', 'I-neutral', 'I-negative', 'START']

    # Initialization.
    # cache is a list, where the list index represents the current position in the data
    # each element consists of a dictionary of possible labels at that position
    #IMPORTANT
    # each possible label maps to a list of format [probability up to this point, parent label]
    # 0: START 1: 1st word ... n: nth word n+1: END --> size = n+2

    n_inf = -math.inf
    cache = [{'START':[n_inf, None],
    'STOP': [n_inf, None], 
    'O':[n_inf, None],
    'B-positive':[n_inf, None],
    'B-neutral':[n_inf, None],
    'B-negative':[n_inf, None],
    'I-positive':[n_inf, None],
    'I-neutral':[n_inf, None],
    'I-negative':[n_inf, None]} for i in range(n+2)]

    cache[0]['START'][0] = 0 # Technically this should be 0

    for j in range(0, n):
        next_word = data[j]
        # print("\n\n Step", j+1, "current word is:", next_word)
        # Iterate over all of the current labels in this step.
        for u in labels:
            # print("\n Checking u: ", u)
            maximum = n_inf
            max_label = None
            # Because we want to find the maximum v
            for v in labels:
                # If any of the observed probabilities are 0, we should skip because that is an impossible path
                if (cache[j][v][0] == n_inf or t_params[v][u] == 0):
                    # print(v, "to", u, "is impossible")
                    continue
                prev_cached_value = cache[j][v][0]
                if next_word in word_set:
                    if next_word not in e_params[u].keys():
                        # print("impossible emission: word in training set yet not seen for this label.")
                        continue
                    else:
                        emission_prob = e_params[u][next_word]
                else:
                    # print("not in training set. using the #UNK# probability")
                    emission_prob = e_params[u]['#UNK#']
                transmission_prob = t_params[v][u]
                # print("cache:", prev_cached_value, "emiss:", emission_prob, "trans:", transmission_prob)
                prob = prev_cached_value + math.log(emission_prob) + math.log(transmission_prob)
                # print(v, 'to', u, 'emitting', next_word, 'has prob', prob)
                if maximum < prob:
                    maximum = prob
                    max_label = v
            
            # print('best v is', max_label, 'with prob', maximum)
            if maximum == n_inf:
                continue
            cache[j+1][u][0] = maximum
            cache[j+1][u][1] = max_label
    
    # Final Step (n+1)
    maximum = n_inf
    max_label = None
    for v in labels:
        prev_cached_value = cache[n][v][0]
        transmission_prob = t_params[v]['STOP']
        if (prev_cached_value == 0 or transmission_prob == 0):
            continue
        prob = prev_cached_value + math.log(transmission_prob)
        if maximum < prob:
            maximum = prob
            max_label = v

    if maximum != n_inf:
        cache[n+1]['STOP'][0] = maximum
        cache[n+1]['STOP'][1] = max_label

    # for i in range(len(cache)):
    #     print(i, cache[i])
    # print('\n')
    
    # Finding the most probable labels.
    output = ['' for i in range(n)]

    # Default to "O" if emission isn't possible.
    if max_label == None:
        max_label = "O"

    # for the n-1th to 1st word
    for j in range(n+1, 1, -1):
        # print("step", j, "old max:", max_label, "in cache:", cache[j])
        max_label = cache[j][max_label][1]
        if max_label == None:
            max_label = "O"
        output[j-2] = max_label
    
    return output

In [ ]:
def viterbi_loop(separated, t_params, e_params, word_set):
    final = []
    for doc in separated:
        final.append(viterbi(doc, t_params, e_params, word_set))
    return final

In [ ]:
def get_training_set_words(data):
    words = set()
    for i in data:
        if len(data) > 1:
            words.add(i[0])
    return words

In [ ]:
def read_dev(path):
  out = [[]]
  f = open(path, "r", encoding="utf-8")
  lines_in = f.readlines()
  for word in lines_in:
    if word == "\n":
      out.append([])
    else:
      out[-1].append(word.rstrip())
  return out[:-1]

In [ ]:
def count_tags(training_set):
  unique_tag_count = {'START':0,'O':0,'B-positive':0,'B-neutral':0,'B-negative':0,'I-positive':0,'I-neutral':0,'I-negative':0,'STOP':0}
  #to store the unique tag + the total count of each from the training dataset
  for data_pair in training_set:
    if len(data_pair) > 1:
      if data_pair[1] in unique_tag_count.keys():
        unique_tag_count[data_pair[1]] += 1
    elif len(data_pair)==1:
      unique_tag_count['START'] += 1 #for every newlin \n add to start and stop
      unique_tag_count['STOP'] += 1
  return unique_tag_count

In [ ]:
def count_tag_words(training_set):
  # We want to estimate the x given the ys
  # From the above code block we have count(y)
  label_generate_all = {'O':{},'B-positive':{},'B-neutral':{},'B-negative':{},'I-positive':{},'I-neutral':{},'I-negative':{}}
  for data in training_set:
    if len(data) > 1:
      try:
        if data[0] not in label_generate_all[data[1]].keys():
          label_generate_all[data[1]][data[0]] = 1
        else:
          label_generate_all[data[1]][data[0]] += 1 
      except KeyError:
        print("error", KeyError, data)
  return label_generate_all

In [ ]:
def output_prediction(prediction, data, path):
    assert(len(prediction) == len(data))
    file = open(path, "w", encoding="utf-8")
    n = len(data)
    print("Writing", n, "lines")
    for i in range(n):
        assert(len(data[i]) == len(prediction[i]))
        m = len(data[i])
        for j in range(m):
            file.write(data[i][j] + " " + prediction[i][j] + "\n")
        file.write("\n")
    print("Wrote predictions to", path)

In [ ]:
def run_viterbi(training_path, test_path, output_path,mode,alpha_sm=0.0): #mode ES or RU
    if mode =="ES":
        stopwords_a = read_stopwords(STOP_words_ES_file_path)
    elif mode =="RU":
        stopwords_a = read_stopwords(STOP_words_RU_file_path)
    train = remove_stopwords(training_path,stopwords_a)
    train_words = get_training_set_words(train)
    test = read_dev(test_path)
    tags = count_tags(train)
    tag_words = count_tag_words(train)
    
    transmission_counts = count_transmissions(train)
    t_params = estimate_transmission_parameters(transmission_counts, tags)
    t_params = smooth_labels_transmission(t_params,alpha_sm=alpha_sm)
    e_params = estimate_emission_parameters_with_unk(tags, tag_words)
    e_params = smooth_labels_emission(e_params,alpha_sm=alpha_sm)
    prediction = viterbi_loop(test, t_params, e_params, train_words)
    output_prediction(prediction, test, output_path)

In [ ]:
def run_everything(training_path, test_path, output_path, mode,alpha_sm=0.1):
    run_viterbi(training_path,test_path,output_path,mode,alpha_sm=alpha_sm)

In [59]:
if __name__ == "__main__":
    n = len(sys.argv)
    if os.path.exists(RU_test_dev_write):
        os.remove(RU_test_dev_write)
    if os.path.exists(RU_test_write):
        os.remove(RU_test_write)
    if os.path.exists(ES_test_dev_write):
        os.remove(ES_test_dev_write)
    if os.path.exists(ES_test_write):
        os.remove(ES_test_write)

    if n == 1:
        #RU
        run_everything(RU_train,RU_test_dev,RU_test_dev_write,"RU")
        run_everything(RU_train,RU_test,RU_test_write,"RU")
        #ES
        run_everything(ES_train,ES_test_dev,ES_test_dev_write, "ES")
        run_everything(ES_train,ES_test,ES_test_write,"ES")
    else:
        if n == 4:
            run_everything(sys.argv[1], sys.argv[2], sys.argv[3])
        elif n == 2:
            factor = float(sys.argv[1])
            #RU
            run_everything(RU_train,RU_test_dev,RU_test_dev_write,"RU",alpha_sm=factor)
            run_everything(RU_train,RU_test,RU_test_write,"RU",alpha_sm=factor)
            #ES
            run_everything(ES_train,ES_test_dev,ES_test_dev_write, "ES", alpha_sm=factor)
            run_everything(ES_train,ES_test,ES_test_write,"ES",alpha_sm=factor)
        else:
            print("usage: python part4.py [train_path] [test_path] [output_path]")
    
    python_cmd = "python3"
    if os.name != "nt":
        python_cmd = "python"
    #evaluation portion
    print('The scores for the russian dataset is:')
    os.system(f'{python_cmd} EvalScript/evalResult.py RU/dev.out RU/dev.p4.out')
    print('The scores for the ES dataset is:')
    os.system(f'{python_cmd} EvalScript/evalResult.py ES/dev.out ES/dev.p4.out')


usage: python part4.py [train_path] [test_path] [output_path]
The scores for the russian dataset is:
The scores for the ES dataset is:
